In [ ]:
%%writefile submission.py

import random
import numpy as np
import pandas as pd
import collections
from scipy.stats import beta
from collections import Counter
import re
from collections import defaultdict
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import secrets
import math
from operator import itemgetter
import pickle

import operator
import cmath
from typing import List
from collections import namedtuple
import traceback
import sys


basis = np.array(
    [1, cmath.exp(2j * cmath.pi * 1 / 3), cmath.exp(2j * cmath.pi * 2 / 3)]
)


HistMatchResult = namedtuple("HistMatchResult", "idx length")


def find_all_longest(seq, max_len=None) -> List[HistMatchResult]:
    """
    Find all indices where end of `seq` matches some past.
    """
    result = []

    i_search_start = len(seq) - 2

    while i_search_start > 0:
        i_sub = -1
        i_search = i_search_start
        length = 0

        while i_search >= 0 and seq[i_sub] == seq[i_search]:
            length += 1
            i_sub -= 1
            i_search -= 1

            if max_len is not None and length > max_len:
                break

        if length > 0:
            result.append(HistMatchResult(i_search_start + 1, length))

        i_search_start -= 1

    result = sorted(result, key=operator.attrgetter("length"), reverse=True)

    return result


def probs_to_complex(p):
    return p @ basis


def _fix_probs(probs):
    """
    Put probs back into triangle. Sometimes this happens due to rounding errors or if you
    use complex numbers which are outside the triangle.
    """
    if min(probs) < 0:
        probs -= min(probs)

    probs /= sum(probs)

    return probs


def complex_to_probs(z):
    probs = (2 * (z * basis.conjugate()).real + 1) / 3
    probs = _fix_probs(probs)
    return probs


def z_from_action(action):
    return basis[action]


def sample_from_z(z):
    probs = complex_to_probs(z)
    return np.random.choice(3, p=probs)


def bound(z):
    return probs_to_complex(complex_to_probs(z))


def norm(z):
    return bound(z / abs(z))


class Pred:
    def __init__(self, *, alpha):
        self.offset = 0
        self.alpha = alpha
        self.last_feat = None

    def train(self, target):
        if self.last_feat is not None:
            offset = target * self.last_feat.conjugate()   # fixed

            self.offset = (1 - self.alpha) * self.offset + self.alpha * offset

    def predict(self, feat):
        """
        feat is an arbitrary feature with a probability on 0,1,2
        anything which could be useful anchor to start with some kind of sensible direction
        """
        feat = norm(feat)

        # offset = mean(target - feat)
        # so here we see something like: result = feat + mean(target - feat)
        # which seem natural and accounts for the correlation between target and feat
        # all RPSContest bots do no more than that, just in a hidden way
        
        result = feat * self.offset

        self.last_feat = feat

        return result
    
    
class BaseAgent:
    def __init__(self):
        self.my_hist = []
        self.opp_hist = []
        self.my_opp_hist = []
        self.outcome_hist = []
        self.step = None

    def __call__(self, obs, conf):
        try:
            if obs.step == 0:
                action = np.random.choice(3)
                self.my_hist.append(action)
                return action

            self.step = obs.step

            opp = int(obs.lastOpponentAction)
            my = self.my_hist[-1]

            self.my_opp_hist.append((my, opp))
            self.opp_hist.append(opp)

            outcome = {0: 0, 1: 1, 2: -1}[(my - opp) % 3]
            self.outcome_hist.append(outcome)

            action = self.action()

            self.my_hist.append(action)

            return action
        except Exception:
            traceback.print_exc(file=sys.stderr)
            raise

    def action(self):
        pass


class Geometry_Agent(BaseAgent):
    def __init__(self, alpha=0.01):
        super().__init__()

#         self.predictor = Pred(alpha=alpha)
        self.predictor = Pred(alpha=0)

    def action(self):
        self.train()

        pred = self.preds()

        return_action = sample_from_z(pred)

        return return_action

    def train(self):
        last_beat_opp = z_from_action((self.opp_hist[-1] + 1) % 3)
        self.predictor.train(last_beat_opp)

    def preds(self):
        hist_match = find_all_longest(self.my_opp_hist, max_len=20)

        if not hist_match:
             return 0

        feat = z_from_action(self.opp_hist[hist_match[0].idx])

        pred = self.predictor.predict(feat)

        return pred
    
    
geometry_agent = Geometry_Agent()


    

# END OF GEOMETRY AGENT


ages = [1000, 100, 10, 5, 2, 1]
code = compile(
    """
import random

number_of_predictors = 60
number_of_metapredictors = 4


if not input:
    limits = [50,20,6]
    beat={'R':'P','P':'S','S':'R'}
    urmoves=""
    mymoves=""
    DNAmoves=""
    outputs=[random.choice("RPS")]*number_of_metapredictors
    predictorscore1=[3]*number_of_predictors
    predictorscore2=[3]*number_of_predictors
    predictorscore3=[3]*number_of_predictors
    predictorscore4=[3]*number_of_predictors
    nuclease={'RP':'a','PS':'b','SR':'c','PR':'d','SP':'e','RS':'f','RR':'g','PP':'h','SS':'i'}
    length=0
    predictors=[random.choice("RPS")]*number_of_predictors
    metapredictors=[random.choice("RPS")]*number_of_metapredictors
    metapredictorscore=[3]*number_of_metapredictors
else:

    for i in range(number_of_predictors):
        #metapredictor 1
        predictorscore1[i]*=0.8
        predictorscore1[i]+=(input==predictors[i])*3
        predictorscore1[i]-=(input==beat[beat[predictors[i]]])*3
        #metapredictor 2: beat metapredictor 1 (probably contains a bug)
        predictorscore2[i]*=0.8
        predictorscore2[i]+=(output==predictors[i])*3
        predictorscore2[i]-=(output==beat[beat[predictors[i]]])*3
        #metapredictor 3
        predictorscore3[i]+=(input==predictors[i])*3
        if input==beat[beat[predictors[i]]]:
            predictorscore3[i]=0
            #metapredictor 4: beat metapredictor 3 (probably contains a bug)
            predictorscore4[i]+=(output==predictors[i])*3
        if output==beat[beat[predictors[i]]]:
            predictorscore4[i]=0

    for i in range(number_of_metapredictors):
        metapredictorscore[i]*=0.96
        metapredictorscore[i]+=(input==metapredictors[i])*3
        metapredictorscore[i]-=(input==beat[beat[metapredictors[i]]])*3


    #Predictors 1-18: History matching
    urmoves+=input
    mymoves+=output
    DNAmoves+=nuclease[input+output]
    length+=1

    for z in range(3):
        limit = min([length,limits[z]])
        j=limit
        while j>=1 and not DNAmoves[length-j:length] in DNAmoves[0:length-1]:
            j-=1
        if j>=1:
            i = DNAmoves.rfind(DNAmoves[length-j:length],0,length-1) 
            predictors[0+6*z] = urmoves[j+i] 
            predictors[1+6*z] = beat[mymoves[j+i]] 
        j=limit
        while j>=1 and not urmoves[length-j:length] in urmoves[0:length-1]:
            j-=1
        if j>=1:
            i = urmoves.rfind(urmoves[length-j:length],0,length-1) 
            predictors[2+6*z] = urmoves[j+i] 
            predictors[3+6*z] = beat[mymoves[j+i]] 
        j=limit
        while j>=1 and not mymoves[length-j:length] in mymoves[0:length-1]:
            j-=1
        if j>=1:
            i = mymoves.rfind(mymoves[length-j:length],0,length-1) 
            predictors[4+6*z] = urmoves[j+i] 
            predictors[5+6*z] = beat[mymoves[j+i]]
    #Predictor 19,20: RNA Polymerase
    L=len(mymoves)
    i=DNAmoves.rfind(DNAmoves[L-j:L-1],0,L-2)
    while i==-1:
        j-=1
        i=DNAmoves.rfind(DNAmoves[L-j:L-1],0,L-2)
        if j<2:
            break
    if i==-1 or j+i>=L:
        predictors[18]=predictors[19]=random.choice("RPS")
    else:
        predictors[18]=beat[mymoves[j+i]]
        predictors[19]=urmoves[j+i]

    #Predictors 21-60: rotations of Predictors 1:20
    for i in range(20,60):
        predictors[i]=beat[beat[predictors[i-20]]] #Trying to second guess me?

    metapredictors[0]=predictors[predictorscore1.index(max(predictorscore1))]
    metapredictors[1]=beat[predictors[predictorscore2.index(max(predictorscore2))]]
    metapredictors[2]=predictors[predictorscore3.index(max(predictorscore3))]
    metapredictors[3]=beat[predictors[predictorscore4.index(max(predictorscore4))]]

    #compare predictors
output = beat[metapredictors[metapredictorscore.index(max(metapredictorscore))]]
if max(metapredictorscore)<0:
    output = beat[random.choice(urmoves)]
""", '<string>', 'exec')
code_dllu = compile(
    """
import random
numPre = 30
numMeta = 6
if not input:
    limit = 8
    beat={'R':'P','P':'S','S':'R'}
    moves=['','','','']
    pScore=[[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre,[5]*numPre]
    centrifuge={'RP':0,'PS':1,'SR':2,'PR':3,'SP':4,'RS':5,'RR':6,'PP':7,'SS':8}
    centripete={'R':0,'P':1,'S':2}
    soma = [0,0,0,0,0,0,0,0,0];
    rps = [1,1,1];
    a="RPS"
    best = [0,0,0];
    length=0
    p=[random.choice("RPS")]*numPre
    m=[random.choice("RPS")]*numMeta
    mScore=[5,2,5,2,4,2]
else:
    for i in range(numPre):
        pp = p[i]
        bpp = beat[pp]
        bbpp = beat[bpp]
        pScore[0][i]=0.9*pScore[0][i]+((input==pp)-(input==bbpp))*3
        pScore[1][i]=0.9*pScore[1][i]+((output==pp)-(output==bbpp))*3
        pScore[2][i]=0.87*pScore[2][i]+(input==pp)*3.3-(input==bpp)*1.2-(input==bbpp)*2.3
        pScore[3][i]=0.87*pScore[3][i]+(output==pp)*3.3-(output==bpp)*1.2-(output==bbpp)*2.3
        pScore[4][i]=(pScore[4][i]+(input==pp)*3)*(1-(input==bbpp))
        pScore[5][i]=(pScore[5][i]+(output==pp)*3)*(1-(output==bbpp))
    for i in range(numMeta):
        mScore[i]=0.96*(mScore[i]+(input==m[i])-(input==beat[beat[m[i]]]))
    soma[centrifuge[input+output]] +=1;
    rps[centripete[input]] +=1;
    moves[0]+=str(centrifuge[input+output])
    moves[1]+=input
    moves[2]+=output
    length+=1
    for y in range(3):
        j=min([length,limit])
        while j>=1 and not moves[y][length-j:length] in moves[y][0:length-1]:
            j-=1
        i = moves[y].rfind(moves[y][length-j:length],0,length-1)
        p[0+2*y] = moves[1][j+i] 
        p[1+2*y] = beat[moves[2][j+i]]
    j=min([length,limit])
    while j>=2 and not moves[0][length-j:length-1] in moves[0][0:length-2]:
        j-=1
    i = moves[0].rfind(moves[0][length-j:length-1],0,length-2)
    if j+i>=length:
        p[6] = p[7] = random.choice("RPS")
    else:
        p[6] = moves[1][j+i] 
        p[7] = beat[moves[2][j+i]]
        
    best[0] = soma[centrifuge[output+'R']]*rps[0]/rps[centripete[output]]
    best[1] = soma[centrifuge[output+'P']]*rps[1]/rps[centripete[output]]
    best[2] = soma[centrifuge[output+'S']]*rps[2]/rps[centripete[output]]
    p[8] = p[9] = a[best.index(max(best))]
    
    for i in range(10,numPre):
        p[i]=beat[beat[p[i-10]]]
        
    for i in range(0,numMeta,2):
        m[i]=       p[pScore[i  ].index(max(pScore[i  ]))]
        m[i+1]=beat[p[pScore[i+1].index(max(pScore[i+1]))]]
output = beat[m[mScore.index(max(mScore))]]
if max(mScore)<0.07 or random.randint(3,40)>length:
    output=beat[random.choice("RPS")]
""", '<string>', 'exec')

def get_score(S, A1, A2):
    return (S + A1 - A2 + 1) % S - 1

class Mob():
    def __init__(self, counts, l_counts, scores, waypoints, reset, a, b, losses, gap, timing, sensitivity, decay, cutoff, stages, change):
        self.counts = counts
        self.gap = gap
        self.temp_counts = counts
        self.losses = losses
        self.loss_counts = l_counts
        self.scores = scores
        self.reset = reset
        self.waypoint = waypoints
        self.sensitivity = sensitivity
        self.timing = timing
        self.r_multiplier = 1
        self.competitors = []
        self.cutoff = cutoff
        self.stages = stages
        self.change = change
        self.checker = gap*6
        self.base_sense = sensitivity
        ## specify initial values
        self.init = {"beta_0": 0,
                "beta_1": 0,
                "tau": 2}

        ## specify hyper parameters
        self.hypers = {"mu_0": 0,
                 "tau_0": 1,
                 "mu_1": 0,
                 "tau_1": 1,
                 "alpha": 2,
                 "beta": 1}
        
        self.w = a
        self.l = b
        self.d = None
        
        self.beta_0 = None
        self.beta_1 = None
        self.tau = None
        
        self.decay = decay
        self.gibbs_value = None
        return
    
    #Initialize values for agents
    def initialize(self, n_agents):
        self.counts = [0 for col in range(n_agents)]
        self.temp_counts = [0 for col in range(n_agents)]
        self.w = [1 for agent in range(n_agents)]
        self.l = [0 for agent in range(n_agents)]
        self.d = [[1,1,1] for agent in range(n_agents)]
        self.loss_counts = [[0 for col in range(self.losses)] for col in range(n_agents)]
        self.scores = [[0.0 for col in range(self.losses)] for col in range(n_agents)]
        self.waypoint = [1000 for col in range(n_agents)]
        self.reset = [False for col in range(n_agents)]
        self.gaps = [self.gap for col in range(n_agents)]
        self.competitors = [1 for agent in range(n_agents)]
        self.gibbs_value = [[] for agent in range(len(self.competitors))]
        self.beta_0 = [self.init["beta_0"] for agent in range(len(self.competitors))]
        self.beta_1 = [self.init["beta_1"] for agent in range(len(self.competitors))]
        self.tau = [self.init["tau"] for agent in range(len(self.competitors))]
        return
    
    def select_agent(self):
        n_agents = len(self.counts)
        all_results = [np.random.dirichlet(x, size=1) for x in self.d]
        best = 0
        score = 0
        if self.stages == True:
            for i in range(len(self.competitors)):
                if self.competitors[i] == 1:
                    gb = 0
                    if len(self.gibbs_value[i])>1:
                        gb = sum(self.gibbs_value[i])/len(self.gibbs_value[i])
                    if all_results[i][0][0]*(self.w[i]/(self.w[i]+self.l[i]))+2*gb > score:
                        score = all_results[i][0][0]*(self.w[i]/(self.w[i]+self.l[i]))+2*gb
                        best = i
            if score > 0:
                return best
        score = 0
        best = 0
        for i in range(len(self.competitors)):
            if self.competitors[i] == 1:
                gb = 0
                if len(self.gibbs_value[i])>1:
                    gb = sum(self.gibbs_value[i])/len(self.gibbs_value[i])
                if all_results[i][0][0]+1.5*gb > score:
                    score = all_results[i][0][0]+1.5*gb
                    best = i
        return best
    
    def sample_beta_0(self, y, x, beta_1, tau, mu_0, tau_0):
        N = len(y)
        precision = tau_0 + tau * N
        mean = tau_0 * mu_0 + tau * np.sum(y - beta_1 * x)
        mean /= precision
        return np.random.normal(mean, 1 / np.sqrt(precision))
    
    def sample_beta_1(self, y, x, beta_0, tau, mu_1, tau_1):
        N = len(y)
        precision = tau_1 + tau * np.sum(x * x)
        mean = tau_1 * mu_1 + tau * np.sum( (y - beta_0) * x)
        mean /= precision
        return np.random.normal(mean, 1 / np.sqrt(precision))
    
    def sample_tau(self, y, x, beta_0, beta_1, alpha, beta):
        N = len(y)
        alpha_new = alpha + N / 2
        resid = y - beta_0 - beta_1 * x
        beta_new = beta + np.sum(resid * resid) / 2
        return np.random.gamma(alpha_new, 1 / beta_new)
    
    def gibbs(self, a, b, agent): 
        y = np.array(a)
        x = np.array(b)
        for it in range(100):
            self.beta_0[agent] = self.sample_beta_0(y, x, self.beta_1[agent], self.tau[agent], self.hypers["mu_0"], self.hypers["tau_0"])
            self.beta_1[agent] = self.sample_beta_1(y, x, self.beta_0[agent], self.tau[agent], self.hypers["mu_1"], self.hypers["tau_1"])
            self.tau[agent] = self.sample_tau(y, x, self.beta_0[agent], self.beta_1[agent], self.hypers["alpha"], self.hypers["beta"])
        
        self.gibbs_value[agent].append(self.beta_1[agent])
    
    def select_top(self, n):
        all_results = [np.random.dirichlet(x, size=1) for x in self.d]
        top = sorted(range(len(all_results)), key = lambda sub: all_results[sub][0][0])[-n:]
        return top
    
    def reset_all(self, n, r):
        global history, agents, hst, rwd
        if n > self.checker and n < 700 and r < -20:
            history = [[] for _ in range(len(self.competitors))]
            hst = [[] for _ in range(len(self.competitors))]
            rwd = [[] for _ in range(len(self.competitors))]
            self.checker += self.gap*10
            self.gap = n + 50
            self.sensitivity = self.base_sense
            self.counts = [0 for col in range(len(self.competitors))]
            self.temp_counts = [0 for col in range(len(self.competitors))]
            self.w = [1 for agent in range(len(self.competitors))]
            self.l = [0 for agent in range(len(self.competitors))]
            self.d = [[1,1,1] for agent in range(len(self.competitors))]
            self.loss_counts = [[0 for col in range(self.losses)] for col in range(len(self.competitors))]
            self.scores = [[0.0 for col in range(self.losses)] for col in range(len(self.competitors))]
            self.competitors = [1 for agent in range(len(self.w))]
            self.gibbs_value = [[] for agent in range(len(self.competitors))]
            self.beta_0 = [self.init["beta_0"] for agent in range(len(self.competitors))]
            self.beta_1 = [self.init["beta_1"] for agent in range(len(self.competitors))]
            self.tau = [self.init["tau"] for agent in range(len(self.competitors))]
            for i in range(len(self.competitors)):
                agents[i].reset()
    
    def dropout(self):
        worst = 999
        loss = 999
        min_counts = 20
        for i in range(len(self.competitors)):
            if self.competitors[i] == 1 and self.counts[i] > min_counts:
                performance = self.d[i][0] / max(self.counts[i],0.1)
                if performance < loss:
                    loss = performance
                    worst = i
        if worst < 999: self.competitors[worst] = 0
        return
    
    def switch(self):
        best = 0
        wins = 0
        worst = 0
        losses = 0
        min_counts = 20
        for i in range(len(self.competitors)):
            if self.competitors[i] == 1 and self.counts[i] > min_counts:
                performance = self.d[i][2] / max(self.counts[i],0.1)
                if performance > losses:
                    losses = performance
                    worst = i
        for j in range(len(self.competitors)):
            if self.competitors[j] == 0 and self.counts[j] > min_counts:
                performance = self.d[j][0] / max(self.counts[i],0.1)
                if performance > wins:
                    wins = performance
                    best = j
        if losses > wins:
            self.competitors[best] = 1
            self.competitors[worst] = 0
        return
    
    def manager(self, n):
        cycle = 8
        final_num = 10
        remaining = sum(self.competitors)
            
        if n > self.gap+55:
            if remaining > final_num:
                if n % cycle == 0:
                    self.dropout()
            else:
                if n % int(cycle*.75) == 0:
                    self.switch()
        return
    
    def update_sensitivity(self, reward, n):
        if self.sensitivity == 0:
            return
        if n < self.gap:
            return
        if reward == 0:
            self.sensitivity -= self.change
            if self.sensitivity < 0.15: self.sensitivity = 0.15
        elif reward == 2:
            self.sensitivity += self.change
            if self.sensitivity > 0.30: self.sensitivity = 0.30
        return
    
    def update(self, chosen_agent, reward, history, turn):
        global agents, hst, rwd
        if turn == self.cutoff:
            self.sensitivity = 0
        performance = self.d[chosen_agent][0]/max(self.d[chosen_agent][0]+self.d[chosen_agent][1]+self.d[chosen_agent][2],1)
        if performance < self.sensitivity:
            losing = True
        else:
            losing = False
        if reward == 0:
            won = 1
        else:
            won= 0
        self.counts[chosen_agent] = self.counts[chosen_agent] + 1
        n = self.counts[chosen_agent]
        if turn in self.timing:
            self.d[chosen_agent] = [1,1,1]
            self.counts[chosen_agent] = 1
            history = []
            hst[chosen_agent] = []
            rwd[chosen_agent] = []
            agents[chosen_agent].reset()
        if n >= self.gaps[chosen_agent] and losing == True and self.reset[chosen_agent] == False:
            self.waypoint[chosen_agent] = n
            history = []
            hst[chosen_agent] = []
            rwd[chosen_agent] = []
            agents[chosen_agent].reset()
            self.d[chosen_agent] = [1,1,1]
            self.counts[chosen_agent] = 1
            self.reset[chosen_agent] = True
        if self.reset[chosen_agent] == True:
            if n == self.waypoint[chosen_agent] + int(self.gaps[chosen_agent]/3):
                self.reset[chosen_agent] = False
                self.gaps[chosen_agent] = n + int(self.gap/2)
            else:
                for i in range(3):
                    self.d[chosen_agent][i] = self.d[chosen_agent][i]*self.decay
                self.d[chosen_agent][reward] += 1
                self.w[chosen_agent] = won
                self.l[chosen_agent] = 1 - won
                return history
        
        for i in range(3):
            self.d[chosen_agent][i] = self.d[chosen_agent][i]*self.decay
        self.d[chosen_agent][reward] += 1
        self.w[chosen_agent] = won
        self.l[chosen_agent] = 1 - won
        return history
    
# base class for all agents, random agent
class agent():
    def reset(self):
        return
    
    def initial_step(self):
        return np.random.randint(3)
    
    def history_step(self, history):
        return np.random.randint(3)
    
    def step(self, history):
        if len(history) == 0:
            return int(self.initial_step())
        else:
            return int(self.history_step(history))

# simple transition matrix: previous step -> next step
class transition_matrix(agent):
    def __init__(self, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type = 'step' 
        else:
            self.step_type = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        
    def reset(self):
        return
        
    def history_step(self, history):
        if len(history) == 0:
            return np.random.randint(3)
        matrix = np.zeros((3,3)) + self.init_value
        for i in range(len(history) - 1):
            matrix = (matrix - self.init_value) / self.decay + self.init_value
            matrix[int(history[i][self.step_type]), int(history[i+1][self.step_type])] += 1

        if  self.deterministic:
            step = np.argmax(matrix[int(history[-1][self.step_type])])
        else:
            step = np.random.choice([0,1,2], p = matrix[int(history[-1][self.step_type])]/matrix[int(history[-1][self.step_type])].sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3
    

# similar to the transition matrix but rely on both previous steps
class transition_tensor(agent):
    
    def __init__(self, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type1 = 'step' 
            self.step_type2 = 'competitorStep'
        else:
            self.step_type2 = 'step' 
            self.step_type1 = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        
    def reset(self):
        return
        
    def history_step(self, history):
        if len(history) == 0:
            return np.random.randint(3)
        matrix = np.zeros((3,3, 3)) + 0.1
        for i in range(len(history) - 1):
            matrix = (matrix - self.init_value) / self.decay + self.init_value
            matrix[int(history[i][self.step_type1]), int(history[i][self.step_type2]), int(history[i+1][self.step_type1])] += 1

        if  self.deterministic:
            step = np.argmax(matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])])
        else:
            step = np.random.choice([0,1,2], p = matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])]/matrix[int(history[-1][self.step_type1]), int(history[-1][self.step_type2])].sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3

        
# looks for the same pattern in history and returns the best answer to the most possible counter strategy
class pattern_matching(agent):
    def __init__(self, steps = 3, deterministic = False, counter_strategy = False, init_value = 0.1, decay = 1):
        self.deterministic = deterministic
        self.counter_strategy = counter_strategy
        if counter_strategy:
            self.step_type = 'step' 
        else:
            self.step_type = 'competitorStep'
        self.init_value = init_value
        self.decay = decay
        self.steps = steps
        
    def reset(self):
        return
        
    def history_step(self, history):
        if len(history) == 0:
            return np.random.randint(3)
        if len(history) < self.steps + 1:
            return self.initial_step()
        
        next_step_count = np.zeros(3) + self.init_value
        pattern = [history[i][self.step_type] for i in range(- self.steps, 0)]
        
        for i in range(len(history) - self.steps):
            next_step_count = (next_step_count - self.init_value)/self.decay + self.init_value
            current_pattern = [history[j][self.step_type] for j in range(i, i + self.steps)]
            if np.sum([pattern[j] == current_pattern[j] for j in range(self.steps)]) == self.steps:
                next_step_count[history[i + self.steps][self.step_type]] += 1
        
        if next_step_count.max() == self.init_value:
            return self.initial_step()
        
        if  self.deterministic:
            step = np.argmax(next_step_count)
        else:
            step = np.random.choice([0,1,2], p = next_step_count/next_step_count.sum())
        
        if self.counter_strategy:
            # we predict our step using transition matrix (as competitor can do) and beat probable competitor step
            return (step + 2) % 3 
        else:
            # we just predict competitors step and beat it
            return (step + 1) % 3
        
class calculation(agent):    
    def __init__(self, size = 0, counter_strategy = False):
        self.counter_strategy = counter_strategy
        self.gg = {}
        self.size = size
        
    def reset(self):
        self.gg = {}
        return
                
    def history_step(self, history):
        global code
        inp = ''
        try:
            inp = 'RPS'[history[-1]['competitorStep']]
        except:
            pass
        self.gg['input'] = inp
        exec(code, self.gg)
        return {'R': 0, 'P': 1, 'S': 2}[self.gg['output']]
    
class calculation_dllu(agent):    
    def __init__(self, size = 0, counter_strategy = False):
        self.counter_strategy = counter_strategy
        self.gg = {}
        
    def reset(self):
        self.gg = {}
        return
                
    def history_step(self, history):
        global code_dllu
        inp = ''
        try:
            inp = 'RPS'[history[-1]['competitorStep']]
        except:
            pass
        self.gg['input'] = inp
        exec(code_dllu, self.gg)
        return {'R': 0, 'P': 1, 'S': 2}[self.gg['output']]
    
action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])    
class markov(agent):
    def reset(self):
        global table, action_seq
        action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])  
        return
    
    def history_step(self, history):
        
        global table, action_seq
        
        k = 2
        if len(history) % 250 == 0: # refresh table every 250 steps
            action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])    
        if len(action_seq) <= 2 * k + 1:
            action = int(np.random.randint(3))
            if len(history) > 0:
                action_seq.extend([history[-1]['competitorStep'], action])
            else:
                action_seq.append(action)
            return action
        # update table
        key = ''.join([str(a) for a in action_seq[:-1]])
        table[key][history[-1]['competitorStep']] += 1
        # update action seq
        action_seq[:-2] = action_seq[2:]
        action_seq[-2] = history[-1]['competitorStep']
        # predict opponent next move
        key = ''.join([str(a) for a in action_seq[:-1]])
        if len(history) < 500:
            next_opponent_action_pred = np.argmax(table[key])
        else:
            scores = np.array(table[key])
            next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum()) # add stochasticity for second part of the game
        # make an action
        action = (next_opponent_action_pred + 1) % 3
        # if high probability to lose -> let's surprise our opponent with sudden change of our strategy
        if len(history) > 900:
            action = next_opponent_action_pred
        action_seq[-1] = action
        return int(action)

class subsequence_matching(agent):
    def __init__(self, char_to_match= 4, most_freq = False):
        self.char_to_match = char_to_match
        self.s = ''
        self.most_freq = most_freq

    def reset(self):
        self.s = ""
        return
    
    #return index after the whole matched substring. (Equivalent to opponent next action given recent_act)
    def list_subsequent_act(self, mem = '0102022111', recent_act = '202'):
        #below return the exclusive end index, aka opponent's historical next action given string of recent_act
        return [m.end() for m in re.finditer(recent_act, mem)]

    def history_step(self, history):
        if len(history) < 5:
            return random.randint(0,2)
        else:
            self.s += str(history[-1]['competitorStep'])

            next_acts = self.list_subsequent_act(mem = self.s, recent_act = self.s[-(self.char_to_match):])
            acts = []
            for a in next_acts:
                try:
                    acts.append(int(self.s[a]))
                except Exception:
                    break

            #counter opponents last pattern
            try:
                #counter most_freq if true otherwise counter last_one
                give_hand = max(set(acts), key=acts.count) if self.most_freq else (acts[-1] + 1 ) % 3
            except Exception:
                return random.randint(0,2)

            return give_hand
        
class legion_agent(agent):
    def __init__(self, shift=0, dec=0.75, app=0, mult=1.0, mm=0.4, score=4):
        #Length of my and opponent subsequences to store in dictionary, and number of tiers
        self.my_range = 3
        self.opp_range = 3
        self.tiers = 3
        #Sequence of my past moves
        self.my_seq = []
        #Sequence of opponent's past moves
        self.opp_seq = []
        #A dictionary of my+opp subsequences, statistics for next opp move, and future predictions
        self.patterns = []
        #A buffer of historical patterns waiting to check results of future moves
        self.pattern_buffer = [[] for x in range(self.tiers)]
        #Length of time to wait before collecting subsequence data
        self.collect_data = self.my_range + self.opp_range + 2*(self.tiers) + 2
        #Length of time to wait before making predictions from the dictionary
        self.warmup_time = 8*(self.my_range+self.tiers+1)
        #My last action
        self.action = 0
        #Opponent's last action
        self.move = 0
        #Amount to increment result/prediction statistics (starting value)
        self.score = score
        #Decay for past scores (1 = no decay)
        self.decay = dec
        #Decay for lower tiers
        self.tier_decay = 0.4
        #Appreciation for scores (higher means higher weight on later results)
        self.appreciate = app
        #Multiplier for scores (0 = purely random, higher means less randomness)
        self.mult = mult
        #Turns to gather memory data before applying accuracy
        self.mem_turns = 10
        #Minimum accuracy to keep pattern memory
        self.min_mem = mm
        self.shift = shift
        
    def reset(self):
        self.my_seq = []
        self.opp_seq = []
        self.patterns = []
        self.pattern_buffer = [[] for x in range(self.tiers)]
        return
        
    def beats(self, x, y):
        if y == 0:
            if x == 1:return True
        elif y == 1:
            if x == 2:return True
        else:
            if x == 0:return True
        return False

    def process_patterns(self, mine, opps):
        for i in range(self.tiers):
            entry = {}
            pattern = []
            preds = {"future_1":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}, 
                     "future_2":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}, 
                     "future_3":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}}
            pattern.extend(mine[-self.my_range-1-i:-1])
            pattern.extend(opps[-self.opp_range-1-i:-1])
            strings = [str(x) for x in pattern]
            int_pattern = "". join(strings)
            entry["predictions"] = preds
            if not int_pattern in self.patterns[i]:
                self.patterns[i][int_pattern] = entry
            self.pattern_buffer[i].append(pattern)
        return
    
    def update_predictions(self, move):
        futures = ["future_1", "future_2", "future_3"]
        if len(self.pattern_buffer[0]) <= 4:
            return
        for j in range(self.tiers):
            for i in range(3):
                key = self.pattern_buffer[j][-2-i]
                strings = [str(y) for y in key]
                int_key = "". join(strings)
                if int_key in self.patterns[j] and futures[i] in self.patterns[j][int_key]["predictions"]:
                    self.patterns[j][int_key]["predictions"][futures[i]][{0:"rock",1:"paper",2:"scissors"}[self.move]]+=self.score
                    self.patterns[j][int_key]["predictions"][futures[i]]["turns"]+=1
                    if self.beats(self.action,move)==True: self.patterns[j][int_key]["predictions"][futures[i]]["wins"]+=1
                    w = self.patterns[j][int_key]["predictions"][futures[i]]["wins"]
                    t = self.patterns[j][int_key]["predictions"][futures[i]]["turns"]
                    self.patterns[j][int_key]["predictions"][futures[i]]["accuracy"]=w/t
                    if self.patterns[j][int_key]["predictions"][futures[i]]["accuracy"] < self.min_mem and self.patterns[j][int_key]["predictions"][futures[i]]["turns"] > self.mem_turns:
                        self.patterns[j][int_key]["predictions"][futures[i]]["rock"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["paper"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["scissors"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["turns"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["wins"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["accuracy"]=0
        return

    def make_prediction(self, mine, opps):
        results = [[1,1,1] for x in range(self.tiers)]
        futures = ["future_1", "future_2", "future_3"]
        final_results = [0,0,0]
    
        for j in range(self.tiers):
            matrix = [[0,0,0],[0,0,0],[0,0,0]]
            for i in range(3):
                pattern = []
                pattern.extend(mine[-self.my_range-1-i-j:-1-i])
                pattern.extend(opps[-self.opp_range-1-i-j:-1-i])
                strings = [str(y) for y in pattern]
                key = "". join(strings)
                if key in self.patterns[j]:
                    matrix[i][0] = self.patterns[j][key]["predictions"][futures[i]]["rock"]
                    matrix[i][1] = self.patterns[j][key]["predictions"][futures[i]]["paper"]
                    matrix[i][2] = self.patterns[j][key]["predictions"][futures[i]]["scissors"]
            for i in range(3):
                results[j][i] += (matrix[0][i])**self.mult+self.decay*(matrix[1][i]**self.mult)+(self.decay**2)*(matrix[2][i]**self.mult)
        for i in range(3):
            for j in range(self.tiers):
                final_results[i] += (self.tier_decay**j)*results[j][i]
        prediction = sum(random.choices(population=[0,1,2], weights=[x / sum(final_results) for x in final_results], k=1))
        return (prediction+1)%3
    
    def history_step(self, history):
        self.score += self.appreciate
        if len(history) == 0:
            for i in range(self.tiers):
                self.patterns.append(defaultdict(int))
            self.action = random.randint(0,2)
            return self.action
        elif len(history) <= self.collect_data:
            self.my_seq.append(self.action)
            self.opp_seq.append(history[-1]['competitorStep'])
            self.action = random.randint(0,2)
            return self.action
        elif len(history) < self.warmup_time:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            self.process_patterns(self.my_seq,self.opp_seq)
            self.update_predictions(self.move)
            self.action = random.randint(0,2)
            return self.action
        else:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            self.process_patterns(self.my_seq,self.opp_seq)
            self.update_predictions(self.move)
            self.action = self.make_prediction(self.my_seq,self.opp_seq)
            return (self.action+self.shift)%3
        
class centurion_agent(agent):
    def __init__(self, shift=0, dec=0.75, app=0, mult=1.0, mm=0.4, score=4):
        #Length of my and opponent subsequences to store in dictionary, and number of tiers
        self.my_range = 3
        self.opp_range = 3
        self.tiers = 3
        #Sequence of my past moves
        self.my_seq = []
        self.my_trans = []
        #Sequence of opponent's past moves
        self.opp_seq = []
        self.opp_trans = []
        #A dictionary of my+opp subsequences, statistics for next opp move, and future predictions
        self.patterns = []
        #A buffer of historical patterns waiting to check results of future moves
        self.pattern_buffer = [[] for x in range(self.tiers)]
        #Length of time to wait before collecting subsequence data
        self.collect_data = self.my_range + self.opp_range + 2*(self.tiers) + 2
        #Length of time to wait before making predictions from the dictionary
        self.warmup_time = 8*(self.my_range+self.tiers+1)
        #My last action
        self.action = 0
        #Opponent's last action
        self.move = 0
        #Amount to increment result/prediction statistics (starting value)
        self.score = score
        #Decay for past scores (1 = no decay)
        self.decay = dec
        #Decay for lower tiers
        self.tier_decay = 0.4
        #Appreciation for scores (higher means higher weight on later results)
        self.appreciate = app
        #Multiplier for scores (0 = purely random, higher means less randomness)
        self.mult = mult
        #Turns to gather memory data before applying accuracy
        self.mem_turns = 10
        #Minimum accuracy to keep pattern memory
        self.min_mem = mm
        self.shift = shift
        
    def reset(self):
        self.my_seq = []
        self.opp_seq = []
        self.my_trans = []
        self.opp_trans = []
        self.patterns = []
        self.pattern_buffer = [[] for x in range(self.tiers)]
        return
        
    def beats(self, x, y):
        if y == 0:
            if x == 1:return True
        elif y == 1:
            if x == 2:return True
        else:
            if x == 0:return True
        return False
    
    def mod2(self, x, y):
        if x == 0:
            if y == 0: return 0
            if y == 1: return 1
            if y == 2: return 2
        if x == 1:
            if y == 0: return 2
            if y == 1: return 0
            if y == 2: return 1
        if x == 2:
            if y == 0: return 1
            if y == 1: return 2
            if y == 2: return 0
        return 0

    def process_patterns(self, mine, opps):
        for i in range(self.tiers):
            entry = {}
            pattern = []
            preds = {"future_1":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}, 
                     "future_2":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}, 
                     "future_3":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}}
            pattern.extend(mine[-self.my_range-1-i:-1])
            pattern.extend(opps[-self.opp_range-1-i:-1])
            strings = [str(x) for x in pattern]
            int_pattern = "". join(strings)
            entry["predictions"] = preds
            if not int_pattern in self.patterns[i]:
                self.patterns[i][int_pattern] = entry
            self.pattern_buffer[i].append(pattern)
        return
    
    def update_predictions(self, move):
        futures = ["future_1", "future_2", "future_3"]
        if len(self.pattern_buffer[0]) <= 4:
            return
        for j in range(self.tiers):
            for i in range(3):
                key = self.pattern_buffer[j][-2-i]
                strings = [str(y) for y in key]
                int_key = "". join(strings)
                if int_key in self.patterns[j] and futures[i] in self.patterns[j][int_key]["predictions"]:
                    self.patterns[j][int_key]["predictions"][futures[i]][{0:"rock",1:"paper",2:"scissors"}[self.move]]+=self.score
                    self.patterns[j][int_key]["predictions"][futures[i]]["turns"]+=1
                    if self.beats(self.action,move)==True: self.patterns[j][int_key]["predictions"][futures[i]]["wins"]+=1
                    w = self.patterns[j][int_key]["predictions"][futures[i]]["wins"]
                    t = self.patterns[j][int_key]["predictions"][futures[i]]["turns"]
                    self.patterns[j][int_key]["predictions"][futures[i]]["accuracy"]=w/t
                    if self.patterns[j][int_key]["predictions"][futures[i]]["accuracy"] < self.min_mem and self.patterns[j][int_key]["predictions"][futures[i]]["turns"] > self.mem_turns:
                        self.patterns[j][int_key]["predictions"][futures[i]]["rock"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["paper"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["scissors"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["turns"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["wins"]=0
                        self.patterns[j][int_key]["predictions"][futures[i]]["accuracy"]=0
        return

    def make_prediction(self, mine, opps):
        results = [[1,1,1] for x in range(self.tiers)]
        futures = ["future_1", "future_2", "future_3"]
        final_results = [0,0,0]
    
        for j in range(self.tiers):
            matrix = [[0,0,0],[0,0,0],[0,0,0]]
            for i in range(3):
                pattern = []
                pattern.extend(mine[-self.my_range-1-i-j:-1-i])
                pattern.extend(opps[-self.opp_range-1-i-j:-1-i])
                strings = [str(y) for y in pattern]
                key = "". join(strings)
                if key in self.patterns[j]:
                    matrix[i][0] = self.patterns[j][key]["predictions"][futures[i]]["rock"]
                    matrix[i][1] = self.patterns[j][key]["predictions"][futures[i]]["paper"]
                    matrix[i][2] = self.patterns[j][key]["predictions"][futures[i]]["scissors"]
            for i in range(3):
                results[j][i] += (matrix[0][i])**self.mult+self.decay*(matrix[1][i]**self.mult)+(self.decay**2)*(matrix[2][i]**self.mult)
        for i in range(3):
            for j in range(self.tiers):
                final_results[i] += (self.tier_decay**j)*results[j][i]
        prediction = sum(random.choices(population=[0,1,2], weights=[x / sum(final_results) for x in final_results], k=1))
        return (prediction+1)%3
    
    def history_step(self, history):
        self.score += self.appreciate
        if len(history) == 0:
            for i in range(self.tiers):
                self.patterns.append(defaultdict(int))
            self.action = random.randint(0,2)
            return self.action
        elif len(history) == 1:
            self.my_seq.append(self.action)
            self.opp_seq.append(history[-1]['competitorStep'])
            self.action = random.randint(0,2)
            return self.action
        elif len(history) <= self.collect_data:
            self.my_seq.append(self.action)
            self.opp_seq.append(history[-1]['competitorStep'])
            self.my_trans.append(self.mod2(self.my_seq[-2],self.my_seq[-1]))
            self.opp_trans.append(self.mod2(self.opp_seq[-2],self.opp_seq[-1]))
            self.action = random.randint(0,2)
            return self.action
        elif len(history) < self.warmup_time:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            self.my_trans.append(self.mod2(self.my_seq[-2],self.my_seq[-1]))
            self.opp_trans.append(self.mod2(self.opp_seq[-2],self.opp_seq[-1]))
            self.process_patterns(self.my_trans,self.opp_trans)
            self.update_predictions(self.opp_trans[-1])
            self.action = random.randint(0,2)
            return self.action
        elif len(history) <= self.collect_data:
            self.my_seq.append(self.action)
            self.opp_seq.append(history[-1]['competitorStep'])
            self.my_trans.append(self.mod2(self.my_seq[-2],self.my_seq[-1]))
            self.opp_trans.append(self.mod2(self.opp_seq[-2],self.opp_seq[-1]))
            self.action = random.randint(0,2)
            return self.action
        elif len(history) < self.warmup_time:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            self.my_trans.append(self.mod2(self.my_seq[-2],self.my_seq[-1]))
            self.opp_trans.append(self.mod2(self.opp_seq[-2],self.opp_seq[-1]))
            self.process_patterns(self.my_seq,self.opp_seq)
            self.update_predictions(self.move)
            self.action = random.randint(0,2)
            return self.action
        else:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            self.my_trans.append(self.mod2(self.my_seq[-2],self.my_seq[-1]))
            self.opp_trans.append(self.mod2(self.opp_seq[-2],self.opp_seq[-1]))
            self.process_patterns(self.my_seq,self.opp_seq)
            self.update_predictions(self.move)
            self.action = self.make_prediction(self.my_seq,self.opp_seq)
            return (self.action+self.shift)%3
        
class decision_tree(agent):
    
    def __init__(self, k = 5, min_samples = 25):
        self.rollouts_hist = None
        self.last_move = None
        self.data = None
        self.test_sample = None
        self.k = k
        self.min_samples = min_samples
        
    def reset(self):
        self.rollouts_hist = None
        self.last_move = None
        self.data = None
        self.test_sample = None
        return
        
    def history_step(self, history):
        def construct_local_features(rollouts):
            features = np.array([[step % k for step in rollouts['steps']] for k in (2, 3, 5)])
            features = np.append(features, rollouts['steps'])
            features = np.append(features, rollouts['actions'])
            features = np.append(features, rollouts['opp-actions'])
            return features

        def construct_global_features(rollouts):
            features = []
            for key in ['actions', 'opp-actions']:
                for i in range(3):
                    actions_count = np.mean([r == i for r in rollouts[key]])
                    features.append(actions_count)
            return np.array(features)

        def construct_features(short_stat_rollouts, long_stat_rollouts):
            lf = construct_local_features(short_stat_rollouts)
            gf = construct_global_features(long_stat_rollouts)
            features = np.concatenate([lf, gf])
            return features

        def predict_opponent_move(train_data, test_sample):
            classifier = DecisionTreeClassifier(random_state=42)
            classifier.fit(train_data['x'], train_data['y'])
            return classifier.predict(test_sample)

        def update_rollouts_hist(rollouts_hist, last_move, opp_last_action):
            rollouts_hist['steps'].append(last_move['step'])
            rollouts_hist['actions'].append(last_move['action'])
            rollouts_hist['opp-actions'].append(opp_last_action)
            return rollouts_hist

        def warmup_strategy(history):
            action = int(np.random.randint(3))
            if len(history) == 0:
                self.last_move = {'step': 0, 'action': action}
                self.rollouts_hist = {'steps': [], 'actions': [], 'opp-actions': []}
            else:
                self.rollouts_hist = update_rollouts_hist(self.rollouts_hist, self.last_move, history[-1]['competitorStep'])
                self.last_move = {'step': len(history), 'action': action}
            return int(action)

        def init_training_data(rollouts_hist, k):
            for i in range(len(rollouts_hist['steps']) - k + 1):
                short_stat_rollouts = {key: rollouts_hist[key][i:i+k] for key in rollouts_hist}
                long_stat_rollouts = {key: rollouts_hist[key][:i+k] for key in rollouts_hist}
                features = construct_features(short_stat_rollouts, long_stat_rollouts)        
                self.data['x'].append(features)
            test_sample = self.data['x'][-1].reshape(1, -1)
            self.data['x'] = self.data['x'][:-1]
            self.data['y'] = rollouts_hist['opp-actions'][k:]
            return self.data, test_sample

        # hyperparameters
        k = self.k
        min_samples = self.min_samples
    
        if len(history) == 0:
            self.data = {'x': [], 'y': []}
        # if not enough data -> randomize
        if len(history) <= min_samples + k:
            return warmup_strategy(history)
        # update statistics
        self.rollouts_hist = update_rollouts_hist(self.rollouts_hist, self.last_move, history[-1]['competitorStep'])
        # update training data
        if len(self.data['x']) == 0:
            self.data, test_sample = init_training_data(self.rollouts_hist, k)
        else:        
            short_stat_rollouts = {key: self.rollouts_hist[key][-k:] for key in self.rollouts_hist}
            features = construct_features(short_stat_rollouts, self.rollouts_hist)
            test_sample = features.reshape(1, -1)
        
        # predict opponents move and choose an action
        next_opp_action_pred = predict_opponent_move(self.data, test_sample)
        action = int((next_opp_action_pred + 1) % 3)
        last_move = {'step': len(history), 'action': action}
        return action

class Stats:
    def __init__(self):
        self.sum = [[0, 0, 0]]
    def add(self, move, score):
        self.sum[-1][move] += score
    def advance(self):
        self.sum.append(self.sum[-1])
    def max(self, age, default, score):
        if age >= len(self.sum): diff = self.sum[-1]
        else: diff = [self.sum[-1][i] - self.sum[-1 - age][i] for i in range(3)]
        m = max(diff)
        if m > score: return diff.index(m), m
        return default, score
    
class Predictor:
    def __init__(self):
        self.stats = Stats()
        self.lastguess = -1
    def addguess(self, lastmove, guess):
        if lastmove != -1:
            diff = (lastmove - self.prediction) % 3
            self.stats.add(beat(diff), 1)
            self.stats.add(loseto(diff), -1)
            self.stats.advance()
        self.prediction = guess
    def bestguess(self, age, best):
        bestdiff = self.stats.max(age, (best[0] - self.prediction) % 3, best[1])
        return (bestdiff[0] + self.prediction) % 3, bestdiff[1]
    
def recall(age, hist):
    end, length = 0, 0
    for past in range(1, min(age + 1, len(hist) - 1)):
        if length >= len(hist) - past: break
        for i in range(-1 - length, 0):
            if hist[i - past] != hist[i]: break
        else:
            for length in range(length + 1, len(hist) - past):
                if hist[-past - length - 1] != hist[-length - 1]: break
            else: length += 1
            end = len(hist) - past
    return end

def beat(i):
    return (i + 1) % 3
def loseto(i):
    return (i - 1) % 3   

class Guesser:

    def __init__(self):
        self.predictors = []
        self.predict_history = self.predictor((len(ages), 2, 3))
        self.predict_frequency = self.predictor((len(ages), 2))
        self.predict_fixed = self.predictor()
        self.predict_random = self.predictor()
        self.predict_meta = [Predictor() for a in range(len(ages))]
        self.stats = [Stats() for i in range(2)]
        self.histories = [[], [], []]

    def predictor(self, dims=None):
        if dims: return [self.predictor(dims[1:]) for i in range(dims[0])]
        self.predictors.append(Predictor())
        return self.predictors[-1]

    def move(self, them):

        if them != -1:
            self.histories[1].append(them)
            self.histories[2].append((self.histories[0][-1], them))
            for watch in range(2):
                self.stats[watch].add(self.histories[watch][-1], 1)

        rand = random.randrange(3)
        self.predict_random.addguess(them, rand)
        self.predict_fixed.addguess(them, 0)

        for a, age in enumerate(ages):
            best = [recall(age, hist) for hist in self.histories]
            for mimic in range(2):
                for watch, when in enumerate(best):
                    if not when: move = rand
                    else: move = self.histories[mimic][when]
                    self.predict_history[a][mimic][watch].addguess(them, move)
                mostfreq, score = self.stats[mimic].max(age, rand, -1)
                self.predict_frequency[a][mimic].addguess(them, mostfreq)

        for meta, age in enumerate(ages):
            best = (-1, -1)
            for predictor in self.predictors:
                best = predictor.bestguess(age, best)
            self.predict_meta[meta].addguess(them, best[0])

        best = (-1, -1)
        for meta in range(len(ages)):
            best = self.predict_meta[meta].bestguess(len(self.histories[0]) , best) 
        self.histories[0].append(best[0])

        return best[0]
    
        
class iocane_agent(agent):
    def __init__(self):
        self.guesser = None
        
    def reset(self):
        self.guesser = None
        return
        
    def history_step(self, history):
        if len(history) == 0:
            self.guesser = Guesser()
            action = self.guesser.move(-1)
        else:
            action = self.guesser.move(history[-1]['competitorStep'])
        
        return action
    
class spinoza_agent(agent):
    def __init__(self, lim = 23, rotations = True):
        self.max_limit = lim
        self.add_rotations = rotations

        # number of predictors
        self.numPre = 6
        if self.add_rotations:
            self.numPre *= 3

        # number of meta-predictors
        self.numMeta = 4
        if self.add_rotations:
            self.numMeta *= 3

        # saves history
        self.moves = ['', '', '']

        self.beat = {'R':'P', 'P':'S', 'S':'R'}
        self.dna =  {'RP':0, 'PS':1, 'SR':2,
                'PR':3, 'SP':4, 'RS':5,
                'RR':6, 'PP':7, 'SS':8}

        self.p = ["P"]*self.numPre
        self.m = ["P"]*self.numMeta
        self.pScore = [[0]*self.numPre for i in range(8)]
        self.mScore = [0]*self.numMeta

        self.length = 0
        self.threat = 0
        self.output = "P"
        
    def reset(self):
        self.numPre = 6
        if self.add_rotations:
            self.numPre *= 3

        # number of meta-predictors
        self.numMeta = 4
        if self.add_rotations:
            self.numMeta *= 3

        # saves history
        self.moves = ['', '', '']

        self.beat = {'R':'P', 'P':'S', 'S':'R'}
        self.dna =  {'RP':0, 'PS':1, 'SR':2,
                'PR':3, 'SP':4, 'RS':5,
                'RR':6, 'PP':7, 'SS':8}

        self.p = ["P"]*self.numPre
        self.m = ["P"]*self.numMeta
        self.pScore = [[0]*self.numPre for i in range(8)]
        self.mScore = [0]*self.numMeta

        self.length = 0
        self.threat = 0
        self.output = "P"
        return
        
    def history_step(self, history):
        if len(history) < 2:
            self.output = self.beat[self.output]
            return {'R':0, 'P':1, 'S':2}[self.output]
        input = "RPS"[history[-1]['competitorStep']]

        # threat of opponent
        outcome = (self.beat[input]==self.output) - (input==self.beat[self.output])
        self.threat = 0.9*self.threat - 0.1*outcome
        
        # refresh pScore
        for i in range(self.numPre):
            pp = self.p[i]
            bpp = self.beat[pp]
            bbpp = self.beat[bpp]
            self.pScore[0][i] = 0.9*self.pScore[0][i] + 0.1*((input==pp)-(input==bbpp))
            self.pScore[1][i] = 0.9*self.pScore[1][i] + 0.1*((self.output==pp)-(self.output==bbpp))
            self.pScore[2][i] = 0.8*self.pScore[2][i] + 0.3*((input==pp)-(input==bbpp)) + \
                            0.1*(self.length % 3 - 1)
            self.pScore[3][i] = 0.8*self.pScore[3][i] + 0.3*((self.output==pp)-(self.output==bbpp)) + \
                            0.1*(self.length % 3 - 1)

        # refresh mScore
        for i in range(self.numMeta):
            self.mScore[i] = 0.9*self.mScore[i] + 0.1*((input==self.m[i])-(input==self.beat[self.beat[self.m[i]]])) + \
                        0.05*(self.length % 5 - 2)

        # refresh moves
        self.moves[0] += str(self.dna[input+self.output])
        self.moves[1] += input
        self.moves[2] += self.output

        # refresh length
        self.length += 1

        # new predictors
        limit = min([self.length,self.max_limit])
        for y in range(3):
            j = limit
            while j>=1 and not self.moves[y][self.length-j:self.length] in self.moves[y][0:self.length-1]:
                j-=1
            if j>=1:
                i = self.moves[y].rfind(self.moves[y][self.length-j:self.length],0,self.length-1)
                self.p[0+2*y] = self.moves[1][j+i] 
                self.p[1+2*y] = self.beat[self.moves[2][j+i]]

        # rotations of predictors
        if self.add_rotations:
            for i in range(int(self.numPre/3),self.numPre):
                self.p[i]=self.beat[self.beat[self.p[i-int(self.numPre/3)]]]

        # new meta
        for i in range(0,4,2):
            self.m[i] = self.p[self.pScore[i].index(max(self.pScore[i]))]
            self.m[i+1] = self.beat[self.p[self.pScore[i+1].index(max(self.pScore[i+1]))]]

        # rotations of meta
        if self.add_rotations:
            for i in range(4,12):
                self.m[i]=self.beat[self.beat[self.m[i-4]]]
    
        self.output = self.beat[self.m[self.mScore.index(max(self.mScore))]]

        if self.threat > 0.4:
            # ah take this!
            self.output = self.beat[self.beat[self.output]]

        return {'R':0, 'P':1, 'S':2}[self.output]

class IOU2_agent(agent):
    def __init__(self, num=27):
        self.num_predictor = num
        self.len_rfind = [20]
        self.limit = [10,20,60]
        self.beat = { "R":"P" , "P":"S", "S":"R"}
        self.not_lose = { "R":"PPR" , "P":"SSP" , "S":"RRS" } #50-50 chance
        self.my_his   =""
        self.your_his =""
        self.both_his =""
        self.list_predictor = [""]*self.num_predictor
        self.length = 0
        self.temp1 = { "PP":"1" , "PR":"2" , "PS":"3",
                      "RP":"4" , "RR":"5", "RS":"6",
                      "SP":"7" , "SR":"8", "SS":"9"}
        self.temp2 = { "1":"PP","2":"PR","3":"PS",
                        "4":"RP","5":"RR","6":"RS",
                        "7":"SP","8":"SR","9":"SS"} 
        self.who_win = { "PP": 0, "PR":1 , "PS":-1,
                        "RP": -1,"RR":0, "RS":1,
                        "SP": 1, "SR":-1, "SS":0}
        self.score_predictor = [0]*self.num_predictor
        self.output = random.choice("RPS")
        self.predictors = [self.output]*self.num_predictor
        
    def reset(self):
        self.len_rfind = [20]
        self.limit = [10,20,60]
        self.beat = { "R":"P" , "P":"S", "S":"R"}
        self.not_lose = { "R":"PPR" , "P":"SSP" , "S":"RRS" } #50-50 chance
        self.my_his   =""
        self.your_his =""
        self.both_his =""
        self.list_predictor = [""]*self.num_predictor
        self.length = 0
        self.temp1 = { "PP":"1" , "PR":"2" , "PS":"3",
                      "RP":"4" , "RR":"5", "RS":"6",
                      "SP":"7" , "SR":"8", "SS":"9"}
        self.temp2 = { "1":"PP","2":"PR","3":"PS",
                        "4":"RP","5":"RR","6":"RS",
                        "7":"SP","8":"SR","9":"SS"} 
        self.who_win = { "PP": 0, "PR":1 , "PS":-1,
                        "RP": -1,"RR":0, "RS":1,
                        "SP": 1, "SR":-1, "SS":0}
        self.score_predictor = [0]*self.num_predictor
        self.output = random.choice("RPS")
        self.predictors = [self.output]*self.num_predictor
        return

    def next_action(self, T, A, S):
        to_char = ["R", "P", "S"]
        from_char = {"R": 0, "P": 1, "S": 2}
        if T == 0:
            return from_char[self.output]
        input = to_char[A]

        if len(self.list_predictor[0])<5:
            front =0
        else:
            front =1
        for i in range (self.num_predictor):
            if self.predictors[i]==input:
                result ="1"
            else:
                result ="0"
            self.list_predictor[i] = self.list_predictor[i][front:5]+result #only 5 rounds before
        #history matching 1-6
        self.my_his += self.output
        self.your_his += input
        self.both_his += self.temp1[input+self.output]
        self.length +=1
        for i in range(1):
            len_size = min(self.length,self.len_rfind[i])
            j=len_size
            #self.both_his
            while j>=1 and not self.both_his[self.length-j:self.length] in self.both_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.both_his.rfind(self.both_his[self.length-j:self.length],0,self.length-1)
                self.predictors[0+6*i] = self.your_his[j+k]
                self.predictors[1+6*i] = self.beat[self.my_his[j+k]]
            else:
                self.predictors[0+6*i] = random.choice("RPS")
                self.predictors[1+6*i] = random.choice("RPS")
            j=len_size
            #self.your_his
            while j>=1 and not self.your_his[self.length-j:self.length] in self.your_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.your_his.rfind(self.your_his[self.length-j:self.length],0,self.length-1)
                self.predictors[2+6*i] = self.your_his[j+k]
                self.predictors[3+6*i] = self.beat[self.my_his[j+k]]
            else:
                self.predictors[2+6*i] = random.choice("RPS")
                self.predictors[3+6*i] = random.choice("RPS")
            j=len_size
            #self.my_his
            while j>=1 and not self.my_his[self.length-j:self.length] in self.my_his[0:self.length-1]:
                j-=1
            if j>=1:
                k = self.my_his.rfind(self.my_his[self.length-j:self.length],0,self.length-1)
                self.predictors[4+6*i] = self.your_his[j+k]
                self.predictors[5+6*i] = self.beat[self.my_his[j+k]]
            else:
                self.predictors[4+6*i] = random.choice("RPS")
                self.predictors[5+6*i] = random.choice("RPS")

        for i in range(3):
            temp =""
            search = self.temp1[(self.output+input)] #last round
            for start in range(2, min(self.limit[i],self.length) ):
                if search == self.both_his[self.length-start]:
                    temp+=self.both_his[self.length-start+1]
            if(temp==""):
                self.predictors[6+i] = random.choice("RPS")
            else:
                collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
                for sdf in temp:
                    next_move = self.temp2[sdf]
                    if(self.who_win[next_move]==-1):
                        collectR[self.temp2[sdf][1]]+=3
                    elif(self.who_win[next_move]==0):
                        collectR[self.temp2[sdf][1]]+=1
                    elif(self.who_win[next_move]==1):
                        collectR[self.beat[self.temp2[sdf][0]]]+=1
                max1 = -1
                p1 =""
                for key in collectR:
                    if(collectR[key]>max1):
                        max1 = collectR[key]
                        p1 += key
                self.predictors[6+i] = random.choice(p1)
        for i in range(9,27):
            self.predictors[i] = self.beat[self.beat[self.predictors[i-9]]]
        len_his = len(self.list_predictor[0])
        for i in range(self.num_predictor):
            sum = 0
            for j in range(len_his):
                if self.list_predictor[i][j]=="1":
                    sum+=(j+1)*(j+1)
                else:
                    sum-=(j+1)*(j+1)
            self.score_predictor[i] = sum
        max_score = max(self.score_predictor)
        if max_score>0:
            predict = self.predictors[self.score_predictor.index(max_score)]
        else:
            predict = random.choice(self.your_his)
        self.output = random.choice(self.not_lose[predict])
        return from_char[self.output]
    
        def history_step(self, history):
            if len(history) > 0:
                move = history[-1]['competitorStep']
            else:
                move = 0
            return self.next_action(len(history), move, 3)
        
class xgboost_agent(agent):
    def __init__(self, num = 5, turns = 10):
        self.numTurnsPredictors = num #number of previous turns to use as predictors
        self.minTrainSetRows = turns
        self.myLastMove = None
        self.mySecondLastMove = None
        self.opponentLastMove = None
        self.numDummies = 2 #how many dummy vars we need to represent a move
        self.predictors = pd.DataFrame(columns=[str(x) for x in range(self.numTurnsPredictors * 2 * self.numDummies)]).astype("int")
        self.opponentsMoves = [0] * 1000
        self.roundHistory = [None] * 1000
        self.dummies = [[[0,0,0,0], [0,1,0,0], [1,0,0,0]], [[0,0,0,1], [0,1,0,1], [1,0,0,1]], [[0,0,1,0], [0,1,1,0], [1,0,1,0]]]
        self.clf = XGBClassifier(n_estimators=10)
        
    def reset(self):
        self.myLastMove = None
        self.mySecondLastMove = None
        self.opponentLastMove = None
        self.numDummies = 2 #how many dummy vars we need to represent a move
        self.predictors = pd.DataFrame(columns=[str(x) for x in range(self.numTurnsPredictors * 2 * self.numDummies)]).astype("int")
        self.opponentsMoves = [0] * 1000
        self.roundHistory = [None] * 1000
        self.dummies = [[[0,0,0,0], [0,1,0,0], [1,0,0,0]], [[0,0,0,1], [0,1,0,1], [1,0,0,1]], [[0,0,1,0], [0,1,1,0], [1,0,1,0]]]
        self.clf = XGBClassifier(n_estimators=10)
        return

    def updateFeatures(self, rounds):
        self.predictors.loc[len(self.predictors)] = sum(rounds, [])

    def fitAndPredict(self, x, y, newX):
        self.clf.fit(x.values, y)
        return int(self.clf.predict(np.array(newX).reshape((1,-1)))[0])

    def next_action(self, T, A, S):
        if T == 0:
            self.myLastMove = secrets.randbelow(S)
            return self.myLastMove

        self.roundHistory[T-1] = self.dummies[self.myLastMove][A]
        if T == 1:
            self.myLastMove = secrets.randbelow(S)
            return self.myLastMove
        else:
            self.opponentsMoves[T-2] = A

            if T > self.numTurnsPredictors:
                self.updateFeatures(self.roundHistory[:T][-self.numTurnsPredictors - 1: -1])

            if len(self.predictors) > self.minTrainSetRows:
                predictX = sum(self.roundHistory[:T][-self.numTurnsPredictors:], []) #data to predict next move
                predictedMove = self.fitAndPredict(self.predictors, self.opponentsMoves[:T-1][(self.numTurnsPredictors-1):], predictX)
                self.myLastMove = (predictedMove + 1) % S
                return self.myLastMove
            else:
                self.myLastMove = secrets.randbelow(S)
                return self.myLastMove
            
        def history_step(self, history):
            if len(history) > 0:
                return self.next_action(len(history), history[-1]['competitorStep'], 3)
            else:
                return random.randint(0,2)
        
class aggressive_agent(agent):
    def __init__(self):
        self.Jmax = 2
        self.J = self.Jmax - int(math.sqrt(secrets.randbelow((self.Jmax+1)**2)))
        self.Dmin = 2
        self.Dmax = 5
        self.Hash = []
        self.Map = []
        self.MyMap = []
        for D in range(self.Dmin,self.Dmax+1):
            self.Hash.append([0, 0, 0])
            self.Map.append([{}, {}, {}])
            self.MyMap.append([{}, {}, {}])
        self.G = 2
        self.R = 0.4
        self.V = 0.8
        self.VM = 0.95
        self.B = 0
        self.DT = 200
        
    def reset(self):
        self.Jmax = 2
        self.J = self.Jmax - int(math.sqrt(secrets.randbelow((self.Jmax+1)**2)))
        self.Dmin = 2
        self.Dmax = 5
        self.Hash = []
        self.Map = []
        self.MyMap = []
        for D in range(self.Dmin,self.Dmax+1):
            self.Hash.append([0, 0, 0])
            self.Map.append([{}, {}, {}])
            self.MyMap.append([{}, {}, {}])
        self.G = 2
        self.R = 0.4
        self.V = 0.8
        self.VM = 0.95
        self.B = 0
        self.DT = 200
        return

    def add(self, map1, hash1, A, T):
        if hash1 not in map1:
            map1[hash1] = {'S': []}
        d = map1[hash1]
        if A not in d:
            d[A] = [T]
        else:
            d[A].append(T)
        d['S'].append(T)

    def rank(self, A, T):
        return len([a for a in A if a > T - self.DT])

    def match(self, map1, hash1, S, T):
        if hash1 not in map1:
            return
        d = map1[hash1]
        if self.rank(d['S'], T) >= self.G:
            for A in range(S):
                if A in d and (self.rank(d[A], T) >= self.rank(d['S'], T) * self.R + (1-self.R) * self.G) and secrets.randbelow(1001) < 1000 * self.V:
                    if secrets.randbelow(1001) < 1000 * self.VM:
                        self.B = (A+1) % S
                    else:
                        self.B = A % S
                    self.J = self.Jmax - int(math.sqrt(secrets.randbelow((self.Jmax+1)**2)))
    
    def next_action(self, T, A, S):
        BA = (self.B+1)%S
        self.B = secrets.randbelow(S)
        for D in range(self.Dmin,self.Dmax+1):
            if T > D:
                self.add(self.Map[D-self.Dmin][0], self.Hash[D-self.Dmin][0], A, T)
                self.add(self.Map[D-self.Dmin][1], self.Hash[D-self.Dmin][1], A, T)
                self.add(self.Map[D-self.Dmin][2], self.Hash[D-self.Dmin][2], A, T)
                self.add(self.MyMap[D-self.Dmin][0], self.Hash[D-self.Dmin][0], BA, T)
                self.add(self.MyMap[D-self.Dmin][1], self.Hash[D-self.Dmin][1], BA, T)
                self.add(self.MyMap[D-self.Dmin][2], self.Hash[D-self.Dmin][2], BA, T)
            if T > 0:
                self.Hash[D-self.Dmin][0] = self.Hash[D-self.Dmin][0] // S**2 + (A + S*self.B) * S**(2*D-1)
                self.Hash[D-self.Dmin][1] = self.Hash[D-self.Dmin][1] // S + A * S**(D-1)
                self.Hash[D-self.Dmin][2] = self.Hash[D-self.Dmin][2] // S + self.B * S**(D-1)
            if self.J == 0:
                self.match(self.Map[D-self.Dmin][0], self.Hash[D-self.Dmin][0], S, T)
                self.match(self.Map[D-self.Dmin][1], self.Hash[D-self.Dmin][1], S, T)
                self.match(self.Map[D-self.Dmin][2], self.Hash[D-self.Dmin][2], S, T)
            if self.J == 0:
                self.match(self.MyMap[D-self.Dmin][0], self.Hash[D-self.Dmin][0], S, T)
                self.match(self.MyMap[D-self.Dmin][1], self.Hash[D-self.Dmin][1], S, T)
                self.match(self.MyMap[D-self.Dmin][2], self.Hash[D-self.Dmin][2], S, T)
        if self.J > 0:
            self.J -= 1
        return self.B
    
    def history_step(self, history):
            if len(history) > 0:
                move = history[-1]['competitorStep']
            else:
                move = 0
            return self.next_action(len(history), move, 3)
        
class greenberg_agent(agent):
    def __init__(self):
        self.opp_moves = []
        self.my_moves = []
        self.act = None
        self.opp_history = [0]
        self.my_history = [0]
        self.gear = [[0] for _ in range(24)]
        self.p_random_score = 0
        self.p_full_score = [[[[0 for i in range(3)] for k in range(4)] for j in range(24)] for l in range(50)]
        self.r_full_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(24)] for l in range(50)]
        self.p_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
        self.r_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
        self.s_len = [0] * 6
        self.p_random = secrets.randbelow(3)

        self.p_full = [[0,0,0,0] for _ in range(24)]
        self.r_full = [[0,0] for _ in range(24)]
        self.p_freq = None
        self.my_history_hash = [[0],[0],[0],[0]]
        self.opp_history_hash = [[0],[0],[0],[0]]
        self.freq = [[0,0,0],[0,0,0]]
        
    def reset(self):
        self.opp_moves = []
        self.my_moves = []
        self.act = None
        self.opp_history = [0]
        self.my_history = [0]
        self.gear = [[0] for _ in range(24)]
        self.p_random_score = 0
        self.p_full_score = [[[[0 for i in range(3)] for k in range(4)] for j in range(24)] for l in range(50)]
        self.r_full_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(24)] for l in range(50)]
        self.p_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
        self.r_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
        self.s_len = [0] * 6
        self.p_random = secrets.randbelow(3)

        self.p_full = [[0,0,0,0] for _ in range(24)]
        self.r_full = [[0,0] for _ in range(24)]
        self.p_freq = None
        self.my_history_hash = [[0],[0],[0],[0]]
        self.opp_history_hash = [[0],[0],[0],[0]]
        self.freq = [[0,0,0],[0,0,0]]
        return
    
    @staticmethod
    def min_index(values):
            return min(enumerate(values), key=itemgetter(1))[0]

    @staticmethod
    def max_index(values):
        return max(enumerate(values), key=itemgetter(1))[0]

    def find_best_prediction(self, l, T):  # l = len
        bs = -1000
        bp = 0
        if self.p_random_score > bs:
            bs = self.p_random_score
            bp = self.p_random
        for i in range(3):
            for j in range(24):
                for k in range(4):
                    new_bs = self.p_full_score[T%50][j][k][i] - (self.p_full_score[(50+T-l)%50][j][k][i] if l else 0)
                    if new_bs > bs:
                        bs = new_bs
                        bp = (self.p_full[j][k] + i) % 3
                for k in range(2):
                    new_bs = self.r_full_score[T%50][j][k][i] - (self.r_full_score[(50+T-l)%50][j][k][i] if l else 0)
                    if new_bs > bs:
                        bs = new_bs
                        bp = (self.r_full[j][k] + i) % 3
            for j in range(2):
                for k in range(2):
                    new_bs = self.p_freq_score[T%50][j][k][i] - (self.p_freq_score[(50+T-l)%50][j][k][i] if l else 0)
                    if new_bs > bs:
                        bs = new_bs
                        bp = (self.p_freq[j][k] + i) % 3
                    new_bs = self.r_freq_score[T%50][j][k][i] - (self.r_freq_score[(50+T-l)%50][j][k][i] if l else 0)
                    if new_bs > bs:
                        bs = new_bs
                        bp = (self.r_freq[j][k] + i) % 3
        return bp

    def next_action(self, TT, A, S):
        if TT > 0:
            self.my_moves.append(self.act)
            self.opp_moves.append(A)

        wins_with = (1,2,0)      #superior
        best_without = (2,0,1)   #inferior

        lengths = (10, 20, 30, 40, 49, 0)
        self.p_random = secrets.randbelow(S)

        score_table =((0,-1,1),(1,0,-1),(-1,1,0))
        T = len(self.opp_moves)  #so T is number of trials completed

        if not self.my_moves:
            self.opp_history = [0]
            self.my_history = [0]
            self.gear = [[0] for _ in range(24)]
            self.p_random_score = 0
            self.p_full_score = [[[[0 for i in range(3)] for k in range(4)] for j in range(24)] for l in range(50)]
            self.r_full_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(24)] for l in range(50)]
            self.p_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
            self.r_freq_score = [[[[0 for i in range(3)] for k in range(2)] for j in range(2)] for l in range(50)]
            self.s_len = [0] * 6

            self.p_full = [[0,0,0,0] for _ in range(24)]
            self.r_full = [[0,0] for _ in range(24)]
        else:
            self.my_history.append(self.my_moves[-1])
            self.opp_history.append(self.opp_moves[-1])
            self.p_random_score += score_table[self.p_random][self.opp_history[-1]]
            self.p_full_score[T%50] = [[[self.p_full_score[(T+49)%50][j][k][i] + score_table[(self.p_full[j][k] + i) % 3][self.opp_history[-1]] for i in range(3)] for k in range(4)] for j in range(24)]
            self.r_full_score[T%50] = [[[self.r_full_score[(T+49)%50][j][k][i] + score_table[(self.r_full[j][k] + i) % 3][self.opp_history[-1]] for i in range(3)] for k in range(2)] for j in range(24)]
            self.p_freq_score[T%50] = [[[self.p_freq_score[(T+49)%50][j][k][i] + score_table[(self.p_freq[j][k] + i) % 3][self.opp_history[-1]] for i in range(3)] for k in range(2)] for j in range(2)]
            self.r_freq_score[T%50] = [[[self.r_freq_score[(T+49)%50][j][k][i] + score_table[(self.r_freq[j][k] + i) % 3][self.opp_history[-1]] for i in range(3)] for k in range(2)] for j in range(2)]
            self.s_len = [s + score_table[p][self.opp_history[-1]] for s,p in zip(self.s_len,self.p_len)]

        if not self.my_moves:
            self.my_history_hash = [[0],[0],[0],[0]]
            self.opp_history_hash = [[0],[0],[0],[0]]
        else:
            self.my_history_hash[0].append(self.my_history[-1])
            self.opp_history_hash[0].append(self.opp_history[-1])
            for i in range(1,4):
                self.my_history_hash[i].append(self.my_history_hash[i-1][-1] * 3 + self.my_history[-1])
                self.opp_history_hash[i].append(self.opp_history_hash[i-1][-1] * 3 + self.opp_history[-1])

        for i in range(24):
            self.gear[i].append((3 + self.opp_history[-1] - self.p_full[i][2]) % 3)
            if T > 1:
                self.gear[i][T] += 3 * self.gear[i][T-1]
            self.gear[i][T] %= 9
        if not self.my_moves:
            self.freq = [[0,0,0],[0,0,0]]
            value = [[0,0,0],[0,0,0]]
        else:
            self.freq[0][self.my_history[-1]] += 1
            self.freq[1][self.opp_history[-1]] += 1
            value = [[(1000 * (self.freq[i][2] - self.freq[i][1])) / float(T),
                      (1000 * (self.freq[i][0] - self.freq[i][2])) / float(T),
                      (1000 * (self.freq[i][1] - self.freq[i][0])) / float(T)] for i in range(2)]
        self.p_freq = [[wins_with[greenberg_agent.max_index(self.freq[i])], wins_with[greenberg_agent.max_index(value[i])]] for i in range(2)]
        self.r_freq = [[best_without[greenberg_agent.min_index(self.freq[i])], best_without[greenberg_agent.min_index(value[i])]] for i in range(2)]

        f = [[[[0,0,0] for k in range(4)] for j in range(2)] for i in range(3)]
        t = [[[0,0,0,0] for j in range(2)] for i in range(3)]

        m_len = [[0 for _ in range(T)] for i in range(3)]

        for i in range(T-1,0,-1):
            m_len[0][i] = 4
            for j in range(4):
                if self.my_history_hash[j][i] != self.my_history_hash[j][T]:
                    m_len[0][i] = j
                    break
            for j in range(4):
                if self.opp_history_hash[j][i] != self.opp_history_hash[j][T]:
                    m_len[1][i] = j
                    break
            for j in range(4):
                if self.my_history_hash[j][i] != self.my_history_hash[j][T] or self.opp_history_hash[j][i] != self.opp_history_hash[j][T]:
                    m_len[2][i] = j
                    break

        for i in range(T-1,0,-1):
            for j in range(3):
                for k in range(m_len[j][i]):
                    f[j][0][k][self.my_history[i+1]] += 1
                    f[j][1][k][self.opp_history[i+1]] += 1
                    t[j][0][k] += 1
                    t[j][1][k] += 1

                    if t[j][0][k] == 1:
                        self.p_full[j*8 + 0*4 + k][0] = wins_with[self.my_history[i+1]]
                    if t[j][1][k] == 1:
                        self.p_full[j*8 + 1*4 + k][0] = wins_with[self.opp_history[i+1]]
                    if t[j][0][k] == 3:
                        self.p_full[j*8 + 0*4 + k][1] = wins_with[greenberg_agent.max_index(f[j][0][k])]
                        self.r_full[j*8 + 0*4 + k][0] = best_without[greenberg_agent.min_index(f[j][0][k])]
                    if t[j][1][k] == 3:
                        self.p_full[j*8 + 1*4 + k][1] = wins_with[greenberg_agent.max_index(f[j][1][k])]
                        self.r_full[j*8 + 1*4 + k][0] = best_without[greenberg_agent.min_index(f[j][1][k])]

        for j in range(3):
            for k in range(4):
                self.p_full[j*8 + 0*4 + k][2] = wins_with[greenberg_agent.max_index(f[j][0][k])]
                self.r_full[j*8 + 0*4 + k][1] = best_without[greenberg_agent.min_index(f[j][0][k])]

                self.p_full[j*8 + 1*4 + k][2] = wins_with[greenberg_agent.max_index(f[j][1][k])]
                self.r_full[j*8 + 1*4 + k][1] = best_without[greenberg_agent.min_index(f[j][1][k])]

        for j in range(24):
            gear_freq = [0] * 9
            for i in range(T-1,0,-1):
                if self.gear[j][i] == self.gear[j][T]:
                    gear_freq[self.gear[j][i+1]] += 1
            self.p_full[j][3] = (self.p_full[j][1] + greenberg_agent.max_index(gear_freq)) % 3

        self.p_len = [self.find_best_prediction(l, T) for l in lengths]
        self.act = self.p_len[greenberg_agent.max_index(self.s_len)]
        return self.act
    
    def history_step(self, history):
            if len(history) > 0:
                move = history[-1]['competitorStep']
            else:
                move = 0
            return self.next_action(len(history), move, 3)
        
class lucker_agent(agent):
    def __init__(self, pred=27, meta = 18):
        self.num_predictors = pred
        self.num_meta = meta
        self.len_rfind = [20]
        self.limit = [10,20,60]
        self.beat = { "P":"S" , "R":"P" , "S":"R" }
        self.not_lose = { "R":"PR", "P":"SP", "S":"RS" } 
        self.your_his =""
        self.my_his = ""
        self.both_his=""
        self.both_his2=""
        self.length =0
        self.score1=[3]*self.num_predictors
        self.score2=[3]*self.num_predictors
        self.score3=[3]*self.num_predictors
        self.score4=[3]*self.num_predictors
        self.score5=[3]*self.num_predictors
        self.score6=[3]*self.num_predictors
        self.metascore=[3]*self.num_meta
        self.temp1 = { "PP":"1","PR":"2","PS":"3",
              "RP":"4","RR":"5","RS":"6",
              "SP":"7","SR":"8","SS":"9"}
        self.temp2 = { "1":"PP","2":"PR","3":"PS",
                "4":"RP","5":"RR","6":"RS",
                "7":"SP","8":"SR","9":"SS"} 
        self.who_win = { "PP": 0, "PR":1 , "PS":-1,
                "RP": -1,"RR":0, "RS":1,
                "SP": 1, "SR":-1, "SS":0}
        self.index = { "P":0, "R":1, "S":2 }
        self.chance =[0]*self.num_predictors
        self.chance2 =[0]*self.num_predictors

    def next_action(self, input, output):
        if input == "":
            output = random.choice("RPS")
            self.predictors = [output]*self.num_predictors
            self.metapredictors = [output]*self.num_meta
            return output
        else:
            for i in range(self.num_predictors):
                #meta 1
                self.score1[i]*=0.8
                if input==self.predictors[i]:
                    self.score1[i]+=3
                else:
                    self.score1[i]-=3
                #meta 2
                if input==self.predictors[i]:
                    self.score2[i]+=3
                else:
                    self.score2[i]=0
                #meta 3
                self.score3[i]*=0.8
                if output==self.predictors[i]:
                    self.score3[i]+=3
                else:
                    self.score3[i]-=3
                #meta 4
                if output==self.predictors[i]:
                    self.score4[i]+=3
                else:
                    self.score4[i]=0
                #meta 5
                self.score5[i]*=0.8
                if input==self.predictors[i]:
                    self.score5[i]+=3
                else:
                    if self.chance[i]==1:
                        self.chance[i]=0
                        self.score5[i]-=3
                    else:
                        self.chance[i]=1
                        self.score5[i]=0
                #meta 6
                self.score6[i]*=0.8
                if output==self.predictors[i]:
                    self.score6[i]+=3
                else:
                    if self.chance2[i]==1:
                        self.chance2[i]=0
                        self.score6[i]-=3
                    else:
                        self.chance2[i]=1
                        self.score6[i]=0
            #calculate metascore
            for i in range(self.num_meta):
                self.metascore[i]*=0.9
                if input==self.metapredictors[i]:
                    self.metascore[i]+=3
                else:
                    self.metascore[i]=0
            #Predictors
            #if length>1:
            #    output=beat[predict]
            self.your_his+=input
            self.my_his+=output
            self.both_his+=self.temp1[(input+output)]
            self.both_his2+=self.temp1[(output+input)]
            self.length+=1

            #history matching
            for i in range(1):
                self.len_size = min(self.length,self.len_rfind[i])
                j=self.len_size
                #both_his
                while j>=1 and not self.both_his[self.length-j:self.length] in self.both_his[0:self.length-1]:
                    j-=1
                if j>=1:
                    k = self.both_his.rfind(self.both_his[self.length-j:self.length],0,self.length-1)
                    self.predictors[0+6*i] = self.your_his[j+k]
                    self.predictors[1+6*i] = self.beat[self.my_his[j+k]]
                else:
                    self.predictors[0+6*i] = random.choice("RPS")
                    self.predictors[1+6*i] = random.choice("RPS")
                j=self.len_size
                #your_his
                while j>=1 and not self.your_his[self.length-j:self.length] in self.your_his[0:self.length-1]:
                    j-=1
                if j>=1:
                    k = self.your_his.rfind(self.your_his[self.length-j:self.length],0,self.length-1)
                    self.predictors[2+6*i] = self.your_his[j+k]
                    self.predictors[3+6*i] = self.beat[self.my_his[j+k]]
                else:
                    self.predictors[2+6*i] = random.choice("RPS")
                    self.predictors[3+6*i] = random.choice("RPS")
                j=self.len_size
                #my_his
                while j>=1 and not self.my_his[self.length-j:self.length] in self.my_his[0:self.length-1]:
                    j-=1
                if j>=1:
                    k = self.my_his.rfind(self.my_his[self.length-j:self.length],0,self.length-1)
                    self.predictors[4+6*i] = self.your_his[j+k]
                    self.predictors[5+6*i] = self.beat[self.my_his[j+k]]
                else:
                    self.predictors[4+6*i] = random.choice("RPS")
                    self.predictors[5+6*i] = random.choice("RPS")
    
            #Reverse
            for i in range(3):
                temp =""
                search = self.temp1[(output+input)] #last round
                for start in range(2, min(self.limit[i],self.length) ):
                    if search == self.both_his2[self.length-start]:
                        temp+=self.both_his2[self.length-start+1]
                if(temp==""):
                    self.predictors[6+i] = random.choice("RPS")
                else:
                    collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
                    for sdf in temp:
                        next_move = self.temp2[sdf]
                        if(self.who_win[next_move]==-1):
                            collectR[self.temp2[sdf][1]]+=3
                        elif(self.who_win[next_move]==0):
                            collectR[self.temp2[sdf][1]]+=1
                        elif(self.who_win[next_move]==1):
                            collectR[self.beat[self.temp2[sdf][0]]]+=1
                    max1 = -1
                    p1 =""
                    for key in collectR:
                        if(collectR[key]>max1):
                            max1 = collectR[key]
                            p1 += key
                    self.predictors[6+i] = random.choice(p1)
    
            for i in range(9,27):
                self.predictors[i]=self.beat[self.beat[self.predictors[i-9]]]
            #find prediction for each meta
            self.metapredictors[0]=self.predictors[self.score1.index(max(self.score1))]
            self.metapredictors[1]=self.predictors[self.score2.index(max(self.score2))]
            self.metapredictors[2]=self.beat[self.predictors[self.score3.index(max(self.score3))]]
            self.metapredictors[3]=self.beat[self.predictors[self.score4.index(max(self.score4))]]
            self.metapredictors[4]=self.predictors[self.score5.index(max(self.score5))]
            self.metapredictors[5]=self.beat[self.predictors[self.score6.index(max(self.score6))]]
            for i in range(6,18):
                self.metapredictors[i] = self.beat[self.metapredictors[i-6]]
    
            predict = self.metapredictors[self.metascore.index(max(self.metascore))]
            output = self.beat[predict]
            return output
        
        def history_step(self, history):
            inp = ""
            out = ""
            if len(history) > 0:
                inp = history[-1]['competitorStep']
                out = history[-1]['step']
            return self.next_action(inp, out)
        
class ignore_agent(agent):
    def __init__(self):
        self.score  = {'RR': 0, 'PP': 0, 'SS': 0, \
                  'PR': 1, 'RS': 1, 'SP': 1, \
                  'RP': -1, 'SR': -1, 'PS': -1,}
        self.cscore = {'RR': 'r', 'PP': 'r', 'SS': 'r', \
                  'PR': 'b', 'RS': 'b', 'SP': 'b', \
                  'RP': 'c', 'SR': 'c', 'PS': 'c',}
        self.beat = {'P': 'S', 'S': 'R', 'R': 'P'}
        self.cede = {'P': 'R', 'S': 'P', 'R': 'S'}
        self.rps = ['R', 'P', 'S']
        self.wlt = {1: 0, -1: 1, 0: 2}

        self.played_probs = collections.defaultdict(lambda: 1)
        self.dna_probs = [
            collections.defaultdict(lambda: collections.defaultdict(lambda: 1)) for i in range(18)
        ]
        self.wlt_probs = [collections.defaultdict(lambda: 1) for i in range(9)]
        self.answers = [{'c': 1, 'b': 1, 'r': 1} for i in range(12)]
        self.patterndict = [collections.defaultdict(str) for i in range(6)]
        self.consec_strat_usage = [[0] * 6, [0] * 6,
                                   [0] * 6]  #consecutive strategy usage
        self.consec_strat_candy = [[], [], []]  #consecutive strategy candidates
        self.histories = ["", "", ""]
        self.dna = ["" for i in range(12)]
        self.sc = 0
        self.strats = [[] for i in range(3)]
        
    def reset(self):
        self.score  = {'RR': 0, 'PP': 0, 'SS': 0, \
                  'PR': 1, 'RS': 1, 'SP': 1, \
                  'RP': -1, 'SR': -1, 'PS': -1,}
        self.cscore = {'RR': 'r', 'PP': 'r', 'SS': 'r', \
                  'PR': 'b', 'RS': 'b', 'SP': 'b', \
                  'RP': 'c', 'SR': 'c', 'PS': 'c',}
        self.beat = {'P': 'S', 'S': 'R', 'R': 'P'}
        self.cede = {'P': 'R', 'S': 'P', 'R': 'S'}
        self.rps = ['R', 'P', 'S']
        self.wlt = {1: 0, -1: 1, 0: 2}

        self.played_probs = collections.defaultdict(lambda: 1)
        self.dna_probs = [
            collections.defaultdict(lambda: collections.defaultdict(lambda: 1)) for i in range(18)
        ]
        self.wlt_probs = [collections.defaultdict(lambda: 1) for i in range(9)]
        self.answers = [{'c': 1, 'b': 1, 'r': 1} for i in range(12)]
        self.patterndict = [collections.defaultdict(str) for i in range(6)]
        self.consec_strat_usage = [[0] * 6, [0] * 6,
                                   [0] * 6]  #consecutive strategy usage
        self.consec_strat_candy = [[], [], []]  #consecutive strategy candidates
        self.histories = ["", "", ""]
        self.dna = ["" for i in range(12)]
        self.sc = 0
        self.strats = [[] for i in range(3)]
        return
        
    def counter_prob(self, probs):
        weighted_list = []
        for h in self.rps:
            weighted = 0
            for p in probs.keys():
                points = self.score[h + p]
                prob = probs[p]
                weighted += points * prob
            weighted_list.append((h, weighted))
        return max(weighted_list, key=itemgetter(1))[0]
        
    def next_action(self, T, A, S):
        if T == 0:
            self.B = random.choice(self.rps)
            return {'R': 0, 'P': 1, 'S': 2}[self.B]
        prev_sc = self.sc

        self.sc = self.score[self.B + 'RPS'[A]]
        for j in range(3):
            prev_strats = self.strats[j][:]
            for i, c in enumerate(self.consec_strat_candy[j]):
                if c == 'RPS'[A]:
                    self.consec_strat_usage[j][i] += 1
                else:
                    self.consec_strat_usage[j][i] = 0
            m = max(self.consec_strat_usage[j])
            self.strats[j] = [
                i for i, c in enumerate(self.consec_strat_candy[j])
                if self.consec_strat_usage[j][i] == m
            ]

            for s1 in prev_strats:
                for s2 in self.strats[j]:
                    self.wlt_probs[j * 3 + self.wlt[prev_sc]][chr(s1) + chr(s2)] += 1

            if self.dna[2 * j + 0] and self.dna[2 * j + 1]:
                self.answers[2 * j + 0][self.cscore['RPS'[A] + self.dna[2 * j + 0]]] += 1
                self.answers[2 * j + 1][self.cscore['RPS'[A] + self.dna[2 * j + 1]]] += 1
            if self.dna[2 * j + 6] and self.dna[2 * j + 7]:
                self.answers[2 * j + 6][self.cscore['RPS'[A] + self.dna[2 * j + 6]]] += 1
                self.answers[2 * j + 7][self.cscore['RPS'[A] + self.dna[2 * j + 7]]] += 1

            for length in range(min(10, len(self.histories[j])), 0, -2):
                pattern = self.patterndict[2 * j][self.histories[j][-length:]]
                if pattern:
                    for length2 in range(min(10, len(pattern)), 0, -2):
                        self.patterndict[2 * j + 1][pattern[-length2:]] += self.B + 'RPS'[A]
                self.patterndict[2 * j][self.histories[j][-length:]] += self.B + 'RPS'[A]
        self.played_probs['RPS'[A]] += 1
        self.dna_probs[0][self.dna[0]]['RPS'[A]] += 1
        self.dna_probs[1][self.dna[1]]['RPS'[A]] += 1
        self.dna_probs[2][self.dna[1] + self.dna[0]]['RPS'[A]] += 1
        self.dna_probs[9][self.dna[6]]['RPS'[A]] += 1
        self.dna_probs[10][self.dna[6]]['RPS'[A]] += 1
        self.dna_probs[11][self.dna[7] + self.dna[6]]['RPS'[A]] += 1

        self.histories[0] += self.B + 'RPS'[A]
        self.histories[1] += 'RPS'[A]
        self.histories[2] += self.B

        self.dna = ["" for i in range(12)]
        for j in range(3):
            for length in range(min(10, len(self.histories[j])), 0, -2):
                pattern = self.patterndict[2 * j][self.histories[j][-length:]]
                if pattern != "":
                    self.dna[2 * j + 1] = pattern[-2]
                    self.dna[2 * j + 0] = pattern[-1]
                    for length2 in range(min(10, len(pattern)), 0, -2):
                        pattern2 = self.patterndict[2 * j + 1][pattern[-length2:]]
                        if pattern2 != "":
                            self.dna[2 * j + 7] = pattern2[-2]
                            self.dna[2 * j + 6] = pattern2[-1]
                            break
                    break

        probs = {}
        for hand in self.rps:
            probs[hand] = self.played_probs[hand]

        for j in range(3):
            if self.dna[j * 2] and self.dna[j * 2 + 1]:
                for hand in self.rps:
                    probs[hand] *= self.dna_probs[j*3+0][self.dna[j*2+0]][hand] * \
                                   self.dna_probs[j*3+1][self.dna[j*2+1]][hand] * \
                          self.dna_probs[j*3+2][self.dna[j*2+1]+self.dna[j*2+0]][hand]
                    probs[hand] *= self.answers[j*2+0][self.cscore[hand+self.dna[j*2+0]]] * \
                                   self.answers[j*2+1][self.cscore[hand+self.dna[j*2+1]]]
                self.consec_strat_candy[j] = [self.dna[j*2+0], self.beat[self.dna[j*2+0]], self.cede[self.dna[j*2+0]],\
                                         self.dna[j*2+1], self.beat[self.dna[j*2+1]], self.cede[self.dna[j*2+1]]]
                strats_for_hand = {'R': [], 'P': [], 'S': []}
                for i, c in enumerate(self.consec_strat_candy[j]):
                    strats_for_hand[c].append(i)
                pr = self.wlt_probs[self.wlt[self.sc] + 3 * j]
                for hand in self.rps:
                    for s1 in self.strats[j]:
                        for s2 in strats_for_hand[hand]:
                            probs[hand] *= pr[chr(s1) + chr(s2)]
            else:
                self.consec_strat_candy[j] = []
        for j in range(3):
            if self.dna[j * 2 + 6] and self.dna[j * 2 + 7]:
                for hand in self.rps:
                    probs[hand] *= self.dna_probs[j*3+9][self.dna[j*2+6]][hand] * \
                                   self.dna_probs[j*3+10][self.dna[j*2+7]][hand] * \
                          self.dna_probs[j*3+11][self.dna[j*2+7]+self.dna[j*2+6]][hand]
                    probs[hand] *= self.answers[j*2+6][self.cscore[hand+self.dna[j*2+6]]] * \
                                   self.answers[j*2+7][self.cscore[hand+self.dna[j*2+7]]]

        self.B = self.counter_prob(probs)
        return {'R': 0, 'P': 1, 'S': 2}[self.B]
    
    def history_step(self, history):
            if len(history) > 0:
                move = history[-1]['competitorStep']
            else:
                move = 0
            return self.next_action(len(history), move, 3)
        
class NumpyPatterns(agent):
    def __init__(self):
        self.B = 0
        # Jitter - steps before next non-random move
        self.Jmax = 2
        self.J2 = (self.Jmax+1)**2
        self.J = self.Jmax - int(math.sqrt(secrets.randbelow(self.J2)))
        # Depth - number of previous steps taken into consideration
        self.Dmin = 1
        self.Dmax = 3
        self.DL = self.Dmax-self.Dmin+1
        self.HL = 3
        self.HText = ['Opp',  'Me', 'Score']
        self.Depth = np.arange(self.DL)
        self.Hash = np.zeros((self.HL, self.DL), dtype=int)
        self.G = 2
        self.R = 0.4
        self.RG = (1-self.R) * self.G
        self.Threshold = 0.4
        
    def reset(self):
        self.B = 0
        # Jitter - steps before next non-random move
        self.Jmax = 2
        self.J2 = (self.Jmax+1)**2
        self.J = self.Jmax - int(math.sqrt(secrets.randbelow(self.J2)))
        # Depth - number of previous steps taken into consideration
        self.Dmin = 1
        self.Dmax = 3
        self.DL = self.Dmax-self.Dmin+1
        self.HL = 3
        self.HText = ['Opp',  'Me', 'Score']
        self.Depth = np.arange(self.DL)
        self.Hash = np.zeros((self.HL, self.DL), dtype=int)
        self.G = 2
        self.R = 0.4
        self.RG = (1-self.R) * self.G
        self.Threshold = 0.4
        
    def split_idx(self, idx):
        d = idx % self.DL
        idx //= self.DL
        h2 = idx % self.HL
        idx //= self.HL
        h1 = idx % self.HL
        idx //= self.HL
        return d, h1, h2, idx
    
    def next_action(self, T, A, S):
        B, HL, DL, Dmin, Dmax = self.B, self.HL, self.DL, self.Dmin, self.Dmax
        SD = S**self.DL
        if T == 0:
            self.Map = np.zeros((S, SD**2, HL, HL, DL))
            self.SList = np.arange(S)[:,None,None,None]
            self.Predicts = np.full((HL, HL, DL), S, dtype=int)
            self.Attempts = np.zeros((HL, HL, DL), dtype=int)
            self.Scores = np.zeros((S, HL, HL, DL))
            self.OrgID = np.ogrid[:S, :HL, :HL, :DL]
            self.Hash2 = self.Hash[None,:] + SD*self.Hash[:,None]
        else:
            C = get_score(S, A, B) + 1
            ABC = np.array([A, B, C])[:,None]
            Depth, Hash, Hash2, Map, SList, OrgID, Predicts, Attempts, Scores = self.Depth, self.Hash, self.Hash2, self.Map, self.SList, self.OrgID, self.Predicts, self.Attempts, self.Scores
            # Update Moves Map by previous move and previous Hash
            Map *= 0.995
            Map[OrgID[0], Hash2, OrgID[1], OrgID[2], OrgID[3]] += (T > Depth + Dmin) * (SList == A)
            # Update Hash by previous move
            Hash[:] //= S
            Hash[:] += ABC[:HL] * S**Depth
            Hash2[:] = Hash[None,:] + SD*Hash[:,None]
            
            # Update prediction scores by previous move
            PB = Predicts < S
            Attempts[:] = Attempts + PB
            Scores[:] += PB * get_score(S, Predicts + SList, A)
            #print(T, Scores.T[0])
            # Update prediction scores by previous move
            PR = Map[OrgID[0], Hash2, OrgID[1], OrgID[2], OrgID[3]]
            Sum = np.sum(PR, axis=0)
            Predicts[:] = (np.max((Sum >= self.G) * (PR >= Sum * self.R + self.RG) * (SList + 1), axis=0) - 1) % (S + 1)

        self.B = np.random.choice(S)
        if self.J > 0:
            self.J -= 1
        else:
            sc = np.where(self.Predicts < S, self.Scores / (self.Attempts + 2), 0).ravel()
            idx = np.argmax(sc)
            if sc[idx] > self.Threshold:
                Raw = self.Predicts.ravel()
                L = len(Raw)
                self.B = (Raw[idx % L] + idx // L) % S
                self.J = self.Jmax - int(math.sqrt(secrets.randbelow(self.J2)))
                #parts = self.split_idx(idx)
                #print(T, f'{parts[0]+self.Dmin}: {self.HText[parts[1]]}-{self.HText[parts[2]]}+{parts[3]}', self.Scores[:, parts[1], parts[2], parts[0]], self.B)
        return self.B
    
    def history_step(self, history):
            if len(history) > 0:
                move = history[-1]['competitorStep']
            else:
                move = 0
            return self.next_action(len(history), move, 3)

def FSM_agentX(step, prev_friend_act, prev_foe_act, state, filter_move, state_choices, action_choices):

    # num_actions == 3
    holdoff = 0 # spin at state zero until past this step

    if step == 0:
        condition = 2 # tie
    if step <= holdoff:
        condition = 2 # tie
        #    action = np.int32(np.random.randint(3))
        state = state_choices[state, condition] # get next state
        #a = np.int32(action_choices[state]) # get action to play
        action = np.int32(np.random.randint(3))
        state = 0 # remain at state 0
        #if (showit): print('FSM_agentX() step 0. action', action, 'state', state)
        return (state, action)
    
    # find new state based on condion (win, lose, tie)
    if (prev_friend_act == prev_foe_act): condition = 2 # tie
    elif (prev_friend_act == (prev_foe_act +1) % 3): condition = 0 # win
    else: condition = 1 # lose
    
    #if (condition != 0): #only change state on a loss or tie
    state = state_choices[state, condition] # get next state
    a = np.int32(action_choices[state]) # get strategy to play
        
    if (a < 3): # pick action relative to filter_move
        action = (filter_move + a) % 3
    else: action = int(np.random.randint(3))
    return (state, action)

class INDIVIDUAL():
    def __init__(self):
        self.N = 200 # number of states
        self.action_choices = np.zeros((self.N), np.int32)    # coded state table (0-4)
        self.state_choices  = np.zeros((self.N, 3), np.int32) # coded state table (0-N)

class anti_greenberg(agent):
    def __init__(self):
        self.green = greenberg_agent()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL() 
        self.F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00q\rtq\x0eb.')
        self.F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00`\x00\x00\x00\x13\x00\x00\x00\x1c\x00\x00\x00}\x00\x00\x00\xbd\x00\x00\x00c\x00\x00\x00\x97\x00\x00\x00\x93\x00\x00\x00\x84\x00\x00\x00G\x00\x00\x00`\x00\x00\x00\x10\x00\x00\x00p\x00\x00\x00\xad\x00\x00\x00\xab\x00\x00\x00\xb2\x00\x00\x00o\x00\x00\x00\xa8\x00\x00\x00\x99\x00\x00\x00\x88\x00\x00\x00\x15\x00\x00\x00\xbc\x00\x00\x00-\x00\x00\x00?\x00\x00\x00,\x00\x00\x00\x97\x00\x00\x00\x1c\x00\x00\x00@\x00\x00\x00\xa8\x00\x00\x00\x8f\x00\x00\x00\xb9\x00\x00\x00\xb8\x00\x00\x00F\x00\x00\x00\xb4\x00\x00\x008\x00\x00\x00\x8e\x00\x00\x00\x13\x00\x00\x00\x12\x00\x00\x00\xa5\x00\x00\x00\x04\x00\x00\x00\x10\x00\x00\x00\xab\x00\x00\x00\x12\x00\x00\x00\x95\x00\x00\x00\x9a\x00\x00\x00\\\x00\x00\x00\xa1\x00\x00\x00\x0f\x00\x00\x00\x04\x00\x00\x00\x90\x00\x00\x00\x9a\x00\x00\x00\'\x00\x00\x00O\x00\x00\x00f\x00\x00\x00\xc4\x00\x00\x00J\x00\x00\x00\x8f\x00\x00\x00\x9a\x00\x00\x00\x98\x00\x00\x00p\x00\x00\x00\xa1\x00\x00\x00[\x00\x00\x00\x0c\x00\x00\x00\xc6\x00\x00\x00\x06\x00\x00\x00\x8a\x00\x00\x00\x90\x00\x00\x00\xc6\x00\x00\x00L\x00\x00\x00\xb3\x00\x00\x00D\x00\x00\x00p\x00\x00\x00k\x00\x00\x00v\x00\x00\x00}\x00\x00\x00\xa5\x00\x00\x00\x04\x00\x00\x00\xa5\x00\x00\x00X\x00\x00\x00\x13\x00\x00\x00\xc5\x00\x00\x00z\x00\x00\x00\xc6\x00\x00\x00\xa0\x00\x00\x00-\x00\x00\x00U\x00\x00\x00j\x00\x00\x00L\x00\x00\x00\n\x00\x00\x00\x06\x00\x00\x00i\x00\x00\x00\x14\x00\x00\x00f\x00\x00\x00\x00\x00\x00\x00[\x00\x00\x00\x84\x00\x00\x00\x19\x00\x00\x00\xa5\x00\x00\x00\\\x00\x00\x00\xb7\x00\x00\x00n\x00\x00\x00\xb0\x00\x00\x00\x8e\x00\x00\x00\x1d\x00\x00\x00\x7f\x00\x00\x00\xa9\x00\x00\x00\xc2\x00\x00\x00\x99\x00\x00\x00\xae\x00\x00\x00\x9f\x00\x00\x00\x1a\x00\x00\x00{\x00\x00\x00U\x00\x00\x00@\x00\x00\x00\xb7\x00\x00\x00.\x00\x00\x00]\x00\x00\x00X\x00\x00\x00\xbe\x00\x00\x00\xb4\x00\x00\x00r\x00\x00\x00\x03\x00\x00\x00|\x00\x00\x00\xad\x00\x00\x00E\x00\x00\x00+\x00\x00\x00j\x00\x00\x00\x88\x00\x00\x007\x00\x00\x00\x83\x00\x00\x00k\x00\x00\x00\xc1\x00\x00\x00I\x00\x00\x00I\x00\x00\x00\r\x00\x00\x00o\x00\x00\x00n\x00\x00\x00\xab\x00\x00\x00\x9d\x00\x00\x00N\x00\x00\x00\'\x00\x00\x00\x07\x00\x00\x00\x10\x00\x00\x00T\x00\x00\x00P\x00\x00\x007\x00\x00\x00.\x00\x00\x00\xc4\x00\x00\x00|\x00\x00\x00B\x00\x00\x00\xc0\x00\x00\x009\x00\x00\x00m\x00\x00\x00d\x00\x00\x00\x11\x00\x00\x00\xc3\x00\x00\x00\xc3\x00\x00\x00\xa5\x00\x00\x00\x00\x00\x00\x00\x82\x00\x00\x00=\x00\x00\x00;\x00\x00\x00\x88\x00\x00\x00k\x00\x00\x00(\x00\x00\x00y\x00\x00\x00c\x00\x00\x00\x1f\x00\x00\x00\x1e\x00\x00\x00\xb8\x00\x00\x00\xa5\x00\x00\x00\xa0\x00\x00\x00"\x00\x00\x00\x11\x00\x00\x00\xc7\x00\x00\x00l\x00\x00\x00"\x00\x00\x00G\x00\x00\x00\x89\x00\x00\x00$\x00\x00\x00\xac\x00\x00\x00\x8a\x00\x00\x00&\x00\x00\x00m\x00\x00\x00Y\x00\x00\x00\x8f\x00\x00\x000\x00\x00\x00A\x00\x00\x00O\x00\x00\x00\xa9\x00\x00\x00\x8a\x00\x00\x00\xa3\x00\x00\x00\xa4\x00\x00\x00$\x00\x00\x00\xaf\x00\x00\x00\x02\x00\x00\x00\x0f\x00\x00\x00\r\x00\x00\x00-\x00\x00\x00\x1c\x00\x00\x00\x97\x00\x00\x00\x8e\x00\x00\x00t\x00\x00\x006\x00\x00\x00Z\x00\x00\x00`\x00\x00\x00\x83\x00\x00\x00\x02\x00\x00\x00\xad\x00\x00\x00\x86\x00\x00\x00\x8f\x00\x00\x00j\x00\x00\x00I\x00\x00\x00\x94\x00\x00\x00\x1b\x00\x00\x00/\x00\x00\x00!\x00\x00\x00D\x00\x00\x00M\x00\x00\x00:\x00\x00\x00\x7f\x00\x00\x009\x00\x00\x00\x16\x00\x00\x00A\x00\x00\x00k\x00\x00\x006\x00\x00\x00\x8c\x00\x00\x00-\x00\x00\x00\x90\x00\x00\x00/\x00\x00\x00\x9a\x00\x00\x00U\x00\x00\x00c\x00\x00\x00\xc4\x00\x00\x00J\x00\x00\x00>\x00\x00\x00^\x00\x00\x00\x06\x00\x00\x00G\x00\x00\x00\xb7\x00\x00\x00\x97\x00\x00\x00\xbb\x00\x00\x00F\x00\x00\x00\xa6\x00\x00\x00\x1b\x00\x00\x00x\x00\x00\x00\xc4\x00\x00\x00\x90\x00\x00\x00\x8f\x00\x00\x00*\x00\x00\x00\x05\x00\x00\x00\x96\x00\x00\x00\xbf\x00\x00\x00&\x00\x00\x00B\x00\x00\x00t\x00\x00\x00Y\x00\x00\x00j\x00\x00\x00\x08\x00\x00\x00\x04\x00\x00\x00j\x00\x00\x00\xa9\x00\x00\x00*\x00\x00\x00\x9b\x00\x00\x00\xbb\x00\x00\x00L\x00\x00\x00\xbd\x00\x00\x00\x8f\x00\x00\x00(\x00\x00\x00\xa6\x00\x00\x00\xa8\x00\x00\x00\\\x00\x00\x00\xae\x00\x00\x00j\x00\x00\x00\xc1\x00\x00\x00\xc1\x00\x00\x008\x00\x00\x00i\x00\x00\x00d\x00\x00\x00\xaa\x00\x00\x00\x11\x00\x00\x00J\x00\x00\x00F\x00\x00\x00\x04\x00\x00\x00d\x00\x00\x00]\x00\x00\x00+\x00\x00\x00\\\x00\x00\x00\xaf\x00\x00\x00.\x00\x00\x00J\x00\x00\x00\x9d\x00\x00\x00\x83\x00\x00\x00\xac\x00\x00\x00k\x00\x00\x00O\x00\x00\x00\xc2\x00\x00\x00\x96\x00\x00\x00\x88\x00\x00\x00\xa4\x00\x00\x00\x08\x00\x00\x00\x83\x00\x00\x00d\x00\x00\x00\xbf\x00\x00\x00\xa0\x00\x00\x00\x92\x00\x00\x00\x99\x00\x00\x00R\x00\x00\x009\x00\x00\x00r\x00\x00\x00\x1e\x00\x00\x00\xac\x00\x00\x00\x19\x00\x00\x00\x92\x00\x00\x005\x00\x00\x00\x08\x00\x00\x00w\x00\x00\x00\x17\x00\x00\x00J\x00\x00\x00\x87\x00\x00\x00@\x00\x00\x00\x1f\x00\x00\x00&\x00\x00\x00\xa3\x00\x00\x00\x16\x00\x00\x00\x90\x00\x00\x00\x1d\x00\x00\x00>\x00\x00\x00F\x00\x00\x00\x90\x00\x00\x005\x00\x00\x00\xa5\x00\x00\x00X\x00\x00\x00\x90\x00\x00\x00\x98\x00\x00\x00O\x00\x00\x00\x07\x00\x00\x00h\x00\x00\x00\xae\x00\x00\x00\xc0\x00\x00\x00\x06\x00\x00\x00\xb9\x00\x00\x000\x00\x00\x00\x99\x00\x00\x00\xbd\x00\x00\x00;\x00\x00\x00\x99\x00\x00\x00\xc1\x00\x00\x00\x04\x00\x00\x00\xa2\x00\x00\x00%\x00\x00\x00n\x00\x00\x00V\x00\x00\x00|\x00\x00\x00}\x00\x00\x00K\x00\x00\x00\n\x00\x00\x00\x86\x00\x00\x00\xaa\x00\x00\x00{\x00\x00\x000\x00\x00\x00<\x00\x00\x00n\x00\x00\x00\x95\x00\x00\x00e\x00\x00\x00O\x00\x00\x00R\x00\x00\x00z\x00\x00\x00\x8a\x00\x00\x00y\x00\x00\x00\xb8\x00\x00\x00\xc2\x00\x00\x00J\x00\x00\x00\x80\x00\x00\x00d\x00\x00\x00g\x00\x00\x00\xac\x00\x00\x005\x00\x00\x00\x15\x00\x00\x00L\x00\x00\x00\x94\x00\x00\x00>\x00\x00\x00\xbc\x00\x00\x00%\x00\x00\x00\xb8\x00\x00\x00\xbd\x00\x00\x00\x8e\x00\x00\x00|\x00\x00\x00\x07\x00\x00\x00\x90\x00\x00\x00|\x00\x00\x002\x00\x00\x00\xc1\x00\x00\x00\xba\x00\x00\x00\x0b\x00\x00\x00A\x00\x00\x00`\x00\x00\x00\x7f\x00\x00\x00z\x00\x00\x00]\x00\x00\x00i\x00\x00\x00O\x00\x00\x00\xc1\x00\x00\x00a\x00\x00\x00@\x00\x00\x000\x00\x00\x00\x1a\x00\x00\x00L\x00\x00\x00\xbf\x00\x00\x008\x00\x00\x00W\x00\x00\x00\x85\x00\x00\x00~\x00\x00\x00\x07\x00\x00\x00\x88\x00\x00\x00#\x00\x00\x00\x19\x00\x00\x00&\x00\x00\x00\xac\x00\x00\x00\x19\x00\x00\x00\xa7\x00\x00\x00\x08\x00\x00\x00\x01\x00\x00\x00\x90\x00\x00\x00\xb3\x00\x00\x00T\x00\x00\x00d\x00\x00\x00\x84\x00\x00\x00\xb5\x00\x00\x004\x00\x00\x00\\\x00\x00\x00\x00\x00\x00\x00\xa1\x00\x00\x00\x7f\x00\x00\x00W\x00\x00\x00~\x00\x00\x00\x1c\x00\x00\x00\x12\x00\x00\x00\xa1\x00\x00\x00\xae\x00\x00\x00\x85\x00\x00\x00\x9e\x00\x00\x00\x00\x00\x00\x00]\x00\x00\x00\xb3\x00\x00\x00l\x00\x00\x00*\x00\x00\x00 \x00\x00\x00\x0b\x00\x00\x001\x00\x00\x00I\x00\x00\x00r\x00\x00\x00$\x00\x00\x00x\x00\x00\x00,\x00\x00\x00\x13\x00\x00\x00\x0c\x00\x00\x00\x06\x00\x00\x00\x19\x00\x00\x00H\x00\x00\x00Y\x00\x00\x00/\x00\x00\x00\xb3\x00\x00\x00S\x00\x00\x00{\x00\x00\x00:\x00\x00\x00\x18\x00\x00\x00p\x00\x00\x00B\x00\x00\x00\x98\x00\x00\x00y\x00\x00\x00$\x00\x00\x00k\x00\x00\x00i\x00\x00\x00"\x00\x00\x00H\x00\x00\x00A\x00\x00\x00@\x00\x00\x00+\x00\x00\x00q\x00\x00\x00)\x00\x00\x00\xa2\x00\x00\x00\x89\x00\x00\x00(\x00\x00\x00\xc0\x00\x00\x00\\\x00\x00\x00H\x00\x00\x00\x17\x00\x00\x00\xba\x00\x00\x00\xaf\x00\x00\x00,\x00\x00\x00}\x00\x00\x00C\x00\x00\x00g\x00\x00\x00r\x00\x00\x00&\x00\x00\x00\xb5\x00\x00\x00\x82\x00\x00\x00%\x00\x00\x00o\x00\x00\x00\x99\x00\x00\x00\xb7\x00\x00\x00\xc2\x00\x00\x00\x07\x00\x00\x00\x02\x00\x00\x00\x0c\x00\x00\x00a\x00\x00\x00Q\x00\x00\x00j\x00\x00\x00\x0b\x00\x00\x00\n\x00\x00\x00\xb9\x00\x00\x00_\x00\x00\x00\x14\x00\x00\x00\x8b\x00\x00\x00\x95\x00\x00\x00\xa7\x00\x00\x00\x87\x00\x00\x00\x01\x00\x00\x00\x06\x00\x00\x00\xa4\x00\x00\x00\xc6\x00\x00\x00<\x00\x00\x00\x96\x00\x00\x00\xbb\x00\x00\x00<\x00\x00\x00\xc7\x00\x00\x00\xad\x00\x00\x003\x00\x00\x00\x14\x00\x00\x00\xb3\x00\x00\x00\n\x00\x00\x00\x13\x00\x00\x00\x17\x00\x00\x00\x80\x00\x00\x00I\x00\x00\x00\x03\x00\x00\x00\r\x00\x00\x00\n\x00\x00\x00|\x00\x00\x00\xbf\x00\x00\x00\x0e\x00\x00\x000\x00\x00\x002\x00\x00\x00T\x00\x00\x00g\x00\x00\x00x\x00\x00\x00?\x00\x00\x00\xb6\x00\x00\x00\x89\x00\x00\x005\x00\x00\x00\x81\x00\x00\x00b\x00\x00\x00\xb0\x00\x00\x00"\x00\x00\x00\xa0\x00\x00\x00\xb7\x00\x00\x00$\x00\x00\x00_\x00\x00\x00\xbf\x00\x00\x00n\x00\x00\x00x\x00\x00\x004\x00\x00\x000\x00\x00\x00>\x00\x00\x00\xac\x00\x00\x00C\x00\x00\x00\xc0\x00\x00\x00\'\x00\x00\x00~\x00\x00\x00D\x00\x00\x00\x02\x00\x00\x00k\x00\x00\x00\xbf\x00\x00\x00\x83\x00\x00\x00\xa6\x00\x00\x00E\x00\x00\x00Y\x00\x00\x00\xb2\x00\x00\x00\xc2\x00\x00\x00L\x00\x00\x00-\x00\x00\x00~\x00\x00\x00m\x00\x00\x00\x8e\x00\x00\x00\x0c\x00\x00\x00\x8d\x00\x00\x00:\x00\x00\x00\x13\x00\x00\x00\xa1\x00\x00\x00"\x00\x00\x00@\x00\x00\x00M\x00\x00\x00\xc2\x00\x00\x00\x12\x00\x00\x00I\x00\x00\x00\n\x00\x00\x00.\x00\x00\x00$\x00\x00\x00\x8b\x00\x00\x00q\rtq\x0eb.')
        
    def FSM_move(self, s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A, history):
        filter_prediction = self.green.history_step(history)
        Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
        return(Astate, friend_move)

    def reset(self):
        self.green = greenberg_agent()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL()
        
    def history_step(self, history):
        trials = 1000
        s = len(history)
    
        if (s == 0):
            self.Gstate = 0
            # make some dummy sizes
            Gprev_foe_move = -1
            self.Gprev_friend_move = -1
        if (s > 0):
            Gprev_foe_move = history[-1]['competitorStep']
        
        self.Gstate, Gfriend_move = self.FSM_move(s, self.Gprev_friend_move, Gprev_foe_move, self.Gstate, self.F.state_choices, self.F.action_choices, history) 
        self.Gprev_friend_move = Gfriend_move
        
        return int(Gfriend_move)
    
class anti_decision_tree(agent):
    def __init__(self):
        self.dec = decision_tree(5,25)
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL() 
        self.F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.')
        self.F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00\xa1\x00\x00\x00\xc2\x00\x00\x00w\x00\x00\x00J\x00\x00\x001\x00\x00\x00\x10\x00\x00\x00\x02\x00\x00\x00`\x00\x00\x00\x8e\x00\x00\x00\xb9\x00\x00\x00`\x00\x00\x00\xc1\x00\x00\x001\x00\x00\x00b\x00\x00\x00g\x00\x00\x00\xc4\x00\x00\x00>\x00\x00\x00%\x00\x00\x00p\x00\x00\x00\xac\x00\x00\x00\x99\x00\x00\x00\xbc\x00\x00\x00\xb9\x00\x00\x00\x1d\x00\x00\x00\xad\x00\x00\x00f\x00\x00\x00\x98\x00\x00\x00i\x00\x00\x00\x05\x00\x00\x00\x97\x00\x00\x00\x8b\x00\x00\x00l\x00\x00\x00&\x00\x00\x00\xb7\x00\x00\x00\x9a\x00\x00\x00-\x00\x00\x00\xa1\x00\x00\x00\x88\x00\x00\x00\x99\x00\x00\x00}\x00\x00\x00F\x00\x00\x00V\x00\x00\x00\x1b\x00\x00\x00\xc3\x00\x00\x003\x00\x00\x00"\x00\x00\x00\x1d\x00\x00\x00\x8d\x00\x00\x00\xab\x00\x00\x00\xa6\x00\x00\x00\xaa\x00\x00\x002\x00\x00\x00}\x00\x00\x00O\x00\x00\x00*\x00\x00\x00\x88\x00\x00\x00\xae\x00\x00\x00[\x00\x00\x00(\x00\x00\x00\xbb\x00\x00\x00\x8c\x00\x00\x00H\x00\x00\x00\xa5\x00\x00\x00b\x00\x00\x00o\x00\x00\x00\xb6\x00\x00\x00&\x00\x00\x00\x9c\x00\x00\x00\x87\x00\x00\x00K\x00\x00\x00\x99\x00\x00\x00\x01\x00\x00\x00\xb0\x00\x00\x00\x90\x00\x00\x009\x00\x00\x00\xa4\x00\x00\x00R\x00\x00\x00\xba\x00\x00\x00;\x00\x00\x00i\x00\x00\x00\xa7\x00\x00\x00\x12\x00\x00\x00\'\x00\x00\x00\x9f\x00\x00\x00T\x00\x00\x00L\x00\x00\x00\x80\x00\x00\x00O\x00\x00\x00\x99\x00\x00\x00\xa9\x00\x00\x00\xc5\x00\x00\x00\x1f\x00\x00\x00\x0f\x00\x00\x00<\x00\x00\x00\x1f\x00\x00\x00U\x00\x00\x00.\x00\x00\x00\x85\x00\x00\x00\xc5\x00\x00\x00\x06\x00\x00\x00\x1d\x00\x00\x00\x87\x00\x00\x00\xa5\x00\x00\x00\x1a\x00\x00\x00>\x00\x00\x00\xaf\x00\x00\x00\xc6\x00\x00\x00\x88\x00\x00\x00\x94\x00\x00\x00\x01\x00\x00\x00O\x00\x00\x00\x8f\x00\x00\x00i\x00\x00\x00^\x00\x00\x00\xa4\x00\x00\x00\xc5\x00\x00\x00M\x00\x00\x00\x16\x00\x00\x00Y\x00\x00\x00\xa2\x00\x00\x00N\x00\x00\x00h\x00\x00\x00X\x00\x00\x00\xa6\x00\x00\x00\xa9\x00\x00\x00\xc0\x00\x00\x00\xa9\x00\x00\x00/\x00\x00\x00]\x00\x00\x00B\x00\x00\x00\x13\x00\x00\x00\xbe\x00\x00\x00%\x00\x00\x00\x03\x00\x00\x00Q\x00\x00\x00\xb9\x00\x00\x00f\x00\x00\x00a\x00\x00\x00\x8c\x00\x00\x00\x10\x00\x00\x00E\x00\x00\x00\xb4\x00\x00\x00\xab\x00\x00\x00[\x00\x00\x00\xbb\x00\x00\x00}\x00\x00\x00\x92\x00\x00\x00y\x00\x00\x00\x13\x00\x00\x00\x89\x00\x00\x00\x1f\x00\x00\x00\x9a\x00\x00\x00\x0c\x00\x00\x00\x16\x00\x00\x00y\x00\x00\x00\x89\x00\x00\x00\'\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00{\x00\x00\x00\xa3\x00\x00\x00\x93\x00\x00\x00<\x00\x00\x00@\x00\x00\x00\xa3\x00\x00\x00\x17\x00\x00\x00y\x00\x00\x00Z\x00\x00\x00\x8b\x00\x00\x00F\x00\x00\x000\x00\x00\x00\xb2\x00\x00\x00\x96\x00\x00\x00\x8f\x00\x00\x00\x1c\x00\x00\x00\x92\x00\x00\x00\x1c\x00\x00\x00g\x00\x00\x00&\x00\x00\x00\xa8\x00\x00\x00\xb7\x00\x00\x00\x8f\x00\x00\x00.\x00\x00\x00\xa4\x00\x00\x00\x06\x00\x00\x000\x00\x00\x00K\x00\x00\x00,\x00\x00\x00)\x00\x00\x00\x92\x00\x00\x00\x8b\x00\x00\x00\x10\x00\x00\x00d\x00\x00\x00\x04\x00\x00\x00(\x00\x00\x00\xa1\x00\x00\x007\x00\x00\x00\\\x00\x00\x00 \x00\x00\x00]\x00\x00\x00\x9f\x00\x00\x00>\x00\x00\x00 \x00\x00\x009\x00\x00\x00W\x00\x00\x00\x85\x00\x00\x00\x1e\x00\x00\x00\xb5\x00\x00\x00|\x00\x00\x00\x1c\x00\x00\x00\x89\x00\x00\x007\x00\x00\x00\xbb\x00\x00\x00\t\x00\x00\x00E\x00\x00\x00o\x00\x00\x00~\x00\x00\x00\x08\x00\x00\x00\xbd\x00\x00\x00>\x00\x00\x00=\x00\x00\x00R\x00\x00\x00\x11\x00\x00\x00\x13\x00\x00\x00\xaf\x00\x00\x00\x16\x00\x00\x00 \x00\x00\x00\\\x00\x00\x00\x9c\x00\x00\x00\xbb\x00\x00\x00-\x00\x00\x00\xa9\x00\x00\x00E\x00\x00\x00\x83\x00\x00\x009\x00\x00\x00w\x00\x00\x009\x00\x00\x00[\x00\x00\x00\x03\x00\x00\x00\x97\x00\x00\x005\x00\x00\x00\xc3\x00\x00\x009\x00\x00\x00I\x00\x00\x00=\x00\x00\x00B\x00\x00\x00k\x00\x00\x00\xb7\x00\x00\x00\xa8\x00\x00\x00N\x00\x00\x00\xb2\x00\x00\x007\x00\x00\x00h\x00\x00\x00~\x00\x00\x001\x00\x00\x00\x93\x00\x00\x00\x1d\x00\x00\x00\xbd\x00\x00\x00\x8c\x00\x00\x00=\x00\x00\x00d\x00\x00\x00\x07\x00\x00\x00\x1f\x00\x00\x00\xba\x00\x00\x00\xc3\x00\x00\x00\x0b\x00\x00\x00\x95\x00\x00\x00a\x00\x00\x00\xaa\x00\x00\x00$\x00\x00\x00T\x00\x00\x00\xa8\x00\x00\x00\x1c\x00\x00\x00\xb2\x00\x00\x00.\x00\x00\x00\x0e\x00\x00\x00a\x00\x00\x00A\x00\x00\x00\xb6\x00\x00\x00\x82\x00\x00\x00\xa8\x00\x00\x00>\x00\x00\x00\xa9\x00\x00\x00V\x00\x00\x00\x85\x00\x00\x00\xb2\x00\x00\x00\xab\x00\x00\x00>\x00\x00\x00a\x00\x00\x00c\x00\x00\x006\x00\x00\x00\x9d\x00\x00\x00\x12\x00\x00\x00X\x00\x00\x00\x87\x00\x00\x00\x13\x00\x00\x007\x00\x00\x00\xc0\x00\x00\x00r\x00\x00\x00\x89\x00\x00\x00\x8d\x00\x00\x00\n\x00\x00\x00\x0f\x00\x00\x00X\x00\x00\x00f\x00\x00\x00=\x00\x00\x00@\x00\x00\x00\xbc\x00\x00\x00\x1c\x00\x00\x00\x1a\x00\x00\x00\xc5\x00\x00\x00\x0e\x00\x00\x00n\x00\x00\x00\xb6\x00\x00\x00\x7f\x00\x00\x00\n\x00\x00\x00\xae\x00\x00\x00u\x00\x00\x00\x97\x00\x00\x00\\\x00\x00\x00w\x00\x00\x00^\x00\x00\x00\x91\x00\x00\x00\x01\x00\x00\x00\x91\x00\x00\x00\x85\x00\x00\x00/\x00\x00\x00\xac\x00\x00\x00T\x00\x00\x00\x07\x00\x00\x00\x1a\x00\x00\x00b\x00\x00\x00K\x00\x00\x00E\x00\x00\x00Z\x00\x00\x00)\x00\x00\x00w\x00\x00\x00S\x00\x00\x00M\x00\x00\x00p\x00\x00\x00 \x00\x00\x00\x17\x00\x00\x00\xbf\x00\x00\x00l\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00O\x00\x00\x00b\x00\x00\x00\x0b\x00\x00\x00|\x00\x00\x000\x00\x00\x00i\x00\x00\x00:\x00\x00\x00l\x00\x00\x00\t\x00\x00\x00~\x00\x00\x00\\\x00\x00\x00\x82\x00\x00\x00u\x00\x00\x00\x14\x00\x00\x00\xaa\x00\x00\x00\x90\x00\x00\x009\x00\x00\x00\x8b\x00\x00\x006\x00\x00\x00\xab\x00\x00\x00|\x00\x00\x00?\x00\x00\x00\r\x00\x00\x00\x80\x00\x00\x009\x00\x00\x00\x1e\x00\x00\x00\'\x00\x00\x00\x01\x00\x00\x00\x8c\x00\x00\x00I\x00\x00\x00\xb1\x00\x00\x00\xc6\x00\x00\x00\xb5\x00\x00\x00+\x00\x00\x00|\x00\x00\x00\x0f\x00\x00\x00\x9c\x00\x00\x004\x00\x00\x00:\x00\x00\x00\xc4\x00\x00\x00\xc5\x00\x00\x00\x1c\x00\x00\x00D\x00\x00\x00\x84\x00\x00\x00\x91\x00\x00\x00\x9e\x00\x00\x00l\x00\x00\x00\x1d\x00\x00\x00\xbc\x00\x00\x00\n\x00\x00\x00?\x00\x00\x00\x16\x00\x00\x00\x15\x00\x00\x00\xa4\x00\x00\x00h\x00\x00\x00\xa2\x00\x00\x00I\x00\x00\x00H\x00\x00\x00\x14\x00\x00\x00J\x00\x00\x00\xbf\x00\x00\x00\xad\x00\x00\x00G\x00\x00\x003\x00\x00\x00a\x00\x00\x00O\x00\x00\x00h\x00\x00\x007\x00\x00\x00l\x00\x00\x00\x18\x00\x00\x00\x88\x00\x00\x00\x93\x00\x00\x00\'\x00\x00\x003\x00\x00\x00\xb9\x00\x00\x00v\x00\x00\x00\x90\x00\x00\x00O\x00\x00\x00^\x00\x00\x00S\x00\x00\x00;\x00\x00\x00\x14\x00\x00\x00i\x00\x00\x00\x88\x00\x00\x00r\x00\x00\x00$\x00\x00\x00\x8a\x00\x00\x00:\x00\x00\x00\x92\x00\x00\x00\xc5\x00\x00\x00\xb3\x00\x00\x00v\x00\x00\x00\r\x00\x00\x00\x8b\x00\x00\x00$\x00\x00\x00\x90\x00\x00\x00\xc1\x00\x00\x00\r\x00\x00\x00M\x00\x00\x00\xac\x00\x00\x00+\x00\x00\x00\x9b\x00\x00\x00\xa8\x00\x00\x00\x15\x00\x00\x00\x8e\x00\x00\x00b\x00\x00\x00\x8e\x00\x00\x00\x82\x00\x00\x00\x1e\x00\x00\x00\xc7\x00\x00\x00\x81\x00\x00\x005\x00\x00\x00\x0c\x00\x00\x00\xb8\x00\x00\x00\x88\x00\x00\x00"\x00\x00\x00f\x00\x00\x00\r\x00\x00\x00l\x00\x00\x00\x9d\x00\x00\x00v\x00\x00\x00\r\x00\x00\x00\x9f\x00\x00\x00?\x00\x00\x00v\x00\x00\x00$\x00\x00\x00\x99\x00\x00\x00$\x00\x00\x00\xb3\x00\x00\x00,\x00\x00\x00k\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00\xb1\x00\x00\x004\x00\x00\x00%\x00\x00\x00\x05\x00\x00\x00\x91\x00\x00\x00@\x00\x00\x00\xbc\x00\x00\x00\x1f\x00\x00\x00\t\x00\x00\x00\xc4\x00\x00\x00\x13\x00\x00\x002\x00\x00\x00\x90\x00\x00\x00P\x00\x00\x00\xa6\x00\x00\x00\x92\x00\x00\x00b\x00\x00\x00\x8b\x00\x00\x00\x8d\x00\x00\x00B\x00\x00\x00\'\x00\x00\x00"\x00\x00\x00#\x00\x00\x00e\x00\x00\x00\x00\x00\x00\x00\x99\x00\x00\x00\xa3\x00\x00\x00\xb7\x00\x00\x00\xa7\x00\x00\x00\xc5\x00\x00\x00i\x00\x00\x00w\x00\x00\x00\xb8\x00\x00\x00\x15\x00\x00\x00 \x00\x00\x00/\x00\x00\x00\xc4\x00\x00\x00w\x00\x00\x00\x17\x00\x00\x00\x9f\x00\x00\x00q\x00\x00\x00!\x00\x00\x00)\x00\x00\x00_\x00\x00\x00\n\x00\x00\x005\x00\x00\x00\x16\x00\x00\x00C\x00\x00\x00\x8b\x00\x00\x00\xa5\x00\x00\x00\x88\x00\x00\x00=\x00\x00\x00m\x00\x00\x00U\x00\x00\x00\x1d\x00\x00\x00O\x00\x00\x00\x97\x00\x00\x00\x8a\x00\x00\x00\x84\x00\x00\x00Q\x00\x00\x00 \x00\x00\x00\xb2\x00\x00\x00\xb0\x00\x00\x00>\x00\x00\x00u\x00\x00\x00\xa5\x00\x00\x00\xa4\x00\x00\x00\xbd\x00\x00\x00}\x00\x00\x00\x12\x00\x00\x00\x05\x00\x00\x00W\x00\x00\x00\x96\x00\x00\x00\x97\x00\x00\x00q\x00\x00\x00\xa0\x00\x00\x00u\x00\x00\x00=\x00\x00\x00y\x00\x00\x00T\x00\x00\x00\xb5\x00\x00\x00=\x00\x00\x00W\x00\x00\x00/\x00\x00\x00)\x00\x00\x00n\x00\x00\x00e\x00\x00\x00=\x00\x00\x00X\x00\x00\x00\xa4\x00\x00\x00\x1d\x00\x00\x00e\x00\x00\x00\x1c\x00\x00\x00?\x00\x00\x00h\x00\x00\x00\x12\x00\x00\x00P\x00\x00\x00\x9e\x00\x00\x00Q\x00\x00\x00\xa4\x00\x00\x00\xc0\x00\x00\x00o\x00\x00\x00\xa8\x00\x00\x00\x14\x00\x00\x00\xc5\x00\x00\x00I\x00\x00\x00\xa2\x00\x00\x00\x98\x00\x00\x00\xc5\x00\x00\x00\x97\x00\x00\x00\t\x00\x00\x00\x87\x00\x00\x00J\x00\x00\x00A\x00\x00\x00z\x00\x00\x00\x16\x00\x00\x00\x9e\x00\x00\x00\r\x00\x00\x00y\x00\x00\x00 \x00\x00\x00\xac\x00\x00\x00q\rtq\x0eb.')
        
    def FSM_move(self, s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A, history):
        filter_prediction = self.dec.history_step(history)
        Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
        return(Astate, friend_move)

    def reset(self):
        self.dec = decision_tree(5,25)
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL()
        
    def history_step(self, history):
        trials = 1000
        s = len(history)
    
        if (s == 0):
            self.Gstate = 0
            # make some dummy sizes
            Gprev_foe_move = -1
            self.Gprev_friend_move = -1
        if (s > 0):
            Gprev_foe_move = history[-1]['competitorStep']
        
        self.Gstate, Gfriend_move = self.FSM_move(s, self.Gprev_friend_move, Gprev_foe_move, self.Gstate, self.F.state_choices, self.F.action_choices, history) 
        self.Gprev_friend_move = Gfriend_move
        
        return int(Gfriend_move)
    
class anti_ignore(agent):
    def __init__(self):
        self.ignore = ignore_agent()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL() 
        self.F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00q\rtq\x0eb.')
        self.F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00\xaa\x00\x00\x00\xc1\x00\x00\x00\xbd\x00\x00\x00(\x00\x00\x00\x05\x00\x00\x00\\\x00\x00\x002\x00\x00\x00p\x00\x00\x00\x9e\x00\x00\x00O\x00\x00\x00\xc2\x00\x00\x00H\x00\x00\x00|\x00\x00\x00\x14\x00\x00\x00!\x00\x00\x00!\x00\x00\x00q\x00\x00\x00\xc4\x00\x00\x00H\x00\x00\x00\xba\x00\x00\x00\x0e\x00\x00\x00d\x00\x00\x00\xbd\x00\x00\x00\x90\x00\x00\x00=\x00\x00\x00\x1e\x00\x00\x00\xbd\x00\x00\x00\x0b\x00\x00\x00\x87\x00\x00\x00\xc3\x00\x00\x00\x8a\x00\x00\x00\x99\x00\x00\x00\x0b\x00\x00\x00\xa1\x00\x00\x00\xb4\x00\x00\x00\x99\x00\x00\x00\x8c\x00\x00\x00\x85\x00\x00\x00F\x00\x00\x00\x0f\x00\x00\x00\xbf\x00\x00\x00\x04\x00\x00\x000\x00\x00\x00\x9e\x00\x00\x00M\x00\x00\x00m\x00\x00\x00\x98\x00\x00\x00\x8e\x00\x00\x00\x8e\x00\x00\x00|\x00\x00\x008\x00\x00\x00%\x00\x00\x00\x8e\x00\x00\x00O\x00\x00\x00\xbd\x00\x00\x00f\x00\x00\x00f\x00\x00\x00\xa5\x00\x00\x00F\x00\x00\x00q\x00\x00\x00`\x00\x00\x00\x9d\x00\x00\x00\x19\x00\x00\x00Y\x00\x00\x00]\x00\x00\x001\x00\x00\x00`\x00\x00\x00\xc0\x00\x00\x00\x07\x00\x00\x00y\x00\x00\x00;\x00\x00\x00~\x00\x00\x00,\x00\x00\x00.\x00\x00\x00%\x00\x00\x00%\x00\x00\x00\x02\x00\x00\x00\xbd\x00\x00\x00Z\x00\x00\x00U\x00\x00\x00U\x00\x00\x00.\x00\x00\x00\xb2\x00\x00\x00~\x00\x00\x00P\x00\x00\x00\xab\x00\x00\x00G\x00\x00\x00\x1c\x00\x00\x00E\x00\x00\x00\x0e\x00\x00\x00\x1b\x00\x00\x00\x95\x00\x00\x00\x1e\x00\x00\x00&\x00\x00\x00D\x00\x00\x00$\x00\x00\x00;\x00\x00\x00-\x00\x00\x00\xbc\x00\x00\x00\x93\x00\x00\x006\x00\x00\x00]\x00\x00\x00\xb5\x00\x00\x00\x01\x00\x00\x00]\x00\x00\x00\x84\x00\x00\x008\x00\x00\x00\xa3\x00\x00\x00Y\x00\x00\x00\xc7\x00\x00\x00\x8e\x00\x00\x00)\x00\x00\x00X\x00\x00\x00\xbe\x00\x00\x00)\x00\x00\x00&\x00\x00\x00s\x00\x00\x00\x94\x00\x00\x00\x8c\x00\x00\x006\x00\x00\x00\x0f\x00\x00\x00\x9c\x00\x00\x00\x98\x00\x00\x00\x17\x00\x00\x00%\x00\x00\x00\xb3\x00\x00\x007\x00\x00\x00T\x00\x00\x00y\x00\x00\x00E\x00\x00\x00(\x00\x00\x00U\x00\x00\x008\x00\x00\x00\x9b\x00\x00\x00W\x00\x00\x00\xb5\x00\x00\x00\xc7\x00\x00\x00_\x00\x00\x00 \x00\x00\x00\xb6\x00\x00\x00f\x00\x00\x00\x10\x00\x00\x00\r\x00\x00\x00\x14\x00\x00\x00\x82\x00\x00\x00c\x00\x00\x00~\x00\x00\x00n\x00\x00\x004\x00\x00\x00*\x00\x00\x00\xb1\x00\x00\x000\x00\x00\x00\xb2\x00\x00\x00`\x00\x00\x00A\x00\x00\x00\xaf\x00\x00\x00\xae\x00\x00\x00\x8f\x00\x00\x00P\x00\x00\x00Z\x00\x00\x002\x00\x00\x00\x99\x00\x00\x00h\x00\x00\x00H\x00\x00\x00D\x00\x00\x00\x10\x00\x00\x00\x15\x00\x00\x00Z\x00\x00\x00t\x00\x00\x00w\x00\x00\x00\x91\x00\x00\x00\x1d\x00\x00\x00F\x00\x00\x00\xc7\x00\x00\x00H\x00\x00\x00\x00\x00\x00\x00Z\x00\x00\x00c\x00\x00\x00\xc1\x00\x00\x00\xb9\x00\x00\x00\xb3\x00\x00\x00Y\x00\x00\x00\xaa\x00\x00\x00G\x00\x00\x00\x9d\x00\x00\x00\x9c\x00\x00\x00X\x00\x00\x00\x00\x00\x00\x00Y\x00\x00\x00\xa3\x00\x00\x00?\x00\x00\x00\x82\x00\x00\x00#\x00\x00\x00k\x00\x00\x009\x00\x00\x00\x14\x00\x00\x00Y\x00\x00\x00\x90\x00\x00\x00L\x00\x00\x00\xbe\x00\x00\x00p\x00\x00\x00,\x00\x00\x00F\x00\x00\x00\x0e\x00\x00\x00\xa6\x00\x00\x00\x95\x00\x00\x00\xa5\x00\x00\x00\n\x00\x00\x00-\x00\x00\x005\x00\x00\x00\x18\x00\x00\x00\xb1\x00\x00\x00E\x00\x00\x00\x1a\x00\x00\x00e\x00\x00\x00\xbc\x00\x00\x00A\x00\x00\x00\x86\x00\x00\x00\xaf\x00\x00\x00c\x00\x00\x00}\x00\x00\x00\xb4\x00\x00\x00\x7f\x00\x00\x00\x9c\x00\x00\x00h\x00\x00\x00~\x00\x00\x00#\x00\x00\x00J\x00\x00\x00v\x00\x00\x00\x8f\x00\x00\x00\x8b\x00\x00\x00\xc4\x00\x00\x00\x93\x00\x00\x00C\x00\x00\x00\x9c\x00\x00\x00L\x00\x00\x009\x00\x00\x00\x8f\x00\x00\x00D\x00\x00\x00\x0e\x00\x00\x00\\\x00\x00\x009\x00\x00\x00\x04\x00\x00\x00\x0c\x00\x00\x00\x8f\x00\x00\x00\x07\x00\x00\x00#\x00\x00\x00\x9a\x00\x00\x00\x10\x00\x00\x00?\x00\x00\x00\n\x00\x00\x00\x9b\x00\x00\x00\xb5\x00\x00\x00M\x00\x00\x00\x05\x00\x00\x00(\x00\x00\x00\x14\x00\x00\x00A\x00\x00\x00\xc6\x00\x00\x00\xbd\x00\x00\x00U\x00\x00\x00\xbe\x00\x00\x00\xac\x00\x00\x00\x83\x00\x00\x00d\x00\x00\x00T\x00\x00\x00;\x00\x00\x00>\x00\x00\x00 \x00\x00\x00u\x00\x00\x00\x87\x00\x00\x00?\x00\x00\x00\x11\x00\x00\x00\xb4\x00\x00\x00\xc6\x00\x00\x00\x9f\x00\x00\x00@\x00\x00\x00\x06\x00\x00\x00L\x00\x00\x00\xa6\x00\x00\x00\x92\x00\x00\x00\x1e\x00\x00\x00\xbd\x00\x00\x00\xab\x00\x00\x00\x85\x00\x00\x00L\x00\x00\x00\x92\x00\x00\x00\x8a\x00\x00\x00_\x00\x00\x00}\x00\x00\x00#\x00\x00\x008\x00\x00\x00Q\x00\x00\x00\xae\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x9d\x00\x00\x00\x85\x00\x00\x00\x8f\x00\x00\x00\x12\x00\x00\x00p\x00\x00\x00\x02\x00\x00\x00O\x00\x00\x00\x08\x00\x00\x00\xaa\x00\x00\x00\x9d\x00\x00\x00\xa2\x00\x00\x00\xb8\x00\x00\x00*\x00\x00\x00F\x00\x00\x00\x9d\x00\x00\x00\xb1\x00\x00\x00\x8d\x00\x00\x00\x18\x00\x00\x00Y\x00\x00\x00\xb0\x00\x00\x00\n\x00\x00\x00\x0f\x00\x00\x00\x9a\x00\x00\x00\x0e\x00\x00\x00\x14\x00\x00\x00\xaf\x00\x00\x00X\x00\x00\x009\x00\x00\x00\xb5\x00\x00\x00-\x00\x00\x00\x04\x00\x00\x00\x17\x00\x00\x00e\x00\x00\x00\xb1\x00\x00\x00A\x00\x00\x00\x07\x00\x00\x00Y\x00\x00\x00\xad\x00\x00\x00Z\x00\x00\x00[\x00\x00\x008\x00\x00\x00\x97\x00\x00\x00Z\x00\x00\x00\x1b\x00\x00\x00~\x00\x00\x00v\x00\x00\x00L\x00\x00\x00X\x00\x00\x00s\x00\x00\x00\x13\x00\x00\x00\x83\x00\x00\x00f\x00\x00\x00\xc2\x00\x00\x00\x08\x00\x00\x001\x00\x00\x00\x18\x00\x00\x00\xa7\x00\x00\x00\xac\x00\x00\x00k\x00\x00\x00\x1e\x00\x00\x00\x04\x00\x00\x00\x1e\x00\x00\x00\xad\x00\x00\x00\xaf\x00\x00\x00\\\x00\x00\x00\x10\x00\x00\x00\x12\x00\x00\x00\x07\x00\x00\x00{\x00\x00\x00\x80\x00\x00\x00\xb4\x00\x00\x00\xbc\x00\x00\x00\xb8\x00\x00\x00\x8b\x00\x00\x00\x88\x00\x00\x00"\x00\x00\x00\x19\x00\x00\x00\x91\x00\x00\x00\xb5\x00\x00\x00\x9e\x00\x00\x00W\x00\x00\x00K\x00\x00\x00g\x00\x00\x00]\x00\x00\x00\xbf\x00\x00\x00\xb0\x00\x00\x00\x14\x00\x00\x00\xae\x00\x00\x00\x89\x00\x00\x00p\x00\x00\x00<\x00\x00\x00K\x00\x00\x00,\x00\x00\x00B\x00\x00\x00\xbd\x00\x00\x00\xac\x00\x00\x00\x9c\x00\x00\x00\x01\x00\x00\x00\x9f\x00\x00\x00\xae\x00\x00\x00k\x00\x00\x00U\x00\x00\x00\x12\x00\x00\x00/\x00\x00\x00z\x00\x00\x00\x96\x00\x00\x00^\x00\x00\x00\x9c\x00\x00\x001\x00\x00\x00W\x00\x00\x00N\x00\x00\x00\x7f\x00\x00\x00\x90\x00\x00\x00\xbf\x00\x00\x00\x12\x00\x00\x00\xb4\x00\x00\x00\x82\x00\x00\x00|\x00\x00\x00)\x00\x00\x00C\x00\x00\x00y\x00\x00\x00\xa1\x00\x00\x00\x04\x00\x00\x00\r\x00\x00\x00\x9f\x00\x00\x00\x8c\x00\x00\x009\x00\x00\x00u\x00\x00\x00)\x00\x00\x00h\x00\x00\x00n\x00\x00\x00L\x00\x00\x00\x95\x00\x00\x00\xb2\x00\x00\x00p\x00\x00\x00\xbb\x00\x00\x00D\x00\x00\x00\x9b\x00\x00\x00\x96\x00\x00\x00\x0b\x00\x00\x00 \x00\x00\x00\x99\x00\x00\x006\x00\x00\x00\x93\x00\x00\x00A\x00\x00\x008\x00\x00\x00\x99\x00\x00\x00\x86\x00\x00\x00\n\x00\x00\x00\xad\x00\x00\x00\xb6\x00\x00\x00\x8c\x00\x00\x00\x9f\x00\x00\x00\x01\x00\x00\x00\x08\x00\x00\x00\xb9\x00\x00\x00`\x00\x00\x00\x91\x00\x00\x00\xc6\x00\x00\x00_\x00\x00\x00\xb5\x00\x00\x00{\x00\x00\x00\x9f\x00\x00\x00\xb4\x00\x00\x00u\x00\x00\x00\xba\x00\x00\x00\xa0\x00\x00\x00\xb3\x00\x00\x00\x93\x00\x00\x00\xb6\x00\x00\x00\r\x00\x00\x00\xb6\x00\x00\x00i\x00\x00\x00\x8d\x00\x00\x00?\x00\x00\x00\xa8\x00\x00\x00\x96\x00\x00\x00\x92\x00\x00\x00b\x00\x00\x00\xc2\x00\x00\x006\x00\x00\x00 \x00\x00\x00t\x00\x00\x00j\x00\x00\x00\x11\x00\x00\x00\x1e\x00\x00\x00$\x00\x00\x00M\x00\x00\x00c\x00\x00\x00\'\x00\x00\x00\x0e\x00\x00\x00\xaa\x00\x00\x00~\x00\x00\x00f\x00\x00\x00\x9a\x00\x00\x00\x9f\x00\x00\x006\x00\x00\x00\t\x00\x00\x00\t\x00\x00\x00\x1c\x00\x00\x00\xc3\x00\x00\x00\x10\x00\x00\x00\xad\x00\x00\x00O\x00\x00\x00\t\x00\x00\x00\x1f\x00\x00\x00\x05\x00\x00\x00\x05\x00\x00\x00\x96\x00\x00\x00\x1e\x00\x00\x00\xba\x00\x00\x00\x9c\x00\x00\x00\x18\x00\x00\x00\x9c\x00\x00\x00i\x00\x00\x00i\x00\x00\x00\xc2\x00\x00\x00\x91\x00\x00\x00j\x00\x00\x00\xbc\x00\x00\x00\xa4\x00\x00\x00\x94\x00\x00\x00\x16\x00\x00\x00\x12\x00\x00\x00I\x00\x00\x00\x00\x00\x00\x00v\x00\x00\x00\x88\x00\x00\x00\x9b\x00\x00\x00\x92\x00\x00\x00S\x00\x00\x00\xaf\x00\x00\x00\x0e\x00\x00\x00s\x00\x00\x00w\x00\x00\x00\x86\x00\x00\x00c\x00\x00\x00\x98\x00\x00\x00\x9e\x00\x00\x00!\x00\x00\x00\x8d\x00\x00\x00\xa8\x00\x00\x00@\x00\x00\x00\x0c\x00\x00\x00\x08\x00\x00\x00\x90\x00\x00\x00\xbf\x00\x00\x00.\x00\x00\x00m\x00\x00\x00&\x00\x00\x00\x17\x00\x00\x00M\x00\x00\x00\x04\x00\x00\x00*\x00\x00\x00\x9c\x00\x00\x00\x0c\x00\x00\x00+\x00\x00\x000\x00\x00\x00\xb7\x00\x00\x00\x99\x00\x00\x00\xc1\x00\x00\x00\x13\x00\x00\x00i\x00\x00\x00#\x00\x00\x00G\x00\x00\x00(\x00\x00\x00H\x00\x00\x00l\x00\x00\x00Y\x00\x00\x00\x8e\x00\x00\x003\x00\x00\x00\xc7\x00\x00\x00\x10\x00\x00\x00\xb8\x00\x00\x00\xbe\x00\x00\x002\x00\x00\x00W\x00\x00\x00\x17\x00\x00\x00`\x00\x00\x00\x11\x00\x00\x00\n\x00\x00\x00j\x00\x00\x00\x10\x00\x00\x00\xab\x00\x00\x00\x11\x00\x00\x00\x86\x00\x00\x00@\x00\x00\x00\x81\x00\x00\x00\xac\x00\x00\x00\xac\x00\x00\x00\x02\x00\x00\x00\xbc\x00\x00\x00\xc2\x00\x00\x00\\\x00\x00\x00t\x00\x00\x00\x08\x00\x00\x00\x80\x00\x00\x00:\x00\x00\x00\xb7\x00\x00\x00]\x00\x00\x00y\x00\x00\x00\xc0\x00\x00\x00)\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.')
        
    def FSM_move(self, s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A, history):
        filter_prediction = self.ignore.history_step(history)
        Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
        return(Astate, friend_move)

    def reset(self):
        self.ignore = ignore_agent()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL()
        
    def history_step(self, history):
        trials = 1000
        s = len(history)
    
        if (s == 0):
            self.Gstate = 0
            # make some dummy sizes
            Gprev_foe_move = -1
            self.Gprev_friend_move = -1
        if (s > 0):
            Gprev_foe_move = history[-1]['competitorStep']
        
        self.Gstate, Gfriend_move = self.FSM_move(s, self.Gprev_friend_move, Gprev_foe_move, self.Gstate, self.F.state_choices, self.F.action_choices, history) 
        self.Gprev_friend_move = Gfriend_move
        
        return int(Gfriend_move)
    
class anti_iou(agent):
    def __init__(self):
        self.iou = IOU2_agent(27)
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL() 
        self.F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00q\rtq\x0eb.')
        self.F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00\xbc\x00\x00\x00e\x00\x00\x00;\x00\x00\x00\x98\x00\x00\x00\xbb\x00\x00\x00\x88\x00\x00\x00\x07\x00\x00\x007\x00\x00\x00\x94\x00\x00\x00K\x00\x00\x00\x8b\x00\x00\x00\x0e\x00\x00\x00\xae\x00\x00\x00\xac\x00\x00\x00\x96\x00\x00\x00\xbb\x00\x00\x00\xb2\x00\x00\x00\x92\x00\x00\x00\x11\x00\x00\x00o\x00\x00\x00\x01\x00\x00\x00q\x00\x00\x00\x8c\x00\x00\x00,\x00\x00\x00\xa1\x00\x00\x00\x12\x00\x00\x00\x94\x00\x00\x00\x9a\x00\x00\x00K\x00\x00\x00\x1f\x00\x00\x00o\x00\x00\x00e\x00\x00\x00^\x00\x00\x00f\x00\x00\x00)\x00\x00\x00\xb2\x00\x00\x00o\x00\x00\x00\x08\x00\x00\x00=\x00\x00\x003\x00\x00\x006\x00\x00\x00\xbe\x00\x00\x00(\x00\x00\x00j\x00\x00\x00^\x00\x00\x00}\x00\x00\x00[\x00\x00\x00\xc5\x00\x00\x00I\x00\x00\x00\x9a\x00\x00\x00\x11\x00\x00\x00%\x00\x00\x00\x06\x00\x00\x001\x00\x00\x00\xa9\x00\x00\x00\x9e\x00\x00\x00\x19\x00\x00\x00p\x00\x00\x00\xb4\x00\x00\x00\x93\x00\x00\x00\x19\x00\x00\x00e\x00\x00\x00\xc1\x00\x00\x00\x80\x00\x00\x00`\x00\x00\x00\x13\x00\x00\x00A\x00\x00\x00J\x00\x00\x00k\x00\x00\x00\x06\x00\x00\x00\x14\x00\x00\x00\x06\x00\x00\x00s\x00\x00\x00#\x00\x00\x00*\x00\x00\x00\x03\x00\x00\x00\x9f\x00\x00\x00-\x00\x00\x00\x1e\x00\x00\x00\x14\x00\x00\x006\x00\x00\x00\t\x00\x00\x005\x00\x00\x00\x17\x00\x00\x00\x97\x00\x00\x00\x7f\x00\x00\x00}\x00\x00\x00\xc7\x00\x00\x00\xb8\x00\x00\x007\x00\x00\x003\x00\x00\x00`\x00\x00\x00+\x00\x00\x00\x9a\x00\x00\x00q\x00\x00\x00\x9d\x00\x00\x00\x14\x00\x00\x00s\x00\x00\x00\x93\x00\x00\x00\x93\x00\x00\x003\x00\x00\x00\x9b\x00\x00\x00O\x00\x00\x00\x01\x00\x00\x00M\x00\x00\x00p\x00\x00\x00\xb1\x00\x00\x00&\x00\x00\x00\x82\x00\x00\x00\xab\x00\x00\x00O\x00\x00\x00\x8d\x00\x00\x00n\x00\x00\x00\xc0\x00\x00\x00\xb1\x00\x00\x00 \x00\x00\x00\x98\x00\x00\x00\x97\x00\x00\x00\xae\x00\x00\x00-\x00\x00\x00\x17\x00\x00\x00 \x00\x00\x008\x00\x00\x00 \x00\x00\x00;\x00\x00\x00\t\x00\x00\x00\x94\x00\x00\x00e\x00\x00\x00\x15\x00\x00\x00\xb0\x00\x00\x00\x91\x00\x00\x00\x9f\x00\x00\x00\x81\x00\x00\x008\x00\x00\x00\xb8\x00\x00\x00M\x00\x00\x00~\x00\x00\x00\x0c\x00\x00\x00\x87\x00\x00\x00\xb4\x00\x00\x00\x15\x00\x00\x00*\x00\x00\x00G\x00\x00\x00\x17\x00\x00\x00(\x00\x00\x00\x97\x00\x00\x00*\x00\x00\x00]\x00\x00\x00/\x00\x00\x00J\x00\x00\x00\xc4\x00\x00\x00\x9e\x00\x00\x00\x8d\x00\x00\x00\xb5\x00\x00\x00\'\x00\x00\x00?\x00\x00\x00\x05\x00\x00\x00 \x00\x00\x00)\x00\x00\x00\x98\x00\x00\x00\xc4\x00\x00\x00w\x00\x00\x00\x95\x00\x00\x00\xc6\x00\x00\x00E\x00\x00\x00\xc6\x00\x00\x00\x0e\x00\x00\x00]\x00\x00\x00k\x00\x00\x00\xa7\x00\x00\x00=\x00\x00\x00\x00\x00\x00\x00:\x00\x00\x00\x0b\x00\x00\x00.\x00\x00\x00\xc3\x00\x00\x00\xb0\x00\x00\x00F\x00\x00\x00\x7f\x00\x00\x008\x00\x00\x00Y\x00\x00\x00\x02\x00\x00\x00O\x00\x00\x00S\x00\x00\x00g\x00\x00\x00E\x00\x00\x00S\x00\x00\x00\x08\x00\x00\x007\x00\x00\x00}\x00\x00\x00L\x00\x00\x00\xc2\x00\x00\x00\x9f\x00\x00\x00\xad\x00\x00\x00\x97\x00\x00\x00\x00\x00\x00\x00\x9d\x00\x00\x00=\x00\x00\x00\xc1\x00\x00\x00P\x00\x00\x00T\x00\x00\x00O\x00\x00\x00)\x00\x00\x00\x85\x00\x00\x00G\x00\x00\x00\xad\x00\x00\x00v\x00\x00\x00n\x00\x00\x00\xbf\x00\x00\x000\x00\x00\x00\xb0\x00\x00\x00C\x00\x00\x00\x84\x00\x00\x00s\x00\x00\x00\xa5\x00\x00\x00\x98\x00\x00\x00\x0c\x00\x00\x00y\x00\x00\x00v\x00\x00\x00\r\x00\x00\x00\x8d\x00\x00\x00l\x00\x00\x00\xc6\x00\x00\x00\x08\x00\x00\x00\xc4\x00\x00\x00\x8e\x00\x00\x00\x95\x00\x00\x00\xa3\x00\x00\x00\x12\x00\x00\x00g\x00\x00\x00\x8d\x00\x00\x00\x10\x00\x00\x00\x1d\x00\x00\x00\x04\x00\x00\x00\xb3\x00\x00\x00\x8d\x00\x00\x00\x7f\x00\x00\x00t\x00\x00\x00\x94\x00\x00\x00x\x00\x00\x00\x00\x00\x00\x00y\x00\x00\x00\x11\x00\x00\x00/\x00\x00\x00\x1f\x00\x00\x007\x00\x00\x00\xaa\x00\x00\x00\x08\x00\x00\x00?\x00\x00\x00\xc6\x00\x00\x00L\x00\x00\x00\xb7\x00\x00\x00\xc4\x00\x00\x00\xa1\x00\x00\x00C\x00\x00\x00P\x00\x00\x00\x1c\x00\x00\x00\x8c\x00\x00\x001\x00\x00\x00$\x00\x00\x00\t\x00\x00\x00\x92\x00\x00\x00\x06\x00\x00\x00\xad\x00\x00\x00I\x00\x00\x00\x9c\x00\x00\x00l\x00\x00\x00\x97\x00\x00\x00v\x00\x00\x00R\x00\x00\x009\x00\x00\x00t\x00\x00\x00\x1d\x00\x00\x00\x8a\x00\x00\x00\x9f\x00\x00\x00\x0c\x00\x00\x00\x96\x00\x00\x00\r\x00\x00\x004\x00\x00\x00$\x00\x00\x00d\x00\x00\x00\xa4\x00\x00\x00\x93\x00\x00\x00`\x00\x00\x00\xa4\x00\x00\x00a\x00\x00\x00u\x00\x00\x00I\x00\x00\x00U\x00\x00\x00v\x00\x00\x00h\x00\x00\x00&\x00\x00\x00\x8f\x00\x00\x00b\x00\x00\x00\x14\x00\x00\x00\x08\x00\x00\x00\xa1\x00\x00\x00\xaa\x00\x00\x00\xbe\x00\x00\x00t\x00\x00\x00\x85\x00\x00\x00\x18\x00\x00\x00\x9c\x00\x00\x00\x07\x00\x00\x005\x00\x00\x00\x96\x00\x00\x00F\x00\x00\x00\xc5\x00\x00\x00\x0e\x00\x00\x00q\x00\x00\x00~\x00\x00\x00%\x00\x00\x00\x04\x00\x00\x00\x13\x00\x00\x00\x14\x00\x00\x00y\x00\x00\x00}\x00\x00\x003\x00\x00\x003\x00\x00\x00f\x00\x00\x00\x14\x00\x00\x00\x00\x00\x00\x00\x0c\x00\x00\x00[\x00\x00\x00\x94\x00\x00\x00Y\x00\x00\x00A\x00\x00\x00\'\x00\x00\x00z\x00\x00\x00c\x00\x00\x00f\x00\x00\x00\xa2\x00\x00\x00\xba\x00\x00\x00\x0c\x00\x00\x00\x0b\x00\x00\x00{\x00\x00\x00r\x00\x00\x003\x00\x00\x00\x1e\x00\x00\x007\x00\x00\x00\x7f\x00\x00\x00_\x00\x00\x00.\x00\x00\x00\x0c\x00\x00\x00}\x00\x00\x00-\x00\x00\x001\x00\x00\x00k\x00\x00\x00\xc4\x00\x00\x00\x1e\x00\x00\x00\xc4\x00\x00\x00\xab\x00\x00\x00\x96\x00\x00\x00\x1e\x00\x00\x00\xa3\x00\x00\x00\xc6\x00\x00\x00\x03\x00\x00\x00\x91\x00\x00\x00\x87\x00\x00\x00\x1c\x00\x00\x00\xb8\x00\x00\x00\x86\x00\x00\x00k\x00\x00\x00\x1a\x00\x00\x00\x83\x00\x00\x00H\x00\x00\x00\xaf\x00\x00\x00&\x00\x00\x00\x8d\x00\x00\x007\x00\x00\x00\x8e\x00\x00\x00\x83\x00\x00\x00\x0c\x00\x00\x00f\x00\x00\x00\x9f\x00\x00\x00\xa6\x00\x00\x00\x9c\x00\x00\x00z\x00\x00\x008\x00\x00\x00y\x00\x00\x00\x1b\x00\x00\x00|\x00\x00\x00<\x00\x00\x00H\x00\x00\x00R\x00\x00\x00\xbd\x00\x00\x00R\x00\x00\x00_\x00\x00\x00\x80\x00\x00\x00\x9f\x00\x00\x00y\x00\x00\x00\x8d\x00\x00\x00d\x00\x00\x00!\x00\x00\x00z\x00\x00\x00]\x00\x00\x00[\x00\x00\x00\xa7\x00\x00\x00s\x00\x00\x00\xb6\x00\x00\x00L\x00\x00\x00u\x00\x00\x00u\x00\x00\x00\x97\x00\x00\x00\xa9\x00\x00\x00\xa4\x00\x00\x00\x84\x00\x00\x006\x00\x00\x00#\x00\x00\x00\xc6\x00\x00\x00|\x00\x00\x00\x89\x00\x00\x00\xc4\x00\x00\x00\x8a\x00\x00\x00\x9f\x00\x00\x00<\x00\x00\x00\x11\x00\x00\x00&\x00\x00\x00\t\x00\x00\x00\xb3\x00\x00\x00z\x00\x00\x00[\x00\x00\x00\x08\x00\x00\x00\x08\x00\x00\x00`\x00\x00\x00\'\x00\x00\x00I\x00\x00\x00}\x00\x00\x00\xab\x00\x00\x00P\x00\x00\x00b\x00\x00\x00o\x00\x00\x00\xb3\x00\x00\x00\xc5\x00\x00\x00\xc1\x00\x00\x00\x0c\x00\x00\x00\x82\x00\x00\x00y\x00\x00\x00P\x00\x00\x00\x01\x00\x00\x00b\x00\x00\x00\x07\x00\x00\x00\xa6\x00\x00\x00A\x00\x00\x00\xb5\x00\x00\x00\xb1\x00\x00\x00V\x00\x00\x00f\x00\x00\x00h\x00\x00\x00a\x00\x00\x00)\x00\x00\x001\x00\x00\x00b\x00\x00\x00\xc4\x00\x00\x00\x98\x00\x00\x00\xad\x00\x00\x00\x19\x00\x00\x00N\x00\x00\x00\xae\x00\x00\x00w\x00\x00\x00r\x00\x00\x00\xa3\x00\x00\x00>\x00\x00\x00\x1b\x00\x00\x00\x8d\x00\x00\x00!\x00\x00\x00A\x00\x00\x00\xc5\x00\x00\x00^\x00\x00\x00o\x00\x00\x00a\x00\x00\x00h\x00\x00\x00\xc2\x00\x00\x00|\x00\x00\x00\x86\x00\x00\x00.\x00\x00\x00\xaf\x00\x00\x00\x99\x00\x00\x00Q\x00\x00\x00%\x00\x00\x00\x01\x00\x00\x00\xb5\x00\x00\x00\x80\x00\x00\x00o\x00\x00\x00j\x00\x00\x00\xba\x00\x00\x00|\x00\x00\x00i\x00\x00\x00\x1a\x00\x00\x003\x00\x00\x008\x00\x00\x00\x95\x00\x00\x00s\x00\x00\x00\x15\x00\x00\x00\x10\x00\x00\x00I\x00\x00\x00;\x00\x00\x00|\x00\x00\x00\x88\x00\x00\x00%\x00\x00\x002\x00\x00\x006\x00\x00\x00\x9d\x00\x00\x00\x8c\x00\x00\x00\x9d\x00\x00\x00\xc1\x00\x00\x00\xbf\x00\x00\x00\xc6\x00\x00\x00m\x00\x00\x00\x94\x00\x00\x00r\x00\x00\x00\xc1\x00\x00\x00(\x00\x00\x00\x99\x00\x00\x00\x15\x00\x00\x00d\x00\x00\x00\x9b\x00\x00\x00A\x00\x00\x00\xb1\x00\x00\x00\xc2\x00\x00\x00\x0e\x00\x00\x00\xc0\x00\x00\x00\x99\x00\x00\x00\x02\x00\x00\x00A\x00\x00\x00\x96\x00\x00\x00/\x00\x00\x00\x7f\x00\x00\x00\x9a\x00\x00\x00\xaf\x00\x00\x00L\x00\x00\x00\x97\x00\x00\x00\xa7\x00\x00\x00I\x00\x00\x003\x00\x00\x00\x98\x00\x00\x00\xc0\x00\x00\x00;\x00\x00\x00\x98\x00\x00\x00\x7f\x00\x00\x00\x15\x00\x00\x00\xaa\x00\x00\x00s\x00\x00\x00\xa9\x00\x00\x00"\x00\x00\x00\xb8\x00\x00\x00\x9a\x00\x00\x00\x18\x00\x00\x00\xa7\x00\x00\x00\x17\x00\x00\x00\x0c\x00\x00\x00\xb9\x00\x00\x00\t\x00\x00\x00\xb6\x00\x00\x00/\x00\x00\x00\x10\x00\x00\x00#\x00\x00\x00"\x00\x00\x00j\x00\x00\x00k\x00\x00\x00t\x00\x00\x00\x03\x00\x00\x00p\x00\x00\x00U\x00\x00\x00F\x00\x00\x00\xa1\x00\x00\x00m\x00\x00\x00\xbc\x00\x00\x001\x00\x00\x00\xac\x00\x00\x00\xb8\x00\x00\x00X\x00\x00\x00a\x00\x00\x00\r\x00\x00\x00N\x00\x00\x00!\x00\x00\x00c\x00\x00\x00\x1a\x00\x00\x00i\x00\x00\x00\x1a\x00\x00\x00G\x00\x00\x00\r\x00\x00\x00N\x00\x00\x00\xb5\x00\x00\x00\x0c\x00\x00\x00.\x00\x00\x00\xa8\x00\x00\x00H\x00\x00\x00V\x00\x00\x00\x87\x00\x00\x00\x94\x00\x00\x00\xc7\x00\x00\x00\x10\x00\x00\x00\xaa\x00\x00\x00^\x00\x00\x00\x80\x00\x00\x00V\x00\x00\x00\xb1\x00\x00\x00g\x00\x00\x00d\x00\x00\x00q\rtq\x0eb.')
        
    def FSM_move(self, s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A, history):
        filter_prediction = self.iou.history_step(history)
        Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
        return(Astate, friend_move)

    def reset(self):
        self.iou = IOU2_agent(27)
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL()
        
    def history_step(self, history):
        trials = 1000
        s = len(history)
    
        if (s == 0):
            self.Gstate = 0
            # make some dummy sizes
            Gprev_foe_move = -1
            self.Gprev_friend_move = -1
        if (s > 0):
            Gprev_foe_move = history[-1]['competitorStep']
        
        self.Gstate, Gfriend_move = self.FSM_move(s, self.Gprev_friend_move, Gprev_foe_move, self.Gstate, self.F.state_choices, self.F.action_choices, history) 
        self.Gprev_friend_move = Gfriend_move
        
        return int(Gfriend_move)
    
class anti_numpy(agent):
    def __init__(self):
        self.nmp = NumpyPatterns()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL() 
        self.F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.')
        self.F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00\x93\x00\x00\x00\'\x00\x00\x00J\x00\x00\x00-\x00\x00\x00e\x00\x00\x00S\x00\x00\x00\xad\x00\x00\x00\xbb\x00\x00\x00\x9b\x00\x00\x00\xb5\x00\x00\x00\xa2\x00\x00\x00}\x00\x00\x00\xb0\x00\x00\x00\x10\x00\x00\x00)\x00\x00\x00x\x00\x00\x00\x05\x00\x00\x00\x81\x00\x00\x00\xb2\x00\x00\x00\x8e\x00\x00\x00\x9f\x00\x00\x00\xa1\x00\x00\x00r\x00\x00\x00\xc5\x00\x00\x00-\x00\x00\x00\x0f\x00\x00\x00\x17\x00\x00\x00\xb7\x00\x00\x00)\x00\x00\x00\x9e\x00\x00\x00<\x00\x00\x00\xa2\x00\x00\x00\t\x00\x00\x00D\x00\x00\x00\x07\x00\x00\x00\t\x00\x00\x00\\\x00\x00\x00\x84\x00\x00\x00X\x00\x00\x00\x13\x00\x00\x00t\x00\x00\x00-\x00\x00\x00\x89\x00\x00\x00\'\x00\x00\x00\xbc\x00\x00\x00}\x00\x00\x00,\x00\x00\x00#\x00\x00\x001\x00\x00\x00v\x00\x00\x00{\x00\x00\x00\\\x00\x00\x00v\x00\x00\x00M\x00\x00\x00\x14\x00\x00\x00\x86\x00\x00\x00\xaa\x00\x00\x00\x1a\x00\x00\x00=\x00\x00\x00S\x00\x00\x009\x00\x00\x00>\x00\x00\x00\x8a\x00\x00\x00\x0c\x00\x00\x00u\x00\x00\x00A\x00\x00\x002\x00\x00\x00/\x00\x00\x00\x9c\x00\x00\x00\xa6\x00\x00\x00M\x00\x00\x00>\x00\x00\x00\x91\x00\x00\x00j\x00\x00\x00\x9a\x00\x00\x00e\x00\x00\x00\x14\x00\x00\x00\xbb\x00\x00\x00\xa0\x00\x00\x00M\x00\x00\x00\xa2\x00\x00\x00K\x00\x00\x00\xa3\x00\x00\x003\x00\x00\x00\xbe\x00\x00\x00<\x00\x00\x00#\x00\x00\x00f\x00\x00\x00\x04\x00\x00\x00N\x00\x00\x00@\x00\x00\x00\xc3\x00\x00\x00\x82\x00\x00\x00]\x00\x00\x00\x19\x00\x00\x00B\x00\x00\x00\xb7\x00\x00\x00\xb3\x00\x00\x00u\x00\x00\x00\xc4\x00\x00\x00s\x00\x00\x00\n\x00\x00\x00<\x00\x00\x00k\x00\x00\x00h\x00\x00\x00Q\x00\x00\x00^\x00\x00\x00\xa5\x00\x00\x00*\x00\x00\x00]\x00\x00\x00\x9b\x00\x00\x00"\x00\x00\x00\x16\x00\x00\x00O\x00\x00\x003\x00\x00\x00\x83\x00\x00\x00,\x00\x00\x00N\x00\x00\x00\xbc\x00\x00\x00R\x00\x00\x00\xa2\x00\x00\x00\xbb\x00\x00\x00\'\x00\x00\x00W\x00\x00\x00\x84\x00\x00\x00\xbb\x00\x00\x00\x10\x00\x00\x00=\x00\x00\x00o\x00\x00\x00)\x00\x00\x00*\x00\x00\x00\xad\x00\x00\x00\x86\x00\x00\x00\xab\x00\x00\x00O\x00\x00\x00C\x00\x00\x00k\x00\x00\x00\x8d\x00\x00\x00m\x00\x00\x007\x00\x00\x00Y\x00\x00\x00m\x00\x00\x00\xb7\x00\x00\x00\xab\x00\x00\x00\x83\x00\x00\x00k\x00\x00\x002\x00\x00\x00\x86\x00\x00\x00\x06\x00\x00\x00X\x00\x00\x00\xad\x00\x00\x00\xbf\x00\x00\x00q\x00\x00\x00\x16\x00\x00\x00\x9f\x00\x00\x00\x13\x00\x00\x00L\x00\x00\x00X\x00\x00\x00\x89\x00\x00\x00J\x00\x00\x00\xae\x00\x00\x00\x86\x00\x00\x00\xa6\x00\x00\x00\xc3\x00\x00\x00\x13\x00\x00\x00l\x00\x00\x00\x99\x00\x00\x00\x84\x00\x00\x00\x9a\x00\x00\x00\x16\x00\x00\x00\x14\x00\x00\x00i\x00\x00\x00\x14\x00\x00\x00\xa9\x00\x00\x00}\x00\x00\x00d\x00\x00\x00\x03\x00\x00\x005\x00\x00\x00*\x00\x00\x00\x9b\x00\x00\x00\x9e\x00\x00\x00\xc0\x00\x00\x00\xc6\x00\x00\x00W\x00\x00\x00:\x00\x00\x00y\x00\x00\x006\x00\x00\x00\xa3\x00\x00\x000\x00\x00\x00\xb2\x00\x00\x00g\x00\x00\x00\x90\x00\x00\x00\xbb\x00\x00\x00,\x00\x00\x00\x01\x00\x00\x00\\\x00\x00\x00\xa6\x00\x00\x00\x94\x00\x00\x00O\x00\x00\x00\xa6\x00\x00\x00*\x00\x00\x00\x0f\x00\x00\x002\x00\x00\x00U\x00\x00\x00\t\x00\x00\x00\x91\x00\x00\x00\xa7\x00\x00\x00N\x00\x00\x00\x0e\x00\x00\x00\xb4\x00\x00\x00\x97\x00\x00\x00\xb0\x00\x00\x00\xa8\x00\x00\x00\x8b\x00\x00\x00(\x00\x00\x00\x97\x00\x00\x006\x00\x00\x00\xb3\x00\x00\x00]\x00\x00\x00\t\x00\x00\x001\x00\x00\x00H\x00\x00\x00\x9c\x00\x00\x00\xab\x00\x00\x00j\x00\x00\x00$\x00\x00\x00l\x00\x00\x00\x81\x00\x00\x00Y\x00\x00\x00\xc6\x00\x00\x00\xa6\x00\x00\x00k\x00\x00\x00\x9e\x00\x00\x00S\x00\x00\x00X\x00\x00\x00L\x00\x00\x00\xb4\x00\x00\x00\x0f\x00\x00\x00\xb4\x00\x00\x00l\x00\x00\x00\xc4\x00\x00\x00\'\x00\x00\x00\r\x00\x00\x00H\x00\x00\x00_\x00\x00\x00B\x00\x00\x00\xab\x00\x00\x00\xc2\x00\x00\x00.\x00\x00\x00\xc0\x00\x00\x00[\x00\x00\x00\xb9\x00\x00\x00]\x00\x00\x00\x1a\x00\x00\x00d\x00\x00\x00\xb0\x00\x00\x00`\x00\x00\x00j\x00\x00\x00r\x00\x00\x00\xa2\x00\x00\x00\x95\x00\x00\x00(\x00\x00\x00Y\x00\x00\x00-\x00\x00\x00\x17\x00\x00\x00\x1b\x00\x00\x007\x00\x00\x00\xc5\x00\x00\x00O\x00\x00\x00L\x00\x00\x00w\x00\x00\x00\x8d\x00\x00\x00\xbd\x00\x00\x00|\x00\x00\x00;\x00\x00\x00v\x00\x00\x00h\x00\x00\x00\x08\x00\x00\x00\x05\x00\x00\x00c\x00\x00\x00\x12\x00\x00\x00\xab\x00\x00\x00.\x00\x00\x00\xa3\x00\x00\x00\x88\x00\x00\x00\x94\x00\x00\x00k\x00\x00\x00\x9c\x00\x00\x00:\x00\x00\x00\x95\x00\x00\x00\x82\x00\x00\x00_\x00\x00\x00\xab\x00\x00\x00\x8d\x00\x00\x00\x9c\x00\x00\x00n\x00\x00\x00\xb7\x00\x00\x00@\x00\x00\x00\xbd\x00\x00\x00\x11\x00\x00\x00r\x00\x00\x00^\x00\x00\x00g\x00\x00\x00\xab\x00\x00\x00\xc1\x00\x00\x00^\x00\x00\x00@\x00\x00\x00\xa7\x00\x00\x00\x10\x00\x00\x005\x00\x00\x00\xbb\x00\x00\x00\x83\x00\x00\x00\xc4\x00\x00\x00\x04\x00\x00\x00z\x00\x00\x001\x00\x00\x00U\x00\x00\x00\x91\x00\x00\x00\x86\x00\x00\x00\xac\x00\x00\x00\xa7\x00\x00\x00]\x00\x00\x00z\x00\x00\x00\xa1\x00\x00\x00A\x00\x00\x00W\x00\x00\x00)\x00\x00\x00\x98\x00\x00\x005\x00\x00\x00\xbf\x00\x00\x00N\x00\x00\x00\x07\x00\x00\x00a\x00\x00\x00F\x00\x00\x00"\x00\x00\x00)\x00\x00\x00\x89\x00\x00\x00\xc5\x00\x00\x00\x91\x00\x00\x00\x91\x00\x00\x00)\x00\x00\x00O\x00\x00\x004\x00\x00\x00z\x00\x00\x00\xa4\x00\x00\x00v\x00\x00\x00\x1a\x00\x00\x00@\x00\x00\x00\x08\x00\x00\x00^\x00\x00\x00?\x00\x00\x00%\x00\x00\x00\x07\x00\x00\x00\x06\x00\x00\x00V\x00\x00\x00\xb4\x00\x00\x00\x94\x00\x00\x00z\x00\x00\x00\x80\x00\x00\x00)\x00\x00\x00;\x00\x00\x008\x00\x00\x009\x00\x00\x00\xad\x00\x00\x00\x01\x00\x00\x00\xb7\x00\x00\x00l\x00\x00\x00\x0c\x00\x00\x00m\x00\x00\x00P\x00\x00\x00Q\x00\x00\x00\x9b\x00\x00\x00\x19\x00\x00\x00\xa2\x00\x00\x00\xc4\x00\x00\x00\x04\x00\x00\x00\x08\x00\x00\x00\xba\x00\x00\x00E\x00\x00\x00\x92\x00\x00\x00\x91\x00\x00\x000\x00\x00\x00\x1b\x00\x00\x00O\x00\x00\x00%\x00\x00\x00T\x00\x00\x00\x01\x00\x00\x00\x85\x00\x00\x00g\x00\x00\x00\xc2\x00\x00\x00?\x00\x00\x00U\x00\x00\x00\x05\x00\x00\x00\xb5\x00\x00\x00 \x00\x00\x00\xc4\x00\x00\x00 \x00\x00\x00\xad\x00\x00\x00A\x00\x00\x00D\x00\x00\x00L\x00\x00\x00#\x00\x00\x00=\x00\x00\x00\x80\x00\x00\x00\xc2\x00\x00\x00\x8a\x00\x00\x00\x91\x00\x00\x00\xa4\x00\x00\x00\x00\x00\x00\x00\x1f\x00\x00\x00\r\x00\x00\x00E\x00\x00\x00{\x00\x00\x00\x05\x00\x00\x00\xb7\x00\x00\x00\x07\x00\x00\x00J\x00\x00\x009\x00\x00\x00O\x00\x00\x00\x01\x00\x00\x00\x7f\x00\x00\x00;\x00\x00\x00\x0e\x00\x00\x00\xc3\x00\x00\x00P\x00\x00\x00\xc1\x00\x00\x00\x86\x00\x00\x00\x90\x00\x00\x00\xa6\x00\x00\x00e\x00\x00\x00\xa9\x00\x00\x00\x90\x00\x00\x00\x7f\x00\x00\x00g\x00\x00\x00x\x00\x00\x00c\x00\x00\x00o\x00\x00\x00y\x00\x00\x00Z\x00\x00\x00!\x00\x00\x00=\x00\x00\x00l\x00\x00\x00\x9e\x00\x00\x00G\x00\x00\x00J\x00\x00\x00*\x00\x00\x00B\x00\x00\x00\x9e\x00\x00\x00\\\x00\x00\x00\x1a\x00\x00\x00\x97\x00\x00\x001\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00=\x00\x00\x00\x9b\x00\x00\x00C\x00\x00\x00\xa4\x00\x00\x00h\x00\x00\x00o\x00\x00\x00a\x00\x00\x00X\x00\x00\x00\xa7\x00\x00\x00\xb4\x00\x00\x00\x0e\x00\x00\x00Y\x00\x00\x002\x00\x00\x00\x9e\x00\x00\x007\x00\x00\x00\xb9\x00\x00\x00>\x00\x00\x00f\x00\x00\x00#\x00\x00\x009\x00\x00\x00A\x00\x00\x00\xb4\x00\x00\x00\x1a\x00\x00\x00z\x00\x00\x00`\x00\x00\x00?\x00\x00\x00R\x00\x00\x00\x0b\x00\x00\x00!\x00\x00\x00|\x00\x00\x00I\x00\x00\x00\\\x00\x00\x00\xb5\x00\x00\x00B\x00\x00\x00\t\x00\x00\x00M\x00\x00\x00\x03\x00\x00\x00\xb3\x00\x00\x007\x00\x00\x00V\x00\x00\x007\x00\x00\x00Y\x00\x00\x00_\x00\x00\x004\x00\x00\x002\x00\x00\x00$\x00\x00\x00\x96\x00\x00\x00\x98\x00\x00\x00H\x00\x00\x00\x87\x00\x00\x00h\x00\x00\x00\xc4\x00\x00\x00w\x00\x00\x00\xbe\x00\x00\x00[\x00\x00\x00\xbb\x00\x00\x00\xa1\x00\x00\x00\xc0\x00\x00\x00:\x00\x00\x00\x0b\x00\x00\x00h\x00\x00\x00u\x00\x00\x00\x12\x00\x00\x00U\x00\x00\x00\x18\x00\x00\x00\xb7\x00\x00\x00D\x00\x00\x00\x89\x00\x00\x00!\x00\x00\x00\xb5\x00\x00\x00\x01\x00\x00\x00\xa9\x00\x00\x00\x15\x00\x00\x00\xb0\x00\x00\x00\x92\x00\x00\x00\x90\x00\x00\x00\xc7\x00\x00\x00]\x00\x00\x00/\x00\x00\x00F\x00\x00\x00\x11\x00\x00\x002\x00\x00\x00s\x00\x00\x00y\x00\x00\x00q\x00\x00\x00o\x00\x00\x00\xa0\x00\x00\x00\xb0\x00\x00\x00\xa7\x00\x00\x00G\x00\x00\x00\x08\x00\x00\x00\xad\x00\x00\x00d\x00\x00\x00@\x00\x00\x00\x89\x00\x00\x00\xa3\x00\x00\x00\xc2\x00\x00\x00M\x00\x00\x00\x9c\x00\x00\x00E\x00\x00\x00\xa2\x00\x00\x00\x8a\x00\x00\x00(\x00\x00\x00$\x00\x00\x00\x19\x00\x00\x00$\x00\x00\x00)\x00\x00\x00\x02\x00\x00\x00@\x00\x00\x00\x81\x00\x00\x00\xa1\x00\x00\x00\t\x00\x00\x00\x9c\x00\x00\x00\x14\x00\x00\x00\xc4\x00\x00\x00r\x00\x00\x00\x1f\x00\x00\x00]\x00\x00\x00\x90\x00\x00\x00\x9f\x00\x00\x00\xad\x00\x00\x00\x8d\x00\x00\x00t\x00\x00\x001\x00\x00\x00z\x00\x00\x00\x92\x00\x00\x00F\x00\x00\x00\x95\x00\x00\x00>\x00\x00\x00\x94\x00\x00\x00<\x00\x00\x00\n\x00\x00\x00\x82\x00\x00\x00\xb2\x00\x00\x00\xc5\x00\x00\x00\xa0\x00\x00\x00\xab\x00\x00\x006\x00\x00\x00\x91\x00\x00\x00|\x00\x00\x00\xb6\x00\x00\x00e\x00\x00\x00>\x00\x00\x00p\x00\x00\x00\x86\x00\x00\x00~\x00\x00\x00q\rtq\x0eb.')
        
    def FSM_move(self, s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A, history):
        filter_prediction = self.nmp.history_step(history)
        Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
        return(Astate, friend_move)

    def reset(self):
        self.nmp = NumpyPatterns()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL()
        
    def history_step(self, history):
        trials = 1000
        s = len(history)
    
        if (s == 0):
            self.Gstate = 0
            # make some dummy sizes
            Gprev_foe_move = -1
            self.Gprev_friend_move = -1
        if (s > 0):
            Gprev_foe_move = history[-1]['competitorStep']
        
        self.Gstate, Gfriend_move = self.FSM_move(s, self.Gprev_friend_move, Gprev_foe_move, self.Gstate, self.F.state_choices, self.F.action_choices, history) 
        self.Gprev_friend_move = Gfriend_move
        
        return int(Gfriend_move)
    
class anti_iocane(agent):
    def __init__(self):
        self.io = iocane_agent()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL() 
        self.F.action_choices = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B \x03\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x01\x00\x00\x00\x02\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x03\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00q\rtq\x0eb.')
        self.F.state_choices  = pickle.loads(b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\xc8K\x03\x86q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00i4q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89B`\t\x00\x00c\x00\x00\x00\xac\x00\x00\x00\x81\x00\x00\x00\x1d\x00\x00\x00\xc2\x00\x00\x00\x04\x00\x00\x00\n\x00\x00\x00T\x00\x00\x00\x97\x00\x00\x00\x97\x00\x00\x00\xc1\x00\x00\x00\x91\x00\x00\x00`\x00\x00\x00\xa1\x00\x00\x00\x98\x00\x00\x00\x87\x00\x00\x00T\x00\x00\x002\x00\x00\x00u\x00\x00\x00!\x00\x00\x00k\x00\x00\x00\x98\x00\x00\x00*\x00\x00\x00\x9d\x00\x00\x00\xb1\x00\x00\x00l\x00\x00\x00\x1b\x00\x00\x00\x7f\x00\x00\x00\xa0\x00\x00\x00O\x00\x00\x00\n\x00\x00\x00B\x00\x00\x00i\x00\x00\x00\x9b\x00\x00\x00,\x00\x00\x00\xbf\x00\x00\x00{\x00\x00\x00&\x00\x00\x001\x00\x00\x00\x82\x00\x00\x00\xc6\x00\x00\x00\xb8\x00\x00\x00\x99\x00\x00\x00v\x00\x00\x00\x82\x00\x00\x00\xa8\x00\x00\x00L\x00\x00\x00\xa9\x00\x00\x00m\x00\x00\x00\xb1\x00\x00\x00y\x00\x00\x00\xac\x00\x00\x00h\x00\x00\x00\x0f\x00\x00\x00\x0f\x00\x00\x00\x00\x00\x00\x00q\x00\x00\x00\x85\x00\x00\x00[\x00\x00\x00H\x00\x00\x00\x94\x00\x00\x00H\x00\x00\x00o\x00\x00\x00\xa6\x00\x00\x00Z\x00\x00\x00\x92\x00\x00\x00\x17\x00\x00\x00k\x00\x00\x001\x00\x00\x00\x98\x00\x00\x00\xa9\x00\x00\x00#\x00\x00\x00\x14\x00\x00\x00\x96\x00\x00\x00\x86\x00\x00\x00\x11\x00\x00\x00=\x00\x00\x00h\x00\x00\x00i\x00\x00\x00\x05\x00\x00\x00h\x00\x00\x00\xad\x00\x00\x00K\x00\x00\x00&\x00\x00\x00\xb8\x00\x00\x00\x82\x00\x00\x00\x0b\x00\x00\x002\x00\x00\x00\x1c\x00\x00\x00l\x00\x00\x00\x92\x00\x00\x00X\x00\x00\x00\xae\x00\x00\x00#\x00\x00\x00\x80\x00\x00\x00i\x00\x00\x000\x00\x00\x00\x18\x00\x00\x00\xb5\x00\x00\x00a\x00\x00\x00v\x00\x00\x00)\x00\x00\x00\x06\x00\x00\x00\x8c\x00\x00\x00\n\x00\x00\x00\x9e\x00\x00\x00z\x00\x00\x00\xa8\x00\x00\x00\xc1\x00\x00\x00\x7f\x00\x00\x00,\x00\x00\x00\x9a\x00\x00\x00\x02\x00\x00\x001\x00\x00\x00D\x00\x00\x00\x90\x00\x00\x00\xb4\x00\x00\x00?\x00\x00\x00?\x00\x00\x000\x00\x00\x00#\x00\x00\x00\xa2\x00\x00\x00M\x00\x00\x00V\x00\x00\x00\x80\x00\x00\x00#\x00\x00\x00\xb8\x00\x00\x00\x1c\x00\x00\x00\x90\x00\x00\x00\xc6\x00\x00\x001\x00\x00\x00\x8d\x00\x00\x00\xbe\x00\x00\x00\t\x00\x00\x00\xc3\x00\x00\x00\x88\x00\x00\x00J\x00\x00\x00w\x00\x00\x00\x06\x00\x00\x00H\x00\x00\x00\x9b\x00\x00\x00\x9c\x00\x00\x00\x98\x00\x00\x00\xb5\x00\x00\x00\x8a\x00\x00\x00`\x00\x00\x00\x8d\x00\x00\x00\xa6\x00\x00\x00m\x00\x00\x00\xaf\x00\x00\x00)\x00\x00\x00]\x00\x00\x00j\x00\x00\x00\x83\x00\x00\x00\x96\x00\x00\x00\xb8\x00\x00\x00\xbe\x00\x00\x00W\x00\x00\x000\x00\x00\x00\x9c\x00\x00\x00\xb3\x00\x00\x00N\x00\x00\x00\xb9\x00\x00\x00B\x00\x00\x00\xa9\x00\x00\x00/\x00\x00\x00\x06\x00\x00\x00\xb2\x00\x00\x000\x00\x00\x00K\x00\x00\x00\x1d\x00\x00\x001\x00\x00\x00=\x00\x00\x00;\x00\x00\x00\x03\x00\x00\x00\xc5\x00\x00\x00\x99\x00\x00\x00R\x00\x00\x00A\x00\x00\x00y\x00\x00\x00\x8a\x00\x00\x00Y\x00\x00\x00.\x00\x00\x00T\x00\x00\x00+\x00\x00\x00\x82\x00\x00\x00\x1e\x00\x00\x00\xc2\x00\x00\x00R\x00\x00\x00F\x00\x00\x00H\x00\x00\x00=\x00\x00\x00\xab\x00\x00\x009\x00\x00\x00u\x00\x00\x00\x10\x00\x00\x00\'\x00\x00\x00\x13\x00\x00\x00\x84\x00\x00\x00\x04\x00\x00\x00\x03\x00\x00\x00$\x00\x00\x00p\x00\x00\x00r\x00\x00\x00\xa1\x00\x00\x00}\x00\x00\x00,\x00\x00\x00p\x00\x00\x00I\x00\x00\x00\x18\x00\x00\x00\x15\x00\x00\x00\xb5\x00\x00\x00\xa7\x00\x00\x00\x1b\x00\x00\x00\x04\x00\x00\x00:\x00\x00\x00\x94\x00\x00\x00\x04\x00\x00\x00|\x00\x00\x00\xae\x00\x00\x00\xc0\x00\x00\x00\x04\x00\x00\x00!\x00\x00\x00M\x00\x00\x002\x00\x00\x00\xa9\x00\x00\x00\x12\x00\x00\x00\x19\x00\x00\x00,\x00\x00\x00X\x00\x00\x00\r\x00\x00\x00\x17\x00\x00\x00\x8c\x00\x00\x00\x8b\x00\x00\x00\xb7\x00\x00\x00s\x00\x00\x00\x8b\x00\x00\x00\x1a\x00\x00\x00^\x00\x00\x00\xa2\x00\x00\x00o\x00\x00\x00U\x00\x00\x00\x15\x00\x00\x00\xb4\x00\x00\x00\n\x00\x00\x00\x9f\x00\x00\x00\r\x00\x00\x00\x03\x00\x00\x00\x7f\x00\x00\x00\x1a\x00\x00\x00i\x00\x00\x00\x0e\x00\x00\x00c\x00\x00\x00\x12\x00\x00\x00\xbb\x00\x00\x00&\x00\x00\x00\x0c\x00\x00\x00\xa0\x00\x00\x00\x0f\x00\x00\x00\xb4\x00\x00\x005\x00\x00\x00Q\x00\x00\x009\x00\x00\x00\x19\x00\x00\x00\x00\x00\x00\x00\xb3\x00\x00\x00\xb0\x00\x00\x00U\x00\x00\x00\xbf\x00\x00\x00\x1a\x00\x00\x00\x10\x00\x00\x00N\x00\x00\x00=\x00\x00\x007\x00\x00\x00e\x00\x00\x00\x98\x00\x00\x00V\x00\x00\x00"\x00\x00\x00\xb0\x00\x00\x00\xc1\x00\x00\x00\x82\x00\x00\x00\x0b\x00\x00\x00H\x00\x00\x002\x00\x00\x00Y\x00\x00\x00\xba\x00\x00\x00\x02\x00\x00\x00~\x00\x00\x00\x16\x00\x00\x00\x84\x00\x00\x00\x01\x00\x00\x00s\x00\x00\x00K\x00\x00\x00\xbe\x00\x00\x00Y\x00\x00\x00\x80\x00\x00\x00v\x00\x00\x00\r\x00\x00\x00a\x00\x00\x003\x00\x00\x00\x0b\x00\x00\x00\x88\x00\x00\x00N\x00\x00\x00A\x00\x00\x00d\x00\x00\x00\x9b\x00\x00\x00\x9d\x00\x00\x00*\x00\x00\x00\xbd\x00\x00\x00\x15\x00\x00\x00\x91\x00\x00\x00\x04\x00\x00\x00d\x00\x00\x00n\x00\x00\x00\xab\x00\x00\x00\x8d\x00\x00\x00\x1e\x00\x00\x00\xb7\x00\x00\x00>\x00\x00\x00N\x00\x00\x00\xc1\x00\x00\x00\xa5\x00\x00\x00\xc6\x00\x00\x00\xb1\x00\x00\x00\x93\x00\x00\x00\x95\x00\x00\x00\xb1\x00\x00\x00Q\x00\x00\x00@\x00\x00\x00\x86\x00\x00\x000\x00\x00\x00\xae\x00\x00\x00w\x00\x00\x00\x9e\x00\x00\x00\x8b\x00\x00\x00\x8c\x00\x00\x00;\x00\x00\x00f\x00\x00\x00t\x00\x00\x00A\x00\x00\x00v\x00\x00\x00|\x00\x00\x00\xa4\x00\x00\x00T\x00\x00\x00(\x00\x00\x00^\x00\x00\x00\x13\x00\x00\x00\x84\x00\x00\x00\x10\x00\x00\x00a\x00\x00\x00&\x00\x00\x00S\x00\x00\x00\xad\x00\x00\x00\x90\x00\x00\x00J\x00\x00\x00\x8d\x00\x00\x00\x0b\x00\x00\x00o\x00\x00\x00\xac\x00\x00\x00k\x00\x00\x00\x9c\x00\x00\x00#\x00\x00\x00\x0f\x00\x00\x00[\x00\x00\x001\x00\x00\x00-\x00\x00\x00\x13\x00\x00\x00%\x00\x00\x00t\x00\x00\x00\x1b\x00\x00\x00\x1e\x00\x00\x00D\x00\x00\x00\x13\x00\x00\x00m\x00\x00\x00\xa8\x00\x00\x00\xc0\x00\x00\x00\xaa\x00\x00\x000\x00\x00\x00\x0e\x00\x00\x00s\x00\x00\x00;\x00\x00\x00\xa6\x00\x00\x00`\x00\x00\x00\x8b\x00\x00\x00\x01\x00\x00\x00\x87\x00\x00\x00\x81\x00\x00\x00\x19\x00\x00\x00M\x00\x00\x00!\x00\x00\x00\x81\x00\x00\x00\x06\x00\x00\x00y\x00\x00\x00\xb7\x00\x00\x00\x0f\x00\x00\x00Z\x00\x00\x00o\x00\x00\x00\x12\x00\x00\x00\r\x00\x00\x00\x13\x00\x00\x00\x97\x00\x00\x00\xa9\x00\x00\x00%\x00\x00\x00 \x00\x00\x00\r\x00\x00\x00\x9c\x00\x00\x00{\x00\x00\x00\xa2\x00\x00\x00\x97\x00\x00\x00\x1a\x00\x00\x00\x85\x00\x00\x005\x00\x00\x00?\x00\x00\x00$\x00\x00\x00|\x00\x00\x00\x13\x00\x00\x00\x0c\x00\x00\x00\x9e\x00\x00\x00+\x00\x00\x00\x81\x00\x00\x00z\x00\x00\x00\x04\x00\x00\x00\x12\x00\x00\x00F\x00\x00\x00\x98\x00\x00\x005\x00\x00\x00z\x00\x00\x00i\x00\x00\x001\x00\x00\x00c\x00\x00\x00\xa5\x00\x00\x00r\x00\x00\x00\x82\x00\x00\x00g\x00\x00\x00\xba\x00\x00\x00\x9f\x00\x00\x00\x92\x00\x00\x00+\x00\x00\x00\xa8\x00\x00\x00Q\x00\x00\x00#\x00\x00\x00\x86\x00\x00\x00\xa2\x00\x00\x00\xb2\x00\x00\x00\n\x00\x00\x00\xb5\x00\x00\x00\xaa\x00\x00\x00\xb9\x00\x00\x00\x92\x00\x00\x00\x80\x00\x00\x00s\x00\x00\x00J\x00\x00\x00\xbf\x00\x00\x00\xbb\x00\x00\x00\xb4\x00\x00\x00\x1d\x00\x00\x00\x05\x00\x00\x00\xb6\x00\x00\x00T\x00\x00\x00\xbe\x00\x00\x00\xb7\x00\x00\x00\x9a\x00\x00\x00!\x00\x00\x00\x90\x00\x00\x00\x9f\x00\x00\x00|\x00\x00\x00/\x00\x00\x00j\x00\x00\x00B\x00\x00\x00\xb5\x00\x00\x00=\x00\x00\x00a\x00\x00\x00\x8d\x00\x00\x00?\x00\x00\x00\x0b\x00\x00\x00\xaf\x00\x00\x00O\x00\x00\x00\x15\x00\x00\x00x\x00\x00\x00$\x00\x00\x00\x93\x00\x00\x00\xc3\x00\x00\x00\xb4\x00\x00\x00\x9c\x00\x00\x00S\x00\x00\x00y\x00\x00\x00\x1e\x00\x00\x00^\x00\x00\x00w\x00\x00\x00A\x00\x00\x00A\x00\x00\x00q\x00\x00\x00\xb2\x00\x00\x00\xac\x00\x00\x00\xbc\x00\x00\x00\x8c\x00\x00\x00\x07\x00\x00\x00#\x00\x00\x00D\x00\x00\x00\x86\x00\x00\x00\xc5\x00\x00\x00j\x00\x00\x00k\x00\x00\x00Y\x00\x00\x00m\x00\x00\x008\x00\x00\x00!\x00\x00\x00_\x00\x00\x00\x91\x00\x00\x00\x8e\x00\x00\x00\x1a\x00\x00\x00\x19\x00\x00\x00#\x00\x00\x00f\x00\x00\x00\xb1\x00\x00\x00<\x00\x00\x00U\x00\x00\x00\x17\x00\x00\x00\x0b\x00\x00\x00\xaa\x00\x00\x00\xad\x00\x00\x00\x96\x00\x00\x00$\x00\x00\x00\x1e\x00\x00\x00\x9c\x00\x00\x00r\x00\x00\x00\x18\x00\x00\x00\x0f\x00\x00\x00\xa7\x00\x00\x00w\x00\x00\x00!\x00\x00\x00\xa8\x00\x00\x00\x06\x00\x00\x00\x04\x00\x00\x00A\x00\x00\x00\xb6\x00\x00\x00t\x00\x00\x00V\x00\x00\x00L\x00\x00\x00\x07\x00\x00\x00\x93\x00\x00\x00v\x00\x00\x009\x00\x00\x00\xae\x00\x00\x00\x01\x00\x00\x00\x14\x00\x00\x002\x00\x00\x008\x00\x00\x00\xac\x00\x00\x00b\x00\x00\x00\x9d\x00\x00\x00\xad\x00\x00\x00\x1a\x00\x00\x00\xa4\x00\x00\x00U\x00\x00\x00\x90\x00\x00\x00j\x00\x00\x00/\x00\x00\x00\xa7\x00\x00\x00h\x00\x00\x00\x9c\x00\x00\x00<\x00\x00\x00\xc4\x00\x00\x00t\x00\x00\x00_\x00\x00\x00"\x00\x00\x00\x7f\x00\x00\x00\x87\x00\x00\x00C\x00\x00\x00\x82\x00\x00\x00k\x00\x00\x00\x01\x00\x00\x00\x0c\x00\x00\x00\x95\x00\x00\x00F\x00\x00\x00\xbb\x00\x00\x00\x9f\x00\x00\x00+\x00\x00\x00\xc6\x00\x00\x00\x9d\x00\x00\x00\xae\x00\x00\x00\xb8\x00\x00\x00\xa8\x00\x00\x00\x9e\x00\x00\x00\x1b\x00\x00\x00\x8f\x00\x00\x00\xb9\x00\x00\x00M\x00\x00\x00\x19\x00\x00\x00n\x00\x00\x00\xae\x00\x00\x00y\x00\x00\x00\x02\x00\x00\x00\x8b\x00\x00\x00\x99\x00\x00\x00\xc7\x00\x00\x00\x1d\x00\x00\x00T\x00\x00\x00:\x00\x00\x00\x05\x00\x00\x00q\rtq\x0eb.')
        
    def FSM_move(self, s, prev_friend_move, prev_foe_move, Astate, states_A, actions_A, history):
        filter_prediction = self.io.history_step(history)
        Astate, friend_move = FSM_agentX(s, prev_friend_move, prev_foe_move, Astate, filter_prediction, states_A, actions_A)
        return(Astate, friend_move)

    def reset(self):
        self.io = iocane_agent()
        self.Gstate = None
        self.Gprev_friend_move = None
        self.F = INDIVIDUAL()
        
    def history_step(self, history):
        trials = 1000
        s = len(history)
    
        if (s == 0):
            self.Gstate = 0
            # make some dummy sizes
            Gprev_foe_move = -1
            self.Gprev_friend_move = -1
        if (s > 0):
            Gprev_foe_move = history[-1]['competitorStep']
        
        self.Gstate, Gfriend_move = self.FSM_move(s, self.Gprev_friend_move, Gprev_foe_move, self.Gstate, self.F.state_choices, self.F.action_choices, history) 
        self.Gprev_friend_move = Gfriend_move
        
        return int(Gfriend_move)

        
agents = [
    transition_matrix(False, False),
    transition_matrix(True, False),
    transition_matrix(False, True),
    transition_matrix(True, True),
    #transition_matrix(False, False, decay = 1.01),
    #transition_matrix(False, True, decay = 1.01),
    #transition_matrix(True, False, decay = 1.01),
    #transition_matrix(True, True, decay = 1.01),
    transition_matrix(False, False, decay = 1.05),
    transition_matrix(False, True, decay = 1.05),
    transition_matrix(True, False, decay = 1.05),
    transition_matrix(True, True, decay = 1.05),
    #transition_matrix(False, False, decay = 1.15),
    #transition_matrix(False, True, decay = 1.15),
    #transition_matrix(True, False, decay = 1.15),
    #transition_matrix(True, True, decay = 1.15),
    #transition_matrix(False, False, decay = 1.25),
    #transition_matrix(False, True, decay = 1.25),
    #transition_matrix(True, False, decay = 1.25),
    #transition_matrix(True, True, decay = 1.25),
    transition_tensor(False, False),
    transition_tensor(True, False),
    transition_tensor(False, True),
    transition_tensor(True, True),
    #transition_tensor(False, False, decay = 1.01),
    #transition_tensor(False, True, decay = 1.01),
    #transition_tensor(True, False, decay = 1.01),
    #transition_tensor(True, True, decay = 1.01),
    transition_tensor(False, False, decay = 1.05),
    transition_tensor(False, True, decay = 1.05),
    transition_tensor(True, False, decay = 1.05),
    transition_tensor(True, True, decay = 1.05),
    #transition_tensor(False, False, decay = 1.15),
    #transition_tensor(False, True, decay = 1.15),
    #transition_tensor(True, False, decay = 1.15),
    #transition_tensor(True, True, decay = 1.15),
    #transition_tensor(False, False, decay = 1.25),
    #transition_tensor(False, True, decay = 1.25),
    #transition_tensor(True, False, decay = 1.25),
    #transition_tensor(True, True, decay = 1.25),
    legion_agent(shift=0, dec=0.75, app=0, mult=2.0, mm=0.4, score=4),
    legion_agent(shift=0, dec=0.75, app=0.7, mult=2.0, mm=0.4, score=4),
    legion_agent(shift=0, dec=0.75, app=0, mult=3.0, mm=0.4, score=4),
    legion_agent(shift=0, dec=0.75, app=0.7, mult=3.0, mm=0.4, score=4),
    centurion_agent(shift=0, dec=0.75, app=0, mult=2.0, mm=0.4, score=4),
    centurion_agent(shift=0, dec=0.75, app=0.7, mult=2.0, mm=0.4, score=4),
    centurion_agent(shift=0, dec=0.75, app=0, mult=3.0, mm=0.4, score=4),
    centurion_agent(shift=0, dec=0.75, app=0.7, mult=3.0, mm=0.4, score=4),
    calculation(False),
    markov(),
    decision_tree(3,25),
    decision_tree(4,25),
    decision_tree(5,25),
    decision_tree(6,25),
    iocane_agent(),
    spinoza_agent(12, True),
    spinoza_agent(20, True),
    spinoza_agent(35, True),
    spinoza_agent(50, True),
    spinoza_agent(70, True),
    IOU2_agent(12),
    IOU2_agent(27),
    IOU2_agent(52),
    IOU2_agent(108),
    xgboost_agent(3,10),
    xgboost_agent(4,15),
    xgboost_agent(5,15),
    xgboost_agent(6,20),
    aggressive_agent(),
    greenberg_agent(),
    ignore_agent(),
    calculation_dllu(False),
    lucker_agent(12, 9),
    lucker_agent(27, 18),
    lucker_agent(54, 36),
    NumpyPatterns(),
    anti_greenberg(),
    anti_decision_tree(),
    anti_ignore(),
    anti_iou(),
    anti_numpy(),
    anti_iocane()
]

num_agents = len(agents)
num_types = 3
total_agents = num_agents
wins = [0]*len(agents)
results = [0]*len(agents)
total_wins = 0
total_losses = 0

history = [[] for _ in range(num_agents)]
agent_type = Mob([],[],[],[],[],[],[],10, 80, {50}, 0.19, 0.95, 1000, False, 0.001)
turns_remaining = 0
last_agent = 0
last_move = 0
hst = [[] for _ in range(num_agents)]
rwd = [[] for _ in range(num_agents)]
    
def run_agents(history):
    global agents, results, num_agents
    for i in range(num_agents):
        results[i] = agents[i].history_step(history[i])

def check_wins(opp, multiplier):
    global wins, results, num_agents, hst, rwd
    for i in range(num_agents):
        hst[i].append(results[i])
        if beats(results[i], opp):
            wins[i] = 0
        elif beats(opp, results[i]):
            wins[i] = 2
        else:
            wins[i] = 1
        w = 2 - wins[i]
        rwd[i].append(w)

def beats(x, y):
    if y == 0:
        if x == 1:
            return True
        else:
            return False
    elif y == 1:
        if x == 2:
            return True
        else:
            return False
    else:
        if x == 0:
            return True
        else:
            return False
        
def mode(x):
    counter = Counter(x)
    max_count = max(counter.values())
    choice = [item for item, count in counter.items() if count == max_count]
    return int(random.choice(choice))

geometry_active = False

def psycho_showdown(observation, configuration):
    #more dakka
    global agents, num_agents, num_types, total_agents, wins, results
    global history, agent_type, shift, transition, tensor, pattern, turns_remaining
    global last_agent, last_move, results
    global total_wins, total_losses, trigger, hst, rwd
    global geometry_active
    
    #Total turns in one rotation
    government = [0,1,2,3,4,5,6,7,8,9]
    #Turns in the "democracy" set: Choose result based on mode of top x agents (x = voters)
    democracy = {1,8}
    #Turns in the "tyranny" set: Choose result of the "luckiest" agent
    tyranny = {2,4,5,7,9}
    #Turns in the "anarchy" set: Random result to throw off opponent's strategy
    anarchy = {0,3,6}
    cycle = len(government)
    voters = 5
    votes = []
    
    def log_step(s = None, agent = None, competitorStep = None):
        global history, num_agents, results
        if s is None:
            s = np.random.randint(3)
        for i in range(num_agents):
            history[i].append({'step': s, 'competitorStep': competitorStep, 'agent': agent})
        return s
    
    def update_competitor_step(competitorStep):
        global history, num_agents
        for i in range(num_agents):
            history[i][-1]['competitorStep'] = int(competitorStep) 
    
    geometry_agent_hand = geometry_agent(observation, configuration)
    if observation.step == 0:
        history = [[] for _ in range(num_agents)]
        agent_type.initialize(num_agents)
        last_agent = agent_type.select_agent()
        run_agents(history)
        last_move = results[last_agent]
        log_step(last_move)
        return last_move
    else:
        check_wins(observation.lastOpponentAction, 1)
        update_competitor_step(observation.lastOpponentAction)
        agent_type.manager(observation.step)
        agent_type.update_sensitivity(wins[last_agent], observation.step)
        for i in range(len(wins)):
            if len(hst[i])>10:
                agent_type.gibbs(hst[i], rwd[i], i)
            history[i] = agent_type.update(i, wins[i], history[i], observation.step)
        agent_type.reset_all(observation.step, observation.reward)
        run_agents(history)
        if (observation.step % cycle) in tyranny:
            last_agent = agent_type.select_agent()
            last_move = int(results[last_agent])
            log_step(last_move)
            
        elif (observation.step % cycle) in democracy:
            last_agent = agent_type.select_agent()
            final = agent_type.select_top(voters)
            for i in range(len(final)):
                votes.append(results[final[i]])
            last_move = mode(votes)
            log_step(last_move)
            
        else:
            last_agent = agent_type.select_agent()
            last_move = random.randint(0,2)
            log_step(last_move)
            
        
        
        if observation.step > 600 and observation.step < 800 and observation.reward < -2:
            geometry_active = True

        if observation.step > 800 and observation.reward < -2 and geometry_active:
            geometry_active = False

        if geometry_active:
            return geometry_agent_hand
        
        return last_move


In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("rps", configuration={"episodeSteps": 1000}, debug=True)
env.run(["submission.py", "iocane.py"])
print(env.render(mode="ansi", width=600, height=600))

In [ ]:
%%writefile iocane.py

import random


def recall(age, hist):
    end, length = 0, 0
    for past in range(1, min(age + 1, len(hist) - 1)):
        if length >= len(hist) - past: break
        for i in range(-1 - length, 0):
            if hist[i - past] != hist[i]: break
        else:
            for length in range(length + 1, len(hist) - past):
                if hist[-past - length - 1] != hist[-length - 1]: break
            else: length += 1
            end = len(hist) - past
    return end

def beat(i):
    return (i + 1) % 3
def loseto(i):
    return (i - 1) % 3

class Stats:
    def __init__(self):
        self.sum = [[0, 0, 0]]
    def add(self, move, score):
        self.sum[-1][move] += score
    def advance(self):
        self.sum.append(self.sum[-1])
    def max(self, age, default, score):
        if age >= len(self.sum): diff = self.sum[-1]
        else: diff = [self.sum[-1][i] - self.sum[-1 - age][i] for i in range(3)]
        m = max(diff)
        if m > score: return diff.index(m), m
        return default, score

class Predictor:
    def __init__(self):
        self.stats = Stats()
        self.lastguess = -1
    def addguess(self, lastmove, guess):
        if lastmove != -1:
            diff = (lastmove - self.prediction) % 3
            self.stats.add(beat(diff), 1)
            self.stats.add(loseto(diff), -1)
            self.stats.advance()
        self.prediction = guess
    def bestguess(self, age, best):
        bestdiff = self.stats.max(age, (best[0] - self.prediction) % 3, best[1])
        return (bestdiff[0] + self.prediction) % 3, bestdiff[1]

ages = [1000, 100, 10, 5, 2, 1]

class Guesser:

    def __init__(self):
        self.predictors = []
        self.predict_history = self.predictor((len(ages), 2, 3))
        self.predict_frequency = self.predictor((len(ages), 2))
        self.predict_fixed = self.predictor()
        self.predict_random = self.predictor()
        self.predict_meta = [Predictor() for a in range(len(ages))]
        self.stats = [Stats() for i in range(2)]
        self.histories = [[], [], []]

    def predictor(self, dims=None):
        if dims: return [self.predictor(dims[1:]) for i in range(dims[0])]
        self.predictors.append(Predictor())
        return self.predictors[-1]

    def move(self, them):

        if them != -1:
            self.histories[1].append(them)
            self.histories[2].append((self.histories[0][-1], them))
            for watch in range(2):
                self.stats[watch].add(self.histories[watch][-1], 1)

        rand = random.randrange(3)
        self.predict_random.addguess(them, rand)
        self.predict_fixed.addguess(them, 0)

        for a, age in enumerate(ages):
            best = [recall(age, hist) for hist in self.histories]
            for mimic in range(2):
                for watch, when in enumerate(best):
                    if not when: move = rand
                    else: move = self.histories[mimic][when]
                    self.predict_history[a][mimic][watch].addguess(them, move)
                mostfreq, score = self.stats[mimic].max(age, rand, -1)
                self.predict_frequency[a][mimic].addguess(them, mostfreq)

        for meta, age in enumerate(ages):
            best = (-1, -1)
            for predictor in self.predictors:
                best = predictor.bestguess(age, best)
            self.predict_meta[meta].addguess(them, best[0])

        best = (-1, -1)
        for meta in range(len(ages)):
            best = self.predict_meta[meta].bestguess(len(self.histories[0]) , best) 
        self.histories[0].append(best[0])

        return best[0]

guesser = None

def guesser_agent(observation, configuration):
    global guesser
    if observation.step == 0:
        guesser = Guesser()
        action = guesser.move(-1)
    else:
        action = guesser.move(observation.lastOpponentAction)
        
    return action

In [ ]:
%%writefile legion.py

import random
import numpy as np
import pandas as pd
import collections
from scipy.stats import beta
from collections import Counter
from collections import defaultdict 

class ThompsonSampling():
    def __init__(self, counts, values, l_counts, scores, waypoints, reset, a, b, losses, gap, timing, sensitivity):
        self.counts = counts
        self.values = values
        self.gap = gap
        self.temp_values = values
        self.temp_counts = counts
        self.losses = losses
        self.loss_counts = l_counts
        self.scores = scores
        self.reset = reset
        self.waypoint = waypoints
        self.sensitivity = sensitivity
        self.timing = timing
        
        #Parameters for beta distributions
        self.a = a
        self.b = b
        return
    
    #Initialize values for agents
    def initialize(self, n_agents):
        self.counts = [0 for col in range(n_agents)]
        self.values = [0.0 for col in range(n_agents)]
        self.temp_values = [0.0 for col in range(n_agents)]
        self.temp_counts = [0 for col in range(n_agents)]
        # Uniform distribution of prior beta (A,B)
        self.a = [1 for agent in range(n_agents)]
        self.b = [1 for agent in range(n_agents)]
        self.loss_counts = [[0 for col in range(self.losses)] for col in range(n_agents)]
        self.scores = [[0.0 for col in range(self.losses)] for col in range(n_agents)]
        self.waypoint = [1000 for col in range(n_agents)]
        self.reset = [False for col in range(n_agents)]
        self.gaps = [self.gap for col in range(n_agents)]
        return
    
    def select_agent(self):
        n_agents = len(self.counts)
        beta_params = zip(self.a, self.b)
        all_results = [beta.rvs(i[0], i[1], size = 1) for i in beta_params]
        return all_results.index(max(all_results))
    
    def select_top(self, n):
        beta_params = zip(self.a, self.b)
        all_results = [beta.rvs(i[0], i[1], size = 1) for i in beta_params]
        top = sorted(range(len(all_results)), key = lambda sub: all_results[sub])[-n:]
        return top
    
    def update(self, chosen_agent, reward):
        x = self.losses-1
        for y in range(x):
            self.loss_counts[chosen_agent][y] = self.loss_counts[chosen_agent][y+1]
            self.scores[chosen_agent][y] = (y/(x+1))*self.loss_counts[chosen_agent][y]
        self.loss_counts[chosen_agent][-1] = reward
        self.scores[chosen_agent][-1] = self.loss_counts[chosen_agent][-1]
        if sum(self.scores[chosen_agent]) < self.sensitivity:
            losing = True
        else:
            losing = False
        self.counts[chosen_agent] = self.counts[chosen_agent] + 1
        n = self.counts[chosen_agent]
        if n in self.timing:
            self.a[chosen_agent] = 1.0
            self.b[chosen_agent] = 1.0
        if n >= self.gaps[chosen_agent] and losing == True and self.reset[chosen_agent] == False:
            self.temp_values[chosen_agent] = 0.0
            self.temp_counts[chosen_agent] = 0
            self.waypoint[chosen_agent] = n
            self.reset[chosen_agent] = True
        if self.reset[chosen_agent] == True:
            #Start recording new value
            self.temp_counts[chosen_agent] = self.temp_counts[chosen_agent] + 1
            n2 = self.temp_counts[chosen_agent]
            temp_value = self.temp_values[chosen_agent]
            temp_value = ((n2 - 1) / float(n2)) * temp_value + (1 / float(n2)) * reward
            self.temp_values[chosen_agent] = temp_value
            if n == self.waypoint[chosen_agent] + self.gaps[chosen_agent]:
                self.values[chosen_agent] = self.temp_values[chosen_agent]
                self.counts[chosen_agent] = self.temp_counts[chosen_agent]
                self.a[chosen_agent] = 1.0
                self.b[chosen_agent] = 1.0
                self.reset[chosen_agent] = False
                self.gaps[chosen_agent] = n + self.gap
        value = self.values[chosen_agent]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.values[chosen_agent] = new_value
        
        # Update a and b
        self.a[chosen_agent] = self.a[chosen_agent] + reward
        self.b[chosen_agent] = self.b[chosen_agent] + (1-reward)
        
        return
    
# base class for all agents, random agent
class agent():
    def initial_step(self):
        return np.random.randint(3)
    
    def history_step(self, history):
        return np.random.randint(3)
    
    def step(self, history):
        if len(history) == 0:
            return int(self.initial_step())
        else:
            return int(self.history_step(history))

class legion_agent(agent):
    def __init__(self, shift=0, mem_range=3, reset_time=9999, reset_score=-1000, score=4):
        #Sequence of my past moves
        self.my_seq = []
        #Sequence of opponent's past moves
        self.opp_seq = []
        #A dictionary of my+opp subsequences, statistics for next opp move, and future predictions
        self.patterns = defaultdict(int)
        #A buffer of historical patterns waiting to check results of future moves
        self.pattern_buffer = []
        #Length of my and opponent subsequences to store in dictionary
        self.my_range = mem_range
        self.opp_range = mem_range
        #Length of time to wait before collecting subsequence data
        self.collect_data = self.my_range + self.opp_range + 2
        #Length of time to wait before making predictions from the dictionary
        self.warmup_time = 8*self.my_range
        #My last action
        self.action = 0
        #Opponent's last action
        self.move = 0
        #Amount to increment result/prediction statistics
        self.score = score
        #Length of time to run the algorithm before resetting the dictionary and starting over
        #Doing this occasionally throws off opponents who have changed strategies
        self.reset_period = reset_time
        #Turn number when the dictionary was reset
        self.waypoint = 0
        #My wins
        self.wins = 0
        #Score when the dictionary was reset
        self.score_waypoint = 0
        #Tolerance level for resets
        self.tolerance = reset_score
        self.shift = shift

    def reset(): 
        self.my_seq = []
        self.opp_seq = []
        self.patterns = defaultdict(int)
        self.pattern_buffer = []
        return

    def process_patterns(mine, opps):
        entry = {}
        pattern = []
        preds = {"future_1":{"rock":0,"paper":0,"scissors":0}, "future_2":{"rock":0,"paper":0,"scissors":0}}
        pattern.extend(mine[-self.my_range-1:-1])
        pattern.extend(opps[-self.opp_range-1:-1])
        strings = [str(x) for x in pattern]
        int_pattern = "". join(strings)
        entry["rock"] = 0
        entry["paper"] = 0
        entry["scissors"] = 0
        entry["predictions"] = preds
        if not int_pattern in self.patterns:
            self.patterns[int_pattern] = entry
        pattern.append(None)
        pattern.append(None)
        self.pattern_buffer.append(pattern)
        return
    
    def update_predictions(move):
        x = self.my_range+self.opp_range
        if len(self.pattern_buffer) <= 4:
            return
        if self.pattern_buffer[-3][x] == None:
            self.pattern_buffer[-3][x] = move
            key = self.pattern_buffer[-3][0:x]
            strings = [str(y) for y in key]
            int_key = "". join(strings)
            if int_key in self.patterns:
                self.patterns[int_key]["predictions"]["future_1"][{0:"rock",1:"paper",2:"scissors"}[self.move]]+=self.score
            self.pattern_buffer[-4][x+1] = move
            key = self.pattern_buffer[-4][0:x]
            strings = [str(y) for y in key]
            int_key = "". join(strings)
            if int_key in self.patterns:
                self.patterns[int_key]["predictions"]["future_2"][{0:"rock",1:"paper",2:"scissors"}[self.move]]+=self.score
        return
        
    def update_values(mine, opps, move):
        x = self.my_range+self.opp_range
        pattern = []
        pattern.extend(mine[-self.my_range-2:-2])
        pattern.extend(opps[-self.opp_range-2:-2])
        strings = [str(y) for y in pattern]
        key = "". join(strings)
        inner_key = {0:"rock",1:"paper",2:"scissors"}[self.move]
        if key in self.patterns and inner_key in self.patterns[key]:
            self.patterns[key][inner_key] += self.score
        return

    def make_prediction(mine, opps):
        matrix = [[0,0,0],[0,0,0],[0,0,0]]
        results = [1,1,1]
        pattern = []
        pattern.extend(mine[-self.my_range-1:-1])
        pattern.extend(opps[-self.opp_range-1:-1])
        strings = [str(y) for y in pattern]
        key = "". join(strings)
        if key in self.patterns:
            matrix[0][0] = self.patterns[key]["rock"]
            matrix[0][1] = self.patterns[key]["paper"]
            matrix[0][2] = self.patterns[key]["scissors"]
        pattern = []
        pattern.extend(mine[-self.my_range-2:-2])
        pattern.extend(opps[-self.opp_range-2:-2])
        strings = [str(y) for y in pattern]
        key = "". join(strings)
        if key in self.patterns:
            matrix[1][0] = self.patterns[key]["predictions"]["future_1"]["rock"]
            matrix[1][1] = self.patterns[key]["predictions"]["future_1"]["paper"]
            matrix[1][2] = self.patterns[key]["predictions"]["future_1"]["scissors"]
        pattern = []
        pattern.extend(mine[-self.my_range-3:-3])
        pattern.extend(opps[-self.opp_range-3:-3])
        strings = [str(y) for y in pattern]
        key = "". join(strings)
        if key in self.patterns:
            matrix[2][0] = self.patterns[key]["predictions"]["future_2"]["rock"]
            matrix[2][1] = self.patterns[key]["predictions"]["future_2"]["paper"]
            matrix[2][2] = self.patterns[key]["predictions"]["future_2"]["scissors"]
        for i in range(3):
            results[i] += matrix[0][i]+matrix[1][i]+matrix[2][i]
        prediction = sum(random.choices(population=[0,1,2], weights=[x / sum(results) for x in results], k=1))
        return (prediction+1)%3

    def beats(x, y):
        if y == 0:
            if x == 1:
                return True
            else:
                return False
        elif y == 1:
            if x == 2:
                return True
            else:
                return False
        else:
            if x == 0:
                return True
            else:
                return False
    
    def history_step(self, history):
        #Reset protocol
        turn = len(history) - self.waypoint
        if len(history) > 0:
            if beats(self.action, history[-1]['competitorStep']) == True:
                self.wins += 1
            elif beats(history[-1]['competitorStep'], self.action) == True:
                self.wins -= 1
      
        if self.wins - self.score_waypoint < self.tolerance:
            reset()
            self.score_waypoint += self.wins
            self.waypoint += turn
            turn = len(history) - self.waypoint
    
        if turn == self.reset_period:
            reset()
            self.waypoint += turn
            turn = len(history) - self.waypoint
    
        if len(history) == 0:
            self.action = random.randint(0,2)
            return self.action
        elif turn <= collect_data:
            self.my_seq.append(self.action)
            self.opp_seq.append(history[-1]['competitorStep'])
            self.action = random.randint(0,2)
            return self.action
        elif turn < self.warmup_time:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            process_patterns(self.my_seq,self.opp_seq)
            update_values(self.my_seq,self.opp_seq,self.move)
            update_predictions(self.move)
            self.action = random.randint(0,2)
            return self.action
        else:
            self.move = history[-1]['competitorStep']
            self.my_seq.append(self.action)
            self.opp_seq.append(self.move)
            process_patterns(self.my_seq,self.opp_seq)
            update_values(self.my_seq,self.opp_seq,self.move)
            update_predictions(self.move)
            self.action = make_prediction(self.my_seq,self.opp_seq)
            return (self.action+self.shift)%3
        
agents = [
    legion_agent(shift=0, mem_range=2, reset_time=334, reset_score=-1000, score=4),
    legion_agent(shift=0, mem_range=3, reset_time=334, reset_score=-1000, score=4),
    legion_agent(shift=0, mem_range=4, reset_time=334, reset_score=-1000, score=4),
    legion_agent(shift=0, mem_range=5, reset_time=334, reset_score=-1000, score=4),
    legion_agent(shift=0, mem_range=6, reset_time=334, reset_score=-1000, score=4),
    legion_agent(shift=0, mem_range=2, reset_time=334, reset_score=-1000, score=9),
    legion_agent(shift=0, mem_range=3, reset_time=334, reset_score=-1000, score=9),
    legion_agent(shift=0, mem_range=4, reset_time=334, reset_score=-1000, score=9),
    legion_agent(shift=0, mem_range=5, reset_time=334, reset_score=-1000, score=9),
    legion_agent(shift=0, mem_range=6, reset_time=334, reset_score=-1000, score=9),
    legion_agent(shift=0, mem_range=2, reset_time=334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=3, reset_time=334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=4, reset_time=334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=5, reset_time=334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=6, reset_time=334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=2, reset_time=334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=3, reset_time=334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=4, reset_time=334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=5, reset_time=334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=6, reset_time=334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=2, reset_time=1334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=3, reset_time=1334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=4, reset_time=1334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=5, reset_time=1334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=6, reset_time=1334, reset_score=-15, score=4),
    legion_agent(shift=0, mem_range=2, reset_time=1334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=3, reset_time=1334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=4, reset_time=1334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=5, reset_time=1334, reset_score=-15, score=9),
    legion_agent(shift=0, mem_range=6, reset_time=1334, reset_score=-15, score=9)
]

num_agents = len(agents)
num_types = 3
total_agents = num_agents
wins = [0]*len(agents)
results = [0]*len(agents)
total_wins = 0
total_losses = 0
trigger = [False,False,False,False,False]

history = []
agent_type = ThompsonSampling([],[],[],[],[],[],[],[],10, 100, {40}, 10)
turns_remaining = 0
last_agent = 0
last_move = 0
    
def run_agents(history):
    global agents, results, num_agents
    for i in range(num_agents):
        results[i] = agents[i].history_step(history)
            
def check_wins(opp, multiplier):
    global wins, results, num_agents
    for i in range(num_agents):
        if beats(results[i], opp):
            wins[i] = 1*multiplier
        else:
            wins[i] = 0

def beats(x, y):
    if y == 0:
        if x == 1:
            return True
        else:
            return False
    elif y == 1:
        if x == 2:
            return True
        else:
            return False
    else:
        if x == 0:
            return True
        else:
            return False
        
def mode(x):
    counter = Counter(x)
    max_count = max(counter.values())
    choice = [item for item, count in counter.items() if count == max_count]
    return int(random.choice(choice))

#Trigger variables for controlling the flow
#[Trigger 1: Max loss, Trigger 2: Current lead, Trigger 3: Recent drop]
trigger_conditions = [
    [-23,-5,1],
    [-19,11,1],
    [-15,15,2],
    [-11,20,5]
]
trigger_conditions2 = [5,-1]
max_loss = 0
max_lead = 0
triggers = 4
turn_save = 0
turn_save2 = 0
trigger_number = 0
trigger_score = 0
trigger_score2 = 0
score_save = 0

def pull_trigger(win, lose, turn):
    global max_loss, max_lead, trigger, trigger_conditions, triggers, turn_save2
    global turn_save, trigger_number, trigger_score, trigger_score2, trigger_conditions2
    global score_save
    
    turn_start = 300
    turn_limit = 600
    turn_limit2 = 800
    turn_limit3 = 900
    turn_gap = 20
    turn_gap2 = 15
    final_buffer = -10
    
    if turn < turn_start:
        return
    else:
        current = win - lose
    if current < max_loss:
        max_loss = current
    if turn == turn_limit2:
        score_save = current
    if turn == turn_limit3:
        if current < score_save + final_buffer:
            trigger[0] = True
            trigger[1] = True
            trigger[2] = True
            trigger[3] = True
            trigger[4] = True
    if trigger[0] == False and trigger[1] == False and trigger[4] == False:
        if turn > turn_limit:
            if current > max_lead:
                max_lead = current
            for i in range(triggers):
                if max_loss < trigger_conditions[i][0]:
                    if max_lead > trigger_conditions[i][1]:
                        if (max_lead - current) > trigger_conditions[i][2]:
                            turn_save = turn
                            trigger_number = i
                            trigger_score = max_lead - current
                            trigger[0] = True
                            trigger[2] = False
                            trigger[4] = True
    else:
        if trigger[1] == False:
            if (turn - turn_save) > turn_gap:
                turn_save2 = turn
                trigger_score2 = current
                trigger[1] = True
                trigger[0] = False
        else:
            if trigger[2] == False:
                if (turn - turn_save2) > turn_gap2:
                    if (current-trigger_score) > trigger_conditions2[0]:
                        trigger[3] = False
                        trigger[2] = True
                    elif (current-trigger_score) < trigger_conditions2[1]:
                        trigger[2] = True
                        trigger[3] = True
                    else:
                        turn_save = turn
                        turn_save2 = turn
                        turn_limit = turn + 100
                        max_loss = 0
                        max_lead = 0
                        trigger[1] = False
                        trigger[0] = False
                        trigger[2] = False
                        trigger[3] = False
                        trigger[4] = False
                
def trigger_agent(observation, configuration):
    return random.randint(0,2)

def legion_agent(observation, configuration):
    #Because none of us is as smart as all of us
    global agents, num_agents, num_types, total_agents, wins, results
    global history, agent_type, shift, transition, tensor, pattern, turns_remaining
    global last_agent, last_move, results
    global total_wins, total_losses, trigger
    
    #Total turns in one rotation
    government = [0,1,2,3,4,5,6,7,8,9]
    #Turns in the "democracy" set: Choose result based on mode of top x agents (x = voters)
    democracy = {6,8}
    #Turns in the "tyranny" set: Choose result of the "luckiest" agent
    tyranny = {0,2,4,5,7,9}
    #Turns in the "anarchy" set: Random result to throw off opponent's strategy
    anarchy = {1,3}
    cycle = len(government)
    voters = 3
    votes = []
    
    def log_step(step = None, agent = None, competitorStep = None):
        global history
        if step is None:
            step = np.random.randint(3)
        history.append({'step': step, 'competitorStep': competitorStep, 'agent': agent})
        return step
    
    def update_competitor_step(competitorStep):
        global history
        history[-1]['competitorStep'] = int(competitorStep) 

    if observation.step == 0:
        history = []
        agent_type.initialize(num_agents)
        last_agent = agent_type.select_agent()
        run_agents(history)
        step = agents[last_agent].step(history)
        last_move = step
        return log_step(step, last_agent)
    else:
        if beats(last_move, observation.lastOpponentAction) == True:
            total_wins += 1
        else:
            total_losses += 1
        pull_trigger(total_wins, total_losses, observation.step)
        if trigger[0] == True or trigger[3] == True:
            return trigger_agent(observation, configuration)
        check_wins(observation.lastOpponentAction, 1)
        update_competitor_step(observation.lastOpponentAction)
        for i in range(len(wins)):
            agent_type.update(i, wins[i])
        run_agents(history)
        if (observation.step % cycle) in tyranny:
            step = agents[last_agent].step(history)
            last_move = step
            return log_step(step, last_agent)
        elif (observation.step % cycle) in democracy:
            last_agent = agent_type.select_agent()
            final = agent_type.select_top(voters)
            for i in range(len(final)):
                votes.append(results[final[i]])
            action = mode(votes)
            log_step(action)
            last_move = action
            return action
        else:
            last_agent = agent_type.select_agent()
            action = random.randint(0,2)
            log_step(action)
            last_move = action
            return action

In [ ]:
%%writefile trans.py
import random
import numpy as np
from collections import defaultdict

#Length of my and opponent subsequences to store in dictionary, and number of tiers
my_range = 3
opp_range = 3
tiers = 3
#Sequence of my past moves
my_seq = []
my_trans = []
#Sequence of opponent's past moves
opp_seq = []
opp_trans = []
#A dictionary of my+opp subsequences, statistics for next opp move, and future predictions
patterns = []
#A buffer of historical patterns waiting to check results of future moves
pattern_buffer = [[] for x in range(tiers)]
#Length of time to wait before collecting subsequence data
collect_data = my_range + opp_range + 2*(tiers) + 2
#Length of time to wait before making predictions from the dictionary
warmup_time = 8*(my_range+tiers)
#My last action
action = 0
#Opponent's last action
move = 0
#Amount to increment result/prediction statistics (starting value)
score = 4
#Decay for past scores (1 = no decay)
decay = 0.75
#Decay for lower tiers
tier_decay = 0.4
#Appreciation for scores (higher means higher weight on later results)
appreciate = 0.2
#Multiplier for scores (0 = purely random, higher means less randomness)
mult = 3.0
#Turns to gather memory data before applying accuracy
mem_turns = 10
#Minimum accuracy to keep pattern memory
min_mem = 0.4
#Score adjustment for inaccurate memories (0 = delete inaccurate memories)
score_penalty = 0.0
    
def beats(x, y):
    if y == 0:
        if x == 1: return True
    if y == 1:
        if x == 2: return True
    else:
        if x == 0: return True
    return False

def mod2(x, y):
    if x == 0:
        if y == 0: return 0
        if y == 1: return 1
        if y == 2: return 2
    if x == 1:
        if y == 0: return 2
        if y == 1: return 0
        if y == 2: return 1
    if x == 2:
        if y == 0: return 1
        if y == 1: return 2
        if y == 2: return 0
    return 0

def process_patterns(mine, opps):
    global pattern_buffer, my_range, opp_range, patterns, tiers
    
    for i in range(tiers):
        entry = {}
        pattern = []
        preds = {"future_1":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}, 
                 "future_2":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}, 
                 "future_3":{"rock":0,"paper":0,"scissors":0,"turns":0,"wins":0,"accuracy":0}}
        pattern.extend(mine[-my_range-1-i:-1])
        pattern.extend(opps[-opp_range-1-i:-1])
        strings = [str(x) for x in pattern]
        int_pattern = "". join(strings)
        entry["predictions"] = preds
        if not int_pattern in patterns[i]:
            patterns[i][int_pattern] = entry
        pattern_buffer[i].append(pattern)
    return
    
def update_predictions(move):
    global pattern_buffer, patterns, score, action, min_mem, mem_turns, tiers
    global score_penalty
    
    futures = ["future_1", "future_2", "future_3"]
    if len(pattern_buffer[0]) <= 4:
        return
    for j in range(tiers):
        for i in range(3):
            key = pattern_buffer[j][-2-i]
            strings = [str(y) for y in key]
            int_key = "". join(strings)
            if int_key in patterns[j] and futures[i] in patterns[j][int_key]["predictions"]:
                patterns[j][int_key]["predictions"][futures[i]][{0:"rock",1:"paper",2:"scissors"}[move]]+=score
                patterns[j][int_key]["predictions"][futures[i]]["turns"]+=1
                if beats(action,move)==True: patterns[j][int_key]["predictions"][futures[i]]["wins"]+=1
                w = patterns[j][int_key]["predictions"][futures[i]]["wins"]
                t = patterns[j][int_key]["predictions"][futures[i]]["turns"]
                patterns[j][int_key]["predictions"][futures[i]]["accuracy"]=w/t
                if patterns[j][int_key]["predictions"][futures[i]]["accuracy"] < min_mem and patterns[j][int_key]["predictions"][futures[i]]["turns"] > mem_turns:
                    patterns[j][int_key]["predictions"][futures[i]]["rock"]*=score_penalty
                    patterns[j][int_key]["predictions"][futures[i]]["paper"]*=score_penalty
                    patterns[j][int_key]["predictions"][futures[i]]["scissors"]*=score_penalty
                    patterns[j][int_key]["predictions"][futures[i]]["turns"]=0
                    patterns[j][int_key]["predictions"][futures[i]]["wins"]=0
                    patterns[j][int_key]["predictions"][futures[i]]["accuracy"]=0
    return

def make_prediction(mine, opps):
    global patterns, decay, tiers, tier_decay
    global my_range, opp_range, score, mult, opp_seq
    
    results = [[1,1,1] for x in range(tiers)]
    futures = ["future_1", "future_2", "future_3"]
    final_results = [0,0,0]
    
    for j in range(tiers):
        matrix = [[0,0,0],[0,0,0],[0,0,0]]
        for i in range(3):
            pattern = []
            pattern.extend(mine[-my_range-1-i-j:-1-i])
            pattern.extend(opps[-opp_range-1-i-j:-1-i])
            strings = [str(y) for y in pattern]
            key = "". join(strings)
            if key in patterns[j]:
                matrix[i][0] = patterns[j][key]["predictions"][futures[i]]["rock"]
                matrix[i][1] = patterns[j][key]["predictions"][futures[i]]["paper"]
                matrix[i][2] = patterns[j][key]["predictions"][futures[i]]["scissors"]
        for i in range(3):
            results[j][i] += (matrix[0][i])**mult+decay*(matrix[1][i]**mult)+(decay**2)*(matrix[2][i]**mult)
    for i in range(3):
        for j in range(tiers):
            final_results[i] += (tier_decay**j)*results[j][i]
    prediction = sum(random.choices(population=[0,1,2], weights=[x / sum(final_results) for x in final_results], k=1))
    return (((opp_seq[-1]+prediction)%3)+1)%3
    
def logic_agent(observation, configuration):
    global my_seq, opp_seq, patterns, action, move, score, my_trans, opp_trans
    global correct, my_range, opp_range, warmup_time
    global collect_data, waypoint, reset_period, appreciate
    
    score += appreciate
    
    if observation.step == 0:
        for i in range(tiers):
            patterns.append(defaultdict(int))
        action = random.randint(0,2)
        return action
    elif observation.step == 1:
        my_seq.append(action)
        opp_seq.append(observation.lastOpponentAction)
        action = random.randint(0,2)
        return action
    elif observation.step <= collect_data:
        my_seq.append(action)
        opp_seq.append(observation.lastOpponentAction)
        my_trans.append(mod2(my_seq[-2],my_seq[-1]))
        opp_trans.append(mod2(opp_seq[-2],opp_seq[-1]))
        action = random.randint(0,2)
        return action
    elif observation.step < warmup_time:
        move = observation.lastOpponentAction
        my_seq.append(action)
        opp_seq.append(move)
        my_trans.append(mod2(my_seq[-2],my_seq[-1]))
        opp_trans.append(mod2(opp_seq[-2],opp_seq[-1]))
        process_patterns(my_trans,opp_trans)
        update_predictions(opp_trans[-1])
        action = random.randint(0,2)
        return action
    else:
        move = observation.lastOpponentAction
        my_seq.append(action)
        opp_seq.append(move)
        my_trans.append(mod2(my_seq[-2],my_seq[-1]))
        opp_trans.append(mod2(opp_seq[-2],opp_seq[-1]))
        process_patterns(my_trans,opp_trans)
        update_predictions(opp_trans[-1])
        action = make_prediction(my_trans,opp_trans)
        return action

In [ ]:
%%writefile iou.py

import random

# 2 different lengths of history, 3 kinds of history, both, mine, yours
# 3 different limit length of reverse learning
# 6 kinds of strategy based on Iocaine Powder
num_predictor = 27
len_rfind = [20]
limit = [10,20,60]
beat = { "R":"P" , "P":"S", "S":"R"}
not_lose = { "R":"PPR" , "P":"SSP" , "S":"RRS" } #50-50 chance
my_his   =""
your_his =""
both_his =""
list_predictor = [""]*num_predictor
length = 0
temp1 = { "PP":"1" , "PR":"2" , "PS":"3",
              "RP":"4" , "RR":"5", "RS":"6",
              "SP":"7" , "SR":"8", "SS":"9"}
temp2 = { "1":"PP","2":"PR","3":"PS",
                "4":"RP","5":"RR","6":"RS",
                "7":"SP","8":"SR","9":"SS"} 
who_win = { "PP": 0, "PR":1 , "PS":-1,
                "RP": -1,"RR":0, "RS":1,
                "SP": 1, "SR":-1, "SS":0}
score_predictor = [0]*num_predictor
output = random.choice("RPS")
predictors = [output]*num_predictor

def iou_agent(observation, configuration):
    global num_predictor, len_rfind, limit, beat, not_lose, my_his, your_his, both_his, list_predictor, length, temp1, temp2
    global who_win, score_predictor, output, predictors

    if observation.step == 0:
        output = random.choice("RPS")
        return {"R":0,"P":1,"S":2}[output]
    if len(list_predictor[0])<5:
        front =0
    else:
        front =1
    print({0:"R",1:"P",2:"S"}[observation.lastOpponentAction])
    for i in range (num_predictor):
        if predictors[i]=={0:"R",1:"P",2:"S"}[observation.lastOpponentAction]
            result ="1"
        else:
            result ="0"
        list_predictor[i] = list_predictor[i][front:5]+result #only 5 rounds before
    #history matching 1-6
    my_his += output
    your_his += {0:"R",1:"P",2:"S"}[observation.lastOpponentAction]
    both_his += temp1[{0:"R",1:"P",2:"S"}[observation.lastOpponentAction]+output]
    length +=1
    for i in range(1):
        len_size = min(length,len_rfind[i])
        j=len_size
        #both_his
        while j>=1 and not both_his[length-j:length] in both_his[0:length-1]:
            j-=1
        if j>=1:
            k = both_his.rfind(both_his[length-j:length],0,length-1)
            predictors[0+6*i] = your_his[j+k]
            predictors[1+6*i] = beat[my_his[j+k]]
        else:
            predictors[0+6*i] = random.choice("RPS")
            predictors[1+6*i] = random.choice("RPS")
        j=len_size
        #your_his
        while j>=1 and not your_his[length-j:length] in your_his[0:length-1]:
            j-=1
        if j>=1:
            k = your_his.rfind(your_his[length-j:length],0,length-1)
            predictors[2+6*i] = your_his[j+k]
            predictors[3+6*i] = beat[my_his[j+k]]
        else:
            predictors[2+6*i] = random.choice("RPS")
            predictors[3+6*i] = random.choice("RPS")
        j=len_size
        #my_his
        while j>=1 and not my_his[length-j:length] in my_his[0:length-1]:
            j-=1
        if j>=1:
            k = my_his.rfind(my_his[length-j:length],0,length-1)
            predictors[4+6*i] = your_his[j+k]
            predictors[5+6*i] = beat[my_his[j+k]]
        else:
            predictors[4+6*i] = random.choice("RPS")
            predictors[5+6*i] = random.choice("RPS")

    for i in range(3):
        temp =""
        search = temp1[output+{0:"R",1:"P",2:"S"}[observation.lastOpponentAction]] #last round
        for start in range(2, min(limit[i],length) ):
            if search == both_his[length-start]:
                temp+=both_his[length-start+1]
        if(temp==""):
            predictors[6+i] = random.choice("RPS")
        else:
            collectR = {"P":0,"R":0,"S":0} #take win/lose from opponent into account
            for sdf in temp:
                next_move = temp2[sdf]
                if(who_win[next_move]==-1):
                    collectR[temp2[sdf][1]]+=3
                elif(who_win[next_move]==0):
                    collectR[temp2[sdf][1]]+=1
                elif(who_win[next_move]==1):
                    collectR[beat[temp2[sdf][0]]]+=1
            max1 = -1
            p1 =""
            for key in collectR:
                if(collectR[key]>max1):
                    max1 = collectR[key]
                    p1 += key
            predictors[6+i] = random.choice(p1)
    
    #rotate 9-27:
    for i in range(9,27):
        predictors[i] = beat[beat[predictors[i-9]]]
        
    #choose a predictor
    len_his = len(list_predictor[0])
    for i in range(num_predictor):
        sum = 0
        for j in range(len_his):
            if list_predictor[i][j]=="1":
                sum+=(j+1)*(j+1)
            else:
                sum-=(j+1)*(j+1)
        score_predictor[i] = sum
    max_score = max(score_predictor)
    if max_score>0:
        predict = predictors[score_predictor.index(max_score)]
    else:
        predict = random.choice(your_his)
    output = {"R":0,"P":1,"S":2}[random.choice(not_lose[predict])]